# **NEW JERSEY - CITY BIKE WRAPPER**

In [5]:
from __future__ import annotations

from pathlib import Path
from typing import Optional, Union, Dict, Any, List
import json

import pandas as pd
import numpy as np


def compute_nj_docked_onecall_utilities(
    *,
    # ✅ ONLY INPUTS YOU WANT
    station_status_txt: Union[str, Path],
    station_information_csv: Optional[Union[str, Path]] = None,
    station_information_url: str = "https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json",
    trip_csv: Optional[Union[str, Path]] = None,
    trip_csv_already_has_blocks: bool = False,

    # time filters (keep as inputs because they change each run)
    station_time_start: Optional[Union[str, pd.Timestamp]] = None,  # inclusive
    station_time_end: Optional[Union[str, pd.Timestamp]] = None,    # exclusive
    trip_time_start: Optional[Union[str, pd.Timestamp]] = None,     # inclusive
    trip_time_end: Optional[Union[str, pd.Timestamp]] = None,       # inclusive

    # output folder (relative ok)
    output_dir: Union[str, Path] = "NJ_OneCall",

    # switches
    normalize: bool = True,
    compute_safety: bool = True,
    compute_idle: bool = True,
    fill_missing_blocks_with_census_api: bool = True,
    reuse_if_exists: bool = True,
    save_outputs: bool = True,
) -> Dict[str, Any]:
    """
    NJ Docked "one-call" pipeline.

    ✅ Only external inputs:
      - station_status_txt (json-lines)
      - station_information_csv OR station_information_url
      - trip_csv (optional, enables usage + idle time)

    🔒 Everything else is hardcoded to NJ paths inside this function.
    """

    # ============================================================
    # 🔒 HARD-CODED NJ PATHS (change once here, never in calls)
    # ============================================================
    NJ_CENSUS_BLOCKS_SHP = r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NJ\tl_2024_34_tabblock20.shp"
    NJ_CENTROID_TRACT_CSV = r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv"
    NJ_CENTERLINE_SHP = r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\Tran_road.shp"
    NJ_BIKE_LANES_SHP = r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp"

    # ============================================================
    # Assumed schema (matches what you used)
    # ============================================================
    station_id_col = "station_id"
    bikes_col = "num_bikes_available"
    ebikes_col = "num_ebikes_available"
    docks_col = "num_docks_available"

    si_station_id_col = "station_id"
    si_lat_col = "lat"
    si_lon_col = "lon"
    si_capacity_col = "capacity"

    trip_start_lat_col = "start_lat"
    trip_start_lon_col = "start_lng"
    trip_end_lat_col = "end_lat"
    trip_end_lon_col = "end_lng"
    trip_started_at_col = "started_at"
    trip_ended_at_col = "ended_at"
    trip_start_block_col = "start_census_block"
    trip_end_block_col = "end_census_block"

    blocks_geoid_col = "GEOID20"
    tract_digits = 11

    block_time_granularity = "5min"
    tract_time_granularity = "1H"

    drop_station_status_cols_if_present = [
        "num_scooters_available",
        "num_scooters_unavailable",
        "is_installed",
        "is_returning",
        "is_renting",
    ]

    # Census API params (your exact style)
    census_benchmark = "Public_AR_Census2020"
    census_vintage = "2020"
    trip_latlon_round_decimals = 6

    # Safety params
    safety_working_epsg = 2263
    bike_lane_type_col = "type"
    protected_bike_lane_value = "PROTECTED BIKE LANE"

    # ============================================================
    # Setup outputs
    # ============================================================
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    p_station_status_raw = output_dir / "nj_station_status_raw.csv"
    p_station_info_raw = output_dir / "nj_station_information_raw.csv"
    p_station_info_done = output_dir / "nj_station_information_done.csv"
    p_station_status_done = output_dir / "nj_station_status_done.csv"

    p_avail_block_5min = output_dir / "availability_block_5min_nj.csv"
    p_avail_hourly_tract = output_dir / "availability_hourly_tract_nj.csv"
    p_avail_norm_hourly_tract = output_dir / "availability_norm_hourly_tract_nj.csv"

    p_capacity_block = output_dir / "capacity_block_nj.csv"
    p_capacity_tract = output_dir / "capacity_tract_nj.csv"
    p_capacity_norm_tract = output_dir / "capacity_norm_tract_nj.csv"

    p_usage_5min_block = output_dir / "usage_5min_block_nj.csv"
    p_usage_hourly_block = output_dir / "usage_hourly_block_nj.csv"
    p_usage_hourly_tract = output_dir / "usage_hourly_tract_nj.csv"
    p_usage_norm_hourly_tract = output_dir / "usage_norm_hourly_tract_nj.csv"

    p_idle_block_hourly = output_dir / "idle_time_block_hourly_nj.csv"
    p_idle_tract_hourly = output_dir / "idle_time_tract_hourly_nj.csv"
    p_idle_norm_tract_hourly = output_dir / "idle_time_norm_tract_hourly_nj.csv"

    p_safety_block = output_dir / "safety_block_bike_lane_nj.csv"
    p_safety_tract = output_dir / "safety_bike_lane_tract_nj.csv"
    p_safety_norm_tract = output_dir / "safety_bike_lane_norm_tract_nj.csv"

    # ============================================================
    # Helpers
    # ============================================================
    def _minmax_norm(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
        out = df.copy()
        for c in cols:
            mn = out[c].min()
            mx = out[c].max()
            if pd.isna(mn) or pd.isna(mx) or mx == mn:
                out[c + "_norm"] = 0.0
            else:
                out[c + "_norm"] = (out[c] - mn) / (mx - mn)
            out[c + "_norm"] = out[c + "_norm"].round(5)
        return out

    def _load_centroids() -> pd.DataFrame:
        ct = pd.read_csv(NJ_CENTROID_TRACT_CSV, dtype={"census_tract": str})
        if "county_name" not in ct.columns:
            ct["county_name"] = np.nan
        ct["census_tract"] = ct["census_tract"].astype(str)
        return ct[["census_tract", "county_name"]].drop_duplicates()

    # ============================================================
    # A) Parse station status txt -> raw df
    # ============================================================
    if reuse_if_exists and p_station_status_raw.exists():
        station_status_raw = pd.read_csv(p_station_status_raw)
        station_status_raw["timestamp"] = pd.to_datetime(station_status_raw["timestamp"], errors="coerce")
    else:
        station_status_txt = Path(station_status_txt)
        with station_status_txt.open("r", encoding="utf-8") as f:
            lines = f.readlines()

        rows: List[dict] = []
        for line in lines:
            if not line.strip():
                continue
            try:
                blob = json.loads(line)
            except json.JSONDecodeError:
                continue

            ts = list(blob.keys())[0]
            entries = blob[ts]

            for entry in entries:
                # flatten vehicle_types_available if present
                if isinstance(entry, dict) and "vehicle_types_available" in entry:
                    vta = entry.get("vehicle_types_available", [])
                    if isinstance(vta, list):
                        for vt in vta:
                            try:
                                vid = vt["vehicle_type_id"]
                                cnt = vt["count"]
                                entry[f"vehicle_type_{vid}_count"] = cnt
                            except Exception:
                                pass
                    entry.pop("vehicle_types_available", None)

                entry["timestamp"] = ts
                rows.append(entry)

        station_status_raw = pd.DataFrame(rows)

        # remove " EDT" then parse
        station_status_raw["timestamp"] = station_status_raw["timestamp"].astype(str).str.replace(" EDT", "", regex=False)
        station_status_raw["timestamp"] = pd.to_datetime(station_status_raw["timestamp"], errors="coerce")

        # drop noisy columns if present
        drop_now = [c for c in drop_station_status_cols_if_present if c in station_status_raw.columns]
        if drop_now:
            station_status_raw = station_status_raw.drop(columns=drop_now)

        if save_outputs:
            station_status_raw.to_csv(p_station_status_raw, index=False)

    if station_status_raw["timestamp"].isna().any():
        raise ValueError("Station status has unparseable timestamps (NaT). Fix the raw txt timestamps first.")

    # filter station status window
    sts = station_status_raw.copy()
    st_start = pd.to_datetime(station_time_start) if station_time_start is not None else None
    st_end = pd.to_datetime(station_time_end) if station_time_end is not None else None
    if st_start is not None:
        sts = sts[sts["timestamp"] >= st_start].copy()
    if st_end is not None:
        sts = sts[sts["timestamp"] < st_end].copy()
    if sts.empty:
        raise ValueError("Station status is empty after station_time_start/station_time_end filtering.")

    # ============================================================
    # B) Station info (csv or GBFS)
    # ============================================================
    if reuse_if_exists and p_station_info_raw.exists():
        df_si_raw = pd.read_csv(p_station_info_raw)
    else:
        if station_information_csv is not None:
            df_si_raw = pd.read_csv(station_information_csv)
        else:
            import requests
            r = requests.get(station_information_url, timeout=60)
            r.raise_for_status()
            js = r.json()
            df_si_raw = pd.DataFrame(js["data"]["stations"])

        if save_outputs:
            df_si_raw.to_csv(p_station_info_raw, index=False)

    for c in [si_station_id_col, si_lat_col, si_lon_col, si_capacity_col]:
        if c not in df_si_raw.columns:
            raise ValueError(f"Station information missing required column '{c}'.")

    # ============================================================
    # C) Geocode station lat/lon -> blocks/tracts (spatial join + API fallback)
    # ============================================================
    if reuse_if_exists and p_station_info_done.exists():
        df_si_done = pd.read_csv(p_station_info_done, dtype={"census_block": str})
        df_si_done["census_block"] = df_si_done["census_block"].astype(str)
        df_si_done["census_tract"] = df_si_done["census_block"].astype(str).str[:tract_digits]
    else:
        import geopandas as gpd
        from shapely.geometry import Point

        blocks = gpd.read_file(NJ_CENSUS_BLOCKS_SHP).to_crs(epsg=4326)
        if blocks_geoid_col not in blocks.columns:
            raise ValueError(f"NJ block shapefile missing '{blocks_geoid_col}'.")

        latlon_df = df_si_raw.drop_duplicates(subset=[si_lat_col, si_lon_col])[[si_lat_col, si_lon_col]].copy()
        latlon_df = latlon_df.rename(columns={si_lat_col: "lat", si_lon_col: "lon"})
        pts = gpd.GeoDataFrame(
            latlon_df,
            geometry=[Point(xy) for xy in zip(latlon_df["lon"], latlon_df["lat"])],
            crs="EPSG:4326",
        )

        joined = gpd.sjoin(pts, blocks[[blocks_geoid_col, "geometry"]], how="left", predicate="within")
        joined = joined.rename(columns={blocks_geoid_col: "census_block"})
        joined["census_block"] = joined["census_block"].astype(str)
        joined["census_tract"] = joined["census_block"].astype(str).str[:-4]

        latlon_blocks = joined[["lat", "lon", "census_block", "census_tract"]].copy()

        if fill_missing_blocks_with_census_api:
            import requests

            missing = latlon_blocks[latlon_blocks["census_block"].isin(["nan", "None"])][["lat", "lon"]].drop_duplicates()
            if not missing.empty:
                def _get_block(lat: float, lon: float) -> Optional[str]:
                    try:
                        url = (
                            "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
                            f"?x={lon}&y={lat}&benchmark={census_benchmark}&vintage={census_vintage}&format=json"
                        )
                        resp = requests.get(url, timeout=30)
                        resp.raise_for_status()
                        js = resp.json()
                        geos = js.get("result", {}).get("geographies", {})
                        blocks_list = geos.get("Census Blocks", []) or geos.get("2020 Census Blocks", [])
                        if blocks_list:
                            return blocks_list[0].get("GEOID")
                        return None
                    except Exception:
                        return None

                try:
                    from tqdm import tqdm
                    tqdm.pandas()
                    missing["census_block_new"] = missing.progress_apply(lambda r: _get_block(r["lat"], r["lon"]), axis=1)
                except Exception:
                    missing["census_block_new"] = missing.apply(lambda r: _get_block(r["lat"], r["lon"]), axis=1)

                missing["census_tract_new"] = missing["census_block_new"].apply(
                    lambda x: str(x)[:-4] if pd.notna(x) and str(x) not in {"nan", "None"} else None
                )

                latlon_blocks = latlon_blocks.merge(missing, on=["lat", "lon"], how="left")
                latlon_blocks["census_block"] = latlon_blocks["census_block"].replace({"nan": np.nan, "None": np.nan})
                latlon_blocks["census_tract"] = latlon_blocks["census_tract"].replace({"nan": np.nan, "None": np.nan})
                latlon_blocks["census_block"] = latlon_blocks["census_block"].fillna(latlon_blocks["census_block_new"])
                latlon_blocks["census_tract"] = latlon_blocks["census_tract"].fillna(latlon_blocks["census_tract_new"])
                latlon_blocks = latlon_blocks.drop(columns=["census_block_new", "census_tract_new"])

        df_si_done = df_si_raw.merge(
            latlon_blocks,
            left_on=[si_lat_col, si_lon_col],
            right_on=["lat", "lon"],
            how="left",
        ).drop(columns=["lat", "lon"])

        df_si_done["census_block"] = df_si_done["census_block"].astype(str)
        df_si_done["census_tract"] = df_si_done["census_block"].astype(str).str[:-4]

        if save_outputs:
            df_si_done.to_csv(p_station_info_done, index=False)

    # ============================================================
    # D) Merge station status + census_block
    # ============================================================
    if reuse_if_exists and p_station_status_done.exists():
        status_done = pd.read_csv(p_station_status_done, dtype={"census_block": str, "census_tract": str})
        status_done["timestamp"] = pd.to_datetime(status_done["timestamp"], errors="coerce")
    else:
        if station_id_col not in sts.columns:
            raise ValueError(f"Station status missing '{station_id_col}'.")
        status_done = sts.merge(
            df_si_done[[si_station_id_col, "census_block"]],
            left_on=station_id_col,
            right_on=si_station_id_col,
            how="left",
        ).drop(columns=[si_station_id_col])

        status_done["census_block"] = status_done["census_block"].astype(str)
        status_done["census_tract"] = status_done["census_block"].astype(str).str[:-4]
        status_done = status_done.sort_values(["timestamp", "census_block"]).reset_index(drop=True)

        if save_outputs:
            status_done.to_csv(p_station_status_done, index=False)

    # ============================================================
    # E) Availability (block 5-min + tract hourly avg + norm)
    # ============================================================
    for c in [bikes_col, ebikes_col, docks_col]:
        if c not in status_done.columns:
            raise ValueError(f"Status done missing availability column '{c}'.")

    status_done["time_slot"] = pd.to_datetime(status_done["timestamp"]).dt.floor(block_time_granularity)
    status_done["total_vehicle_available"] = status_done[bikes_col] + status_done[ebikes_col]

    availability_block_5min = (
        status_done.groupby(["census_block", "time_slot"], as_index=False)[
            [bikes_col, ebikes_col, docks_col, "total_vehicle_available"]
        ].sum()
        .sort_values(["census_block", "time_slot"])
        .reset_index(drop=True)
    )

    tract_5min = (
        status_done.groupby(["census_tract", "timestamp"], as_index=False)[
            [bikes_col, ebikes_col, docks_col, "total_vehicle_available"]
        ].sum()
    )
    tract_5min["time_slot_hour"] = pd.to_datetime(tract_5min["timestamp"]).dt.floor(tract_time_granularity)

    availability_hourly_tract = (
        tract_5min.groupby(["census_tract", "time_slot_hour"], as_index=False)[
            [bikes_col, ebikes_col, docks_col, "total_vehicle_available"]
        ]
        .mean()
        .round(0)
        .astype({bikes_col: int, ebikes_col: int, docks_col: int, "total_vehicle_available": int})
        .rename(columns={"time_slot_hour": "time_slot"})
        .sort_values(["census_tract", "time_slot"])
        .reset_index(drop=True)
    )

    availability_norm_hourly_tract = (
        _minmax_norm(availability_hourly_tract, [bikes_col, ebikes_col, docks_col, "total_vehicle_available"])
        if normalize
        else availability_hourly_tract.copy()
    )

    centroids = _load_centroids()
    availability_norm_hourly_tract = availability_norm_hourly_tract.merge(centroids, on="census_tract", how="left")

    # ============================================================
    # F) Capacity (block + tract + norm)
    # ============================================================
    capacity_block = (
        df_si_done.groupby("census_block", as_index=False)
        .agg(total_capacity=(si_capacity_col, "sum"), num_station=(si_station_id_col, "count"))
    )

    cap_tract = (
        df_si_done.groupby("census_tract", as_index=False)
        .agg(total_capacity=(si_capacity_col, "sum"), num_station=(si_station_id_col, "count"))
    )
    capacity_tract = centroids.merge(cap_tract, on="census_tract", how="left")
    capacity_tract[["total_capacity", "num_station"]] = capacity_tract[["total_capacity", "num_station"]].fillna(0)

    capacity_norm_tract = _minmax_norm(capacity_tract, ["total_capacity", "num_station"]) if normalize else capacity_tract.copy()

    # ============================================================
    # G) Usage (optional) + Idle time (optional)
    # ============================================================
    usage_5min_block = None
    usage_hourly_block = None
    usage_hourly_tract = None
    usage_norm_hourly_tract = None

    idle_time_block_hourly = None
    idle_time_tract_hourly = None
    idle_time_norm_tract_hourly = None

    if trip_csv is not None:
        trip_df = pd.read_csv(trip_csv, low_memory=False)

        trip_df[trip_started_at_col] = pd.to_datetime(trip_df[trip_started_at_col], errors="coerce")
        trip_df[trip_ended_at_col] = pd.to_datetime(trip_df[trip_ended_at_col], errors="coerce")
        trip_df = trip_df.dropna(subset=[trip_started_at_col, trip_ended_at_col]).copy()

        t_start = pd.to_datetime(trip_time_start) if trip_time_start is not None else trip_df[trip_started_at_col].min()
        t_end = pd.to_datetime(trip_time_end) if trip_time_end is not None else trip_df[trip_started_at_col].max()
        trip_df = trip_df[(trip_df[trip_started_at_col] >= t_start) & (trip_df[trip_started_at_col] <= t_end)].copy()

        trip_df["trip_duration_min"] = (trip_df[trip_ended_at_col] - trip_df[trip_started_at_col]).dt.total_seconds() / 60.0
        trip_df = trip_df[trip_df["trip_duration_min"] <= 240].copy()

        # geocode to census block if needed
        if not trip_csv_already_has_blocks:
            needed = {trip_start_lat_col, trip_start_lon_col, trip_end_lat_col, trip_end_lon_col}
            miss = needed - set(trip_df.columns)
            if miss:
                raise ValueError(f"Trip CSV missing coord columns needed for geocoding: {miss}")

            import geopandas as gpd
            from shapely.geometry import Point
            import requests

            blocks_gdf = gpd.read_file(NJ_CENSUS_BLOCKS_SHP).to_crs(epsg=4326)
            blocks_gdf[blocks_geoid_col] = blocks_gdf[blocks_geoid_col].astype(str)

            start_coords = trip_df[[trip_start_lat_col, trip_start_lon_col]].dropna().drop_duplicates().copy()
            end_coords = trip_df[[trip_end_lat_col, trip_end_lon_col]].dropna().drop_duplicates().copy()

            start_coords["geometry"] = [Point(xy) for xy in zip(start_coords[trip_start_lon_col], start_coords[trip_start_lat_col])]
            end_coords["geometry"] = [Point(xy) for xy in zip(end_coords[trip_end_lon_col], end_coords[trip_end_lat_col])]

            start_gdf = gpd.GeoDataFrame(start_coords, geometry="geometry", crs="EPSG:4326")
            end_gdf = gpd.GeoDataFrame(end_coords, geometry="geometry", crs="EPSG:4326")

            start_join = gpd.sjoin(start_gdf, blocks_gdf[[blocks_geoid_col, "geometry"]], how="left", predicate="intersects")
            end_join = gpd.sjoin(end_gdf, blocks_gdf[[blocks_geoid_col, "geometry"]], how="left", predicate="intersects")

            start_map = start_join[[trip_start_lat_col, trip_start_lon_col, blocks_geoid_col]].rename(columns={blocks_geoid_col: trip_start_block_col})
            end_map = end_join[[trip_end_lat_col, trip_end_lon_col, blocks_geoid_col]].rename(columns={blocks_geoid_col: trip_end_block_col})

            trip_df = trip_df.merge(start_map, on=[trip_start_lat_col, trip_start_lon_col], how="left")
            trip_df = trip_df.merge(end_map, on=[trip_end_lat_col, trip_end_lon_col], how="left")

            if fill_missing_blocks_with_census_api:
                def _get_block(lat: float, lon: float) -> Optional[str]:
                    try:
                        url = (
                            "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
                            f"?x={lon}&y={lat}&benchmark={census_benchmark}&vintage={census_vintage}&format=json"
                        )
                        resp = requests.get(url, timeout=10)
                        resp.raise_for_status()
                        js = resp.json()
                        geos = js.get("result", {}).get("geographies", {})
                        blocks_list = geos.get("Census Blocks", []) or geos.get("2020 Census Blocks", [])
                        if blocks_list:
                            return blocks_list[0].get("GEOID")
                        return None
                    except Exception:
                        return None

                miss_start = trip_df[trip_df[trip_start_block_col].isna()][[trip_start_lat_col, trip_start_lon_col]].dropna().copy()
                miss_end = trip_df[trip_df[trip_end_block_col].isna()][[trip_end_lat_col, trip_end_lon_col]].dropna().copy()
                miss_start = miss_start.rename(columns={trip_start_lat_col: "lat", trip_start_lon_col: "lon"})
                miss_end = miss_end.rename(columns={trip_end_lat_col: "lat", trip_end_lon_col: "lon"})
                miss = pd.concat([miss_start, miss_end], ignore_index=True).drop_duplicates().reset_index(drop=True)

                miss["lat"] = miss["lat"].round(trip_latlon_round_decimals).astype(float)
                miss["lon"] = miss["lon"].round(trip_latlon_round_decimals).astype(float)

                try:
                    from tqdm import tqdm
                    tqdm.pandas()
                    miss["census_block"] = miss.progress_apply(lambda r: _get_block(r["lat"], r["lon"]), axis=1)
                except Exception:
                    miss["census_block"] = miss.apply(lambda r: _get_block(r["lat"], r["lon"]), axis=1)

                miss = miss.dropna(subset=["census_block"]).copy()

                trip_df[trip_start_lat_col] = trip_df[trip_start_lat_col].round(trip_latlon_round_decimals).astype(float)
                trip_df[trip_start_lon_col] = trip_df[trip_start_lon_col].round(trip_latlon_round_decimals).astype(float)
                trip_df = trip_df.merge(
                    miss.rename(columns={"lat": trip_start_lat_col, "lon": trip_start_lon_col, "census_block": trip_start_block_col + "_new"}),
                    on=[trip_start_lat_col, trip_start_lon_col],
                    how="left",
                )
                trip_df[trip_start_block_col] = trip_df[trip_start_block_col].fillna(trip_df[trip_start_block_col + "_new"])
                trip_df = trip_df.drop(columns=[trip_start_block_col + "_new"])

                trip_df[trip_end_lat_col] = trip_df[trip_end_lat_col].round(trip_latlon_round_decimals).astype(float)
                trip_df[trip_end_lon_col] = trip_df[trip_end_lon_col].round(trip_latlon_round_decimals).astype(float)
                trip_df = trip_df.merge(
                    miss.rename(columns={"lat": trip_end_lat_col, "lon": trip_end_lon_col, "census_block": trip_end_block_col + "_new"}),
                    on=[trip_end_lat_col, trip_end_lon_col],
                    how="left",
                )
                trip_df[trip_end_block_col] = trip_df[trip_end_block_col].fillna(trip_df[trip_end_block_col + "_new"])
                trip_df = trip_df.drop(columns=[trip_end_block_col + "_new"])

        # ========== usage 5min ==========
        trip_df["time_slot"] = trip_df[trip_started_at_col].dt.floor(block_time_granularity)
        trip_df["time_slot_end"] = trip_df[trip_ended_at_col].dt.floor(block_time_granularity)

        trips_starting = (
            trip_df.groupby([trip_start_block_col, "time_slot"])
            .size()
            .reset_index(name="trips_starting")
            .rename(columns={trip_start_block_col: "census_block"})
        )
        trips_ending = (
            trip_df.groupby([trip_end_block_col, "time_slot_end"])
            .size()
            .reset_index(name="trips_ending")
            .rename(columns={trip_end_block_col: "census_block", "time_slot_end": "time_slot"})
        )

        all_blocks = pd.unique(pd.concat([trip_df[trip_start_block_col].astype(str), trip_df[trip_end_block_col].astype(str)]))
        all_blocks = pd.Series(all_blocks).astype(str)
        all_blocks = all_blocks[(all_blocks != "nan") & (all_blocks != "0")].unique()
        all_slots = pd.date_range(start=t_start.floor(block_time_granularity), end=t_end.ceil(block_time_granularity), freq=block_time_granularity)
        grid = pd.MultiIndex.from_product([all_blocks, all_slots], names=["census_block", "time_slot"]).to_frame(index=False)

        usage_5min_block = grid.merge(trips_starting, on=["census_block", "time_slot"], how="left")
        usage_5min_block = usage_5min_block.merge(trips_ending, on=["census_block", "time_slot"], how="left")
        usage_5min_block[["trips_starting", "trips_ending"]] = usage_5min_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
        usage_5min_block = usage_5min_block.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

        # ========== usage hourly block ==========
        trip_df["hour_start"] = trip_df[trip_started_at_col].dt.floor(tract_time_granularity)
        trip_df["hour_end"] = trip_df[trip_ended_at_col].dt.floor(tract_time_granularity)

        start_hr = (
            trip_df.groupby([trip_start_block_col, "hour_start"])
            .size()
            .reset_index(name="trips_starting")
            .rename(columns={trip_start_block_col: "census_block", "hour_start": "time_slot"})
        )
        end_hr = (
            trip_df.groupby([trip_end_block_col, "hour_end"])
            .size()
            .reset_index(name="trips_ending")
            .rename(columns={trip_end_block_col: "census_block", "hour_end": "time_slot"})
        )

        all_hours = pd.date_range(start=t_start.floor(tract_time_granularity), end=t_end.ceil(tract_time_granularity), freq=tract_time_granularity)
        grid_hr = pd.MultiIndex.from_product([all_blocks, all_hours], names=["census_block", "time_slot"]).to_frame(index=False)

        usage_hourly_block = grid_hr.merge(start_hr, on=["census_block", "time_slot"], how="left").merge(end_hr, on=["census_block", "time_slot"], how="left")
        usage_hourly_block[["trips_starting", "trips_ending"]] = usage_hourly_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
        usage_hourly_block = usage_hourly_block.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

        # ========== usage hourly tract ==========
        usage_hourly_tract = usage_hourly_block.copy()
        usage_hourly_tract["census_tract"] = usage_hourly_tract["census_block"].astype(str).str[:tract_digits]
        usage_hourly_tract = (
            usage_hourly_tract.groupby(["census_tract", "time_slot"], as_index=False)[["trips_starting", "trips_ending"]]
            .sum()
            .sort_values(["census_tract", "time_slot"])
            .reset_index(drop=True)
        )
        usage_norm_hourly_tract = _minmax_norm(usage_hourly_tract, ["trips_starting", "trips_ending"]) if normalize else usage_hourly_tract.copy()

        # ========== idle time (optional) ==========
        if compute_idle:
            from collections import deque

            station_avail = (
                status_done.groupby(["census_block", "time_slot"], as_index=False)[[bikes_col, ebikes_col, "total_vehicle_available"]]
                .sum()
            )
            station_avail["time_slot"] = pd.to_datetime(station_avail["time_slot"]) - pd.Timedelta(minutes=5)

            merged = station_avail.merge(usage_5min_block, on=["census_block", "time_slot"], how="left")
            merged[["trips_starting", "trips_ending"]] = merged[["trips_starting", "trips_ending"]].fillna(0).astype(int)
            merged["vehicles_moved"] = merged["trips_starting"] + merged["trips_ending"]
            merged["idle_vehicles"] = (merged["total_vehicle_available"] - merged["vehicles_moved"]).clip(lower=0).astype(int)

            merged["hour"] = pd.to_datetime(merged["time_slot"]).dt.floor(tract_time_granularity)
            merged = merged.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

            idle_rows: List[dict] = []
            for block in merged["census_block"].unique():
                bdf = merged[merged["census_block"] == block]
                for hour in bdf["hour"].unique():
                    hdf = bdf[bdf["hour"] == hour].copy().sort_values("time_slot")

                    q = deque()
                    finalized: List[float] = []

                    for _, r in hdf.iterrows():
                        t = r["time_slot"]
                        cur_idle = int(r["idle_vehicles"])

                        if cur_idle > len(q):
                            for _ in range(cur_idle - len(q)):
                                q.append(t)
                        elif cur_idle < len(q):
                            for _ in range(len(q) - cur_idle):
                                stt = q.popleft()
                                finalized.append((t - stt).total_seconds() / 60.0)

                    final_t = pd.to_datetime(hour) + pd.Timedelta(hours=1)
                    while q:
                        stt = q.popleft()
                        finalized.append((final_t - stt).total_seconds() / 60.0)

                    avg_idle = round(float(np.mean(finalized)), 2) if finalized else 0.0
                    idle_rows.append({"census_block": block, "time_slot": pd.to_datetime(hour), "avg_idle_time": avg_idle})

            idle_time_block_hourly = pd.DataFrame(idle_rows).sort_values(["census_block", "time_slot"]).reset_index(drop=True)

            idle_time_tract_hourly = idle_time_block_hourly.copy()
            idle_time_tract_hourly["census_tract"] = idle_time_tract_hourly["census_block"].astype(str).str[:tract_digits]
            idle_time_tract_hourly = (
                idle_time_tract_hourly.groupby(["census_tract", "time_slot"], as_index=False)["avg_idle_time"]
                .mean()
                .sort_values(["census_tract", "time_slot"])
                .reset_index(drop=True)
            )
            idle_time_norm_tract_hourly = _minmax_norm(idle_time_tract_hourly, ["avg_idle_time"]) if normalize else idle_time_tract_hourly.copy()

    # ============================================================
    # H) Safety (optional)
    # ============================================================
    safety_block = None
    safety_tract = None
    safety_norm_tract = None

    if compute_safety:
        import geopandas as gpd

        blocks = gpd.read_file(NJ_CENSUS_BLOCKS_SHP).to_crs(epsg=int(safety_working_epsg))
        streets = gpd.read_file(NJ_CENTERLINE_SHP).to_crs(epsg=int(safety_working_epsg))
        lanes = gpd.read_file(NJ_BIKE_LANES_SHP).to_crs(epsg=int(safety_working_epsg))

        streets_in_blocks = gpd.overlay(streets, blocks, how="intersection")
        streets_in_blocks["segment_length"] = streets_in_blocks.geometry.length
        street_len = (
            streets_in_blocks.groupby(blocks_geoid_col, as_index=False)
            .agg(streets_leng=("segment_length", "sum"))
            .rename(columns={blocks_geoid_col: "census_block"})
        )
        street_len["streets_leng"] = street_len["streets_leng"].round(3)

        lanes_in_blocks = gpd.overlay(lanes, blocks, how="intersection")
        lanes_in_blocks["bike_lane_length"] = lanes_in_blocks.geometry.length
        lane_len = (
            lanes_in_blocks.groupby(blocks_geoid_col, as_index=False)
            .agg(total_bike_lane_length=("bike_lane_length", "sum"))
            .rename(columns={blocks_geoid_col: "census_block"})
        )
        lane_len["total_bike_lane_length"] = lane_len["total_bike_lane_length"].round(3)

        prot_len = pd.DataFrame({"census_block": [], "protected_bike_lane_length": []})
        if bike_lane_type_col in lanes.columns:
            protected = lanes[lanes[bike_lane_type_col] == protected_bike_lane_value].copy()
            if not protected.empty:
                prot_in_blocks = gpd.overlay(protected, blocks, how="intersection")
                prot_in_blocks["protected_lane_length"] = prot_in_blocks.geometry.length
                prot_len = (
                    prot_in_blocks.groupby(blocks_geoid_col, as_index=False)
                    .agg(protected_bike_lane_length=("protected_lane_length", "sum"))
                    .rename(columns={blocks_geoid_col: "census_block"})
                )
                prot_len["protected_bike_lane_length"] = prot_len["protected_bike_lane_length"].round(3)

        safety_block = (
            street_len.merge(lane_len, on="census_block", how="left")
            .merge(prot_len, on="census_block", how="left")
        )
        safety_block["total_bike_lane_length"] = safety_block["total_bike_lane_length"].fillna(0.0)
        safety_block["protected_bike_lane_length"] = safety_block["protected_bike_lane_length"].fillna(0.0)

        safety_block["bike_lane_ratio"] = safety_block.apply(
            lambda r: (r["total_bike_lane_length"] / r["streets_leng"]) if r["streets_leng"] > 0 else 0.0,
            axis=1,
        ).round(3)
        safety_block["protected_bike_lane_ratio"] = safety_block.apply(
            lambda r: (r["protected_bike_lane_length"] / r["streets_leng"]) if r["streets_leng"] > 0 else 0.0,
            axis=1,
        ).round(3)

        safety_block["census_block"] = safety_block["census_block"].astype(str)
        safety_block["census_tract"] = safety_block["census_block"].str[:tract_digits]

        tract_sums = (
            safety_block.groupby("census_tract", as_index=False)[
                ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]
            ].sum()
        )

        safety_tract = centroids.merge(tract_sums, on="census_tract", how="left")
        safety_tract[["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]] = (
            safety_tract[["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]].fillna(0.0)
        )
        safety_tract["bike_lane_ratio"] = safety_tract.apply(
            lambda r: (r["total_bike_lane_length"] / r["streets_leng"]) if r["streets_leng"] > 0 else 0.0,
            axis=1,
        )
        safety_tract["protected_bike_lane_ratio"] = safety_tract.apply(
            lambda r: (r["protected_bike_lane_length"] / r["streets_leng"]) if r["streets_leng"] > 0 else 0.0,
            axis=1,
        )
        safety_tract = safety_tract.sort_values("census_tract").reset_index(drop=True)
        safety_norm_tract = _minmax_norm(safety_tract, ["bike_lane_ratio", "protected_bike_lane_ratio"]) if normalize else safety_tract.copy()

    # ============================================================
    # Save outputs
    # ============================================================
    if save_outputs:
        availability_block_5min.to_csv(p_avail_block_5min, index=False)
        availability_hourly_tract.to_csv(p_avail_hourly_tract, index=False)
        availability_norm_hourly_tract.to_csv(p_avail_norm_hourly_tract, index=False)

        capacity_block.to_csv(p_capacity_block, index=False)
        capacity_tract.to_csv(p_capacity_tract, index=False)
        capacity_norm_tract.to_csv(p_capacity_norm_tract, index=False)

        if usage_5min_block is not None:
            usage_5min_block.to_csv(p_usage_5min_block, index=False)
        if usage_hourly_block is not None:
            usage_hourly_block.to_csv(p_usage_hourly_block, index=False)
        if usage_hourly_tract is not None:
            usage_hourly_tract.to_csv(p_usage_hourly_tract, index=False)
        if usage_norm_hourly_tract is not None:
            usage_norm_hourly_tract.to_csv(p_usage_norm_hourly_tract, index=False)

        if idle_time_block_hourly is not None:
            idle_time_block_hourly.to_csv(p_idle_block_hourly, index=False)
        if idle_time_tract_hourly is not None:
            idle_time_tract_hourly.to_csv(p_idle_tract_hourly, index=False)
        if idle_time_norm_tract_hourly is not None:
            idle_time_norm_tract_hourly.to_csv(p_idle_norm_tract_hourly, index=False)

        if safety_block is not None:
            safety_block.to_csv(p_safety_block, index=False)
        if safety_tract is not None:
            safety_tract.to_csv(p_safety_tract, index=False)
        if safety_norm_tract is not None:
            safety_norm_tract.to_csv(p_safety_norm_tract, index=False)

    return {
        "station_status_raw_df": station_status_raw,
        "station_information_raw_df": df_si_raw,
        "station_information_done_df": df_si_done,
        "station_status_done_df": status_done,

        "availability_block_5min_df": availability_block_5min,
        "availability_hourly_tract_df": availability_hourly_tract,
        "availability_norm_hourly_tract_df": availability_norm_hourly_tract,

        "capacity_block_df": capacity_block,
        "capacity_tract_df": capacity_tract,
        "capacity_norm_tract_df": capacity_norm_tract,

        "usage_5min_block_df": usage_5min_block,
        "usage_hourly_block_df": usage_hourly_block,
        "usage_hourly_tract_df": usage_hourly_tract,
        "usage_norm_hourly_tract_df": usage_norm_hourly_tract,

        "idle_time_block_hourly_df": idle_time_block_hourly,
        "idle_time_tract_hourly_df": idle_time_tract_hourly,
        "idle_time_norm_tract_hourly_df": idle_time_norm_tract_hourly,

        "safety_block_df": safety_block,
        "safety_tract_df": safety_tract,
        "safety_norm_tract_df": safety_norm_tract,

        "paths": {k: str(v) for k, v in {
            "availability_norm_hourly_tract_csv": p_avail_norm_hourly_tract,
            "station_status_done_csv": p_station_status_done,
            "station_information_done_csv": p_station_info_done,
            "capacity_norm_tract_csv": p_capacity_norm_tract,
            "usage_norm_hourly_tract_csv": p_usage_norm_hourly_tract,
            "idle_time_norm_tract_hourly_csv": p_idle_norm_tract_hourly,
            "safety_bike_lane_norm_tract_csv": p_safety_norm_tract,
        }.items()},
    }


## **CALLING THE FUNCTION**

In [7]:
# ----------------------------
# Example call (now only 3 inputs + optional time windows)
# ----------------------------
out = compute_nj_docked_onecall_utilities(
    station_status_txt=r"D:\Research Fellowship\Summer Research Stuff\Collected Data\Week 1\09-June\nyc_docked_station_status_6_9.txt",
    station_information_csv=r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NJ\NJ station information 06_09.csv",  # or pass a local CSV if you prefer
    trip_csv=r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Usage\NJ\JC-202505-citibike-tripdata.csv",
    trip_csv_already_has_blocks=False,
    station_time_start="2025-06-09 06:00:00",
    station_time_end="2025-06-09 12:00:00",
    trip_time_start="2025-05-01 00:00:00",
    trip_time_end="2025-05-31 23:59:59",
    output_dir="NJ_OneCall",
)
print(out["paths"]["availability_norm_hourly_tract_csv"])


C:\Users\Vedant\AppData\Local\Temp\ipykernel_41700\2645987263.py:375: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  tract_5min["time_slot_hour"] = pd.to_datetime(tract_5min["timestamp"]).dt.floor(tract_time_granularity)
100%|██████████| 101/101 [01:28<00:00,  1.14it/s]
C:\Users\Vedant\AppData\Local\Temp\ipykernel_41700\2645987263.py:558: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  trip_df["hour_start"] = trip_df[trip_started_at_col].dt.floor(tract_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_41700\2645987263.py:559: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  trip_df["hour_end"] = trip_df[trip_ended_at_col].dt.floor(tract_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_41700\2645987263.py:574: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.


NJ_OneCall\availability_norm_hourly_tract_nj.csv


In [ ]:
# # ----------------------------
# # Example call (NJ docked)
# # ----------------------------
# out = compute_nj_docked_onecall_utilities(
#     station_status_txt=r"C:\Research Fellowship\Collected Data\Week 1\09-June\nj_docked_station_status_6_9.txt",
#     census_blocks_shp=r"C:\Research Fellowship\Clean_Utilities\Safety\NJ\tl_2024_34_tabblock20.shp",
#     centroid_tract_csv=r"C:\Research Fellowship\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv",
#     output_dir="NJ_OneCall",

#     # use either:
#     # station_information_csv=r"C:\...\NJ station information 06_09.csv",
#     # OR let it fetch from GBFS:
#     station_information_url="https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json",

#     # Optional trips (enables usage + idle time):
#     trip_csv=r"C:\Research Fellowship\Clean_Utilities\Usage\NJ\JC-202505-citibike-tripdata.csv",
#     trip_csv_already_has_blocks=False,

#     station_time_start="2025-06-09 06:00:00",
#     station_time_end="2025-06-09 12:00:00",
#     trip_time_start="2025-05-01 00:00:00",
#     trip_time_end="2025-05-31 23:59:59",

#     # If you ever need to drop leak-in state blocks:
#     # drop_census_prefixes_in_station=["36"],
#     # drop_census_prefixes_in_trip=["36"],

#     # Optional safety:
#     centerline_streets_shp=r"C:\Research Fellowship\Clean_Utilities\Safety\NJ\Tran_road.shp",
#     bike_lanes_shp=r"C:\Research Fellowship\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp",
#     safety_working_epsg=2263,
# )
# print(out["paths"]["availability_norm_hourly_tract_csv"])

In [ ]:
# # ----------------------------
# # Example call (NJ docked)
# # ----------------------------
# out = compute_nj_docked_onecall_utilities(
#     station_status_txt=r"C:\Research Fellowship\Collected Data\Week 1\09-June\nj_docked_station_status_6_9.txt",
#     census_blocks_shp=r"C:\Research Fellowship\Clean_Utilities\Safety\NJ\tl_2024_34_tabblock20.shp",
#     centroid_tract_csv=r"C:\Research Fellowship\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv",
#     output_dir="NJ_OneCall",

#     # use either:
#     # station_information_csv=r"C:\...\NJ station information 06_09.csv",
#     # OR let it fetch from GBFS:
#     station_information_url="https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json",

#     # Optional trips (enables usage + idle time):
#     trip_csv=r"C:\Research Fellowship\Clean_Utilities\Usage\NJ\JC-202505-citibike-tripdata.csv",
#     trip_csv_already_has_blocks=False,

#     station_time_start="2025-06-09 06:00:00",
#     station_time_end="2025-06-09 12:00:00",
#     trip_time_start="2025-05-01 00:00:00",
#     trip_time_end="2025-05-31 23:59:59",

#     # If you ever need to drop leak-in state blocks:
#     # drop_census_prefixes_in_station=["36"],
#     # drop_census_prefixes_in_trip=["36"],

#     # Optional safety:
#     centerline_streets_shp=r"C:\Research Fellowship\Clean_Utilities\Safety\NJ\Tran_road.shp",
#     bike_lanes_shp=r"C:\Research Fellowship\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp",
#     safety_working_epsg=2263,
# )
# print(out["paths"]["availability_norm_hourly_tract_csv"])

# **Version  2**

In [16]:
from __future__ import annotations

from pathlib import Path
from typing import Optional, Union, Dict, Any, Sequence, Tuple
import json
import pandas as pd


# ==================================================
# CITY CONFIG (hardcoded assets for NYC + NJ + PITT)
# ==================================================
CITY_CONFIG: Dict[str, Dict[str, Any]] = {
    "NYC": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NYC\tl_2024_36_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NYC\tl_2024_36_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Capacity_NYC\centroid_tract_computed.csv"
        ),
        # safety (NYC = CSV WKT)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\CSCL_PlowNYC_20250619.csv"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\New_York_City_Bike_Routes_20250619.csv"
        ),
        "safety_input_kind": "csv_wkt",
        "safety_csv_wkt_candidates": ("the_geom", "wkt", "geometry_wkt", "WKT", "geom"),
        "safety_csv_crs": "EPSG:4326",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": "34",  # drop NJ tracts for NYC results
        "has_boro_logic": True,
        "drop_staten_island_default": True,
        "protected_lane_rule": {
            "col_candidates": ("facilitycl", "facility", "class", "ft", "type"),
            "match_type": "equals",
            "match_value": "I",
        },
    },
    "NJ": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\tl_2024_34_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\tl_2024_34_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv"
        ),
        # safety (NJ = SHP)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\Tran_road.shp"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp"
        ),
        "safety_input_kind": "shp",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        "protected_lane_rule": {
            "col_candidates": ("type", "facility", "facilitycl", "class", "lane_type"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
    "PITT": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\tl_2024_42_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\tl_2024_42_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\centroid_tract_pa.csv"
        ),
        # safety (PITT = SHP)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Pittsburgh_Street_Centerline.shp"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Bike Map\Bike Lanes.shp"
        ),
        "safety_input_kind": "shp",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        "protected_lane_rule": {
            "col_candidates": ("facility", "type", "class", "status", "lane_type", "BIKE_FACIL", "CATEGORY"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
}


# ==================================================
# UTILITIES
# ==================================================
def _ensure_exists(p: Path, label: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"{label} not found: {p}")


def _minmax_norm(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    mn = s.min(skipna=True)
    mx = s.max(skipna=True)
    if pd.isna(mn) or pd.isna(mx) or mx <= mn:
        return pd.Series(0.0, index=s.index)
    return (s - mn) / (mx - mn)


def _safe_ratio(numer: pd.Series, denom: pd.Series) -> pd.Series:
    return numer.div(denom.where(denom > 0, other=pd.NA)).fillna(0)


def _pick_col(df: pd.DataFrame, candidates: Sequence[str], required: bool, label: str) -> Optional[str]:
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    if required:
        raise ValueError(
            f"Could not find required column for {label}. Tried {list(candidates)}. "
            f"Found columns (sample): {list(df.columns)[:200]}"
        )
    return None


def _tract_from_block(block_series: pd.Series) -> pd.Series:
    s = block_series.astype(str)
    return s.where(s.str.len() < 15, s.str[:-4])


def _standardize_station_info(df: pd.DataFrame) -> pd.DataFrame:
    sid = _pick_col(df, ["station_id", "stationId", "id", "Station ID", "Station Id"], True, "station_id")
    lat = _pick_col(df, ["lat", "latitude", "station_lat", "y"], True, "lat")
    lon = _pick_col(df, ["lon", "lng", "longitude", "station_lon", "x"], True, "lon")
    cap = _pick_col(df, ["capacity", "dock_count", "num_docks", "total_docks"], True, "capacity")

    out = df.copy().rename(columns={sid: "station_id", lat: "lat", lon: "lon", cap: "capacity"})
    out["station_id"] = out["station_id"].astype(str).str.strip()

    cb = _pick_col(out, ["census_block", "block_geoid", "GEOID20"], False, "census_block")
    if cb and cb != "census_block":
        out = out.rename(columns={cb: "census_block"})
    ct = _pick_col(out, ["census_tract", "tract_geoid", "GEOID"], False, "census_tract")
    if ct and ct != "census_tract":
        out = out.rename(columns={ct: "census_tract"})

    return out


def _standardize_trips(df: pd.DataFrame) -> pd.DataFrame:
    # times (CitiBike + PIT exports)
    started = _pick_col(
        df,
        ["started_at", "start_time", "trip_start_time", "starttime", "Start Date", "start_date", "StartDate"],
        True,
        "started_at",
    )
    ended = _pick_col(
        df,
        ["ended_at", "end_time", "trip_end_time", "stoptime", "End Date", "end_date", "EndDate"],
        True,
        "ended_at",
    )
    out = df.copy().rename(columns={started: "started_at", ended: "ended_at"})

    # rideable_type / rider type (optional)
    rt = _pick_col(out, ["rideable_type", "bike_type", "vehicle_type", "Rider Type", "rider_type"], False, "rideable_type")
    if rt and rt != "rideable_type":
        out = out.rename(columns={rt: "rideable_type"})

    # precomputed census blocks (optional)
    sb = _pick_col(out, ["start_census_block", "start_block", "start_block_geoid"], False, "start_census_block")
    eb = _pick_col(out, ["end_census_block", "end_block", "end_block_geoid"], False, "end_census_block")
    if sb and sb != "start_census_block":
        out = out.rename(columns={sb: "start_census_block"})
    if eb and eb != "end_census_block":
        out = out.rename(columns={eb: "end_census_block"})

    # lat/lng
    slat = _pick_col(out, ["start_lat", "startLatitude", "StartLat"], False, "start_lat")
    slng = _pick_col(out, ["start_lng", "start_lon", "startLongitude", "StartLng"], False, "start_lng")
    elat = _pick_col(out, ["end_lat", "endLatitude", "EndLat"], False, "end_lat")
    elng = _pick_col(out, ["end_lng", "end_lon", "endLongitude", "EndLng"], False, "end_lng")

    if slat and slat != "start_lat":
        out = out.rename(columns={slat: "start_lat"})
    if slng and slng != "start_lng":
        out = out.rename(columns={slng: "start_lng"})
    if elat and elat != "end_lat":
        out = out.rename(columns={elat: "end_lat"})
    if elng and elng != "end_lng":
        out = out.rename(columns={elng: "end_lng"})

    # station ids (optional)
    ssid = _pick_col(out, ["start_station_id", "Start Station Id", "Start Station ID"], False, "start_station_id")
    esid = _pick_col(out, ["end_station_id", "End Station Id", "End Station ID"], False, "end_station_id")
    if ssid and ssid != "start_station_id":
        out = out.rename(columns={ssid: "start_station_id"})
    if esid and esid != "end_station_id":
        out = out.rename(columns={esid: "end_station_id"})

    return out


def _read_lines_layer(path: Path, *, kind: str, target_crs, csv_wkt_candidates: Tuple[str, ...], csv_crs: str):
    import geopandas as gpd
    from shapely import wkt

    if kind == "shp":
        gdf = gpd.read_file(str(path))
        if gdf.crs is None:
            raise ValueError(f"CRS missing for {path}. Define CRS before projecting.")
        return gdf.to_crs(target_crs)

    # csv with WKT
    df = pd.read_csv(path)
    wkt_col = _pick_col(df, csv_wkt_candidates, True, f"WKT geometry in {path.name}")
    df = df.copy()
    df["geometry"] = df[wkt_col].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=csv_crs)
    return gdf.to_crs(target_crs)


def _overlay_length_by_block(lines_gdf, blocks_gdf, *, blocks_id_col: str, out_col: str) -> pd.DataFrame:
    import geopandas as gpd

    blocks = blocks_gdf[[blocks_id_col, "geometry"]].copy()
    lines = lines_gdf[["geometry"]].copy()

    inter = gpd.overlay(lines, blocks, how="intersection", keep_geom_type=False)
    if inter.empty:
        return pd.DataFrame({"census_block": pd.Series([], dtype=str), out_col: pd.Series([], dtype=float)})

    inter["seg_len"] = inter.geometry.length
    inter = inter.rename(columns={blocks_id_col: "census_block"})
    out = inter.groupby("census_block", as_index=False)["seg_len"].sum().rename(columns={"seg_len": out_col})
    out[out_col] = out[out_col].round(3)
    out["census_block"] = out["census_block"].astype(str)
    return out


def _protected_filter(lines_gdf, rule: Dict[str, Any]) -> Tuple[Optional[str], Any]:
    col = _pick_col(lines_gdf, list(rule.get("col_candidates", [])), required=False, label="protected lane col")
    if col is None:
        return None, lines_gdf

    s = lines_gdf[col].astype(str)
    match_type = rule.get("match_type", "equals")
    match_value = str(rule.get("match_value", ""))

    if match_type == "equals":
        mask = s.str.upper() == match_value.upper()
    elif match_type == "contains":
        mask = s.str.upper().str.contains(match_value.upper(), na=False)
    else:
        raise ValueError(f"Unsupported match_type='{match_type}' in protected_lane_rule")

    return col, lines_gdf[mask].copy()


def _geocode_points_to_blocks_spatial(
    df: pd.DataFrame,
    *,
    lon_col: str,
    lat_col: str,
    blocks_path: Path,
    blocks_id_col: str,
) -> pd.DataFrame:
    import geopandas as gpd
    from shapely.geometry import Point

    blocks = gpd.read_file(str(blocks_path)).to_crs(epsg=4326)
    if blocks_id_col not in blocks.columns:
        raise ValueError(f"Block shapefile missing '{blocks_id_col}' for spatial geocoding.")

    pts = df[[lon_col, lat_col]].dropna().drop_duplicates().copy()
    pts = pts.rename(columns={lon_col: "lon", lat_col: "lat"})

    gdf = gpd.GeoDataFrame(
        pts,
        geometry=[Point(xy) for xy in zip(pts["lon"], pts["lat"])],
        crs="EPSG:4326",
    )
    joined = gpd.sjoin(gdf, blocks[[blocks_id_col, "geometry"]], how="left", predicate="within")
    joined = joined.rename(columns={blocks_id_col: "census_block"})
    out = joined[["lon", "lat", "census_block"]].copy()
    out["census_block"] = out["census_block"].astype(str)
    return out


def _census_api_block(lat: float, lon: float, benchmark: str, vintage: str) -> Optional[str]:
    import requests
    try:
        url = (
            "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
            f"?x={lon}&y={lat}"
            f"&benchmark={benchmark}"
            f"&vintage={vintage}"
            "&format=json"
        )
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        js = r.json()
        blocks = js["result"]["geographies"].get("2020 Census Blocks", [])
        if not blocks:
            return None
        return blocks[0].get("GEOID")
    except Exception:
        return None


# ==================================================
# MAIN FUNCTION
# ==================================================
def run_docked_all_utilities_single_function(
    *,
    city: str,
    station_status_txt: Union[str, Path],
    station_information_csv: Union[str, Path],
    trip_csv: Union[str, Path],
    output_dir: Optional[Union[str, Path]] = None,
    save_outputs: bool = True,
    compute_idle_time: bool = True,  # <--- IMPORTANT: PIT trips often don't have bike-type; this allows skip
    availability_time_start: Optional[Union[str, pd.Timestamp]] = None,
    availability_time_end: Optional[Union[str, pd.Timestamp]] = None,
    availability_time_granularity: str = "1H",
    availability_group_level: str = "both",
    peak_time_slot: Optional[Union[str, pd.Timestamp]] = None,
    peak_metric: str = "total_vehicle_available",
    drop_staten_island: Optional[bool] = None,
    usage_time_start: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_end: Optional[Union[str, pd.Timestamp]] = None,
    tracts_to_remove: Optional[Sequence[str]] = None,
    idle_time_start: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_end: Optional[Union[str, pd.Timestamp]] = None,
    remove_tz_suffix: str = " EDT",
    drop_cols: tuple[str, ...] = (
        "num_scooters_available",
        "num_scooters_unavailable",
        "is_installed",
        "is_returning",
        "is_renting",
    ),
    fill_missing_with_census_api: bool = True,
    census_geocoder_benchmark: str = "Public_AR_Census2020",
    census_geocoder_vintage: str = "Census2020_Current",
) -> Dict[str, Any]:
    """
    One-function multi-city docked pipeline (NYC/NJ/PITT):
      Step0: station status JSONL -> df, station info -> add census block/tract
      Availability: block/tract time series + norm (tolerant to missing ebikes)
      Capacity: station capacities + peak vehicle/dock capacity + norm
      Safety: overlay-based street + bike lane + protected lane ratios (CRS-safe: uses blocks CRS)
      Trips: supports CitiBike schema + Pittsburgh export schema; builds start/end census blocks from lat/lng if missing
      Usage: 5-min + hourly + tract hourly norm
      Idle: optional; runs only if compute_idle_time=True AND trip data has usable bike-type field
    """

    city_key = city.strip().upper()
    if city_key in {"PITTSBURGH", "PIT"}:
        city_key = "PITT"

    if city_key not in CITY_CONFIG:
        raise ValueError(f"Unsupported city='{city}'. Supported: {list(CITY_CONFIG.keys())}")

    cfg = CITY_CONFIG[city_key]

    station_status_txt = Path(station_status_txt)
    station_information_csv = Path(station_information_csv)
    trip_csv = Path(trip_csv)

    census_blocks_path = Path(cfg["census_blocks_path"])
    tracts_path = Path(cfg["tracts_path"])
    centroid_tract_csv = Path(cfg["centroid_tract_csv"])
    centerline_path = Path(cfg["centerline_path"])
    bike_lanes_path = Path(cfg["bike_lanes_path"])

    blocks_id_col = str(cfg["blocks_id_col"])
    tract_id_col = str(cfg["tract_id_col"])
    external_tract_prefix = cfg.get("external_tract_prefix", None)

    if drop_staten_island is None:
        drop_staten_island = bool(cfg.get("drop_staten_island_default", False))

    out_dir = Path(output_dir) if output_dir is not None else Path(f"./{city_key}_ALL_IN_ONE_OUTPUTS")
    out_dir.mkdir(parents=True, exist_ok=True)

    # existence checks
    _ensure_exists(station_status_txt, "station_status_txt")
    _ensure_exists(station_information_csv, "station_information_csv")
    _ensure_exists(trip_csv, "trip_csv")
    _ensure_exists(census_blocks_path, "census_blocks_path")
    _ensure_exists(tracts_path, "tracts_path")
    _ensure_exists(centroid_tract_csv, "centroid_tract_csv")
    _ensure_exists(centerline_path, "centerline_path")
    _ensure_exists(bike_lanes_path, "bike_lanes_path")

    # normalize times
    availability_time_start = pd.to_datetime(availability_time_start) if availability_time_start is not None else None
    availability_time_end = pd.to_datetime(availability_time_end) if availability_time_end is not None else None
    usage_time_start = pd.to_datetime(usage_time_start) if usage_time_start is not None else None
    usage_time_end = pd.to_datetime(usage_time_end) if usage_time_end is not None else None
    idle_time_start = pd.to_datetime(idle_time_start) if idle_time_start is not None else None
    idle_time_end = pd.to_datetime(idle_time_end) if idle_time_end is not None else None

    availability_group_level = availability_group_level.lower().strip()
    if availability_group_level not in {"block", "tract", "both"}:
        raise ValueError("availability_group_level must be one of: 'block', 'tract', 'both'")

    if peak_metric not in {"total_vehicle_available", "num_docks_available"}:
        raise ValueError("peak_metric must be 'total_vehicle_available' or 'num_docks_available'")

    results: Dict[str, Any] = {
        "city": city_key,
        "system_type": "docked",
        "output_dir": str(out_dir),
        "city_assets": {
            "census_blocks_path": str(census_blocks_path),
            "tracts_path": str(tracts_path),
            "centroid_tract_csv": str(centroid_tract_csv),
            "centerline_path": str(centerline_path),
            "bike_lanes_path": str(bike_lanes_path),
        },
    }

    # ==================================================
    # STEP 0 — station_status JSONL flatten
    # ==================================================
    with station_status_txt.open("r", encoding="utf-8") as f:
        lines = f.readlines()

    di_list = []
    for item in lines:
        if not item.strip():
            continue
        js = json.loads(item)
        timestamp = list(js.keys())[0]
        for entry in js[timestamp]:
            for vehicle_type in entry.get("vehicle_types_available", []):
                entry[f"vehicle_type_{vehicle_type['vehicle_type_id']}_count"] = vehicle_type["count"]
            if "vehicle_types_available" in entry:
                del entry["vehicle_types_available"]
            entry["timestamp"] = timestamp
            di_list.append(entry)

    station_status_flat = pd.DataFrame(di_list)
    results["station_status_flat"] = station_status_flat
    if save_outputs:
        station_status_flat.to_csv(out_dir / f"{city_key.lower()}_station_status_flat.csv", index=False)

    ss_df = station_status_flat.copy()
    ss_df["timestamp"] = ss_df["timestamp"].astype(str).str.replace(remove_tz_suffix, "", regex=False)
    ss_df["timestamp"] = pd.to_datetime(ss_df["timestamp"], errors="coerce")
    ss_df["hour"] = ss_df["timestamp"].dt.hour
    ss_df["date"] = ss_df["timestamp"].dt.date
    cols_to_drop = [c for c in drop_cols if c in ss_df.columns]
    if cols_to_drop:
        ss_df = ss_df.drop(cols_to_drop, axis=1)

    if "station_id" not in ss_df.columns:
        raise ValueError("station_status data missing station_id after flattening.")
    ss_df["station_id"] = ss_df["station_id"].astype(str).str.strip()

    results["station_status_clean"] = ss_df
    if save_outputs:
        ss_df.to_csv(out_dir / f"{city_key.lower()}_station_status_clean.csv", index=False)

    # ==================================================
    # STEP 0 — station_information -> standardized + census block/tract
    # ==================================================
    si_raw = pd.read_csv(station_information_csv)
    results["station_info_raw"] = si_raw
    if save_outputs:
        si_raw.to_csv(out_dir / f"{city_key.lower()}_station_information_raw.csv", index=False)

    si = _standardize_station_info(si_raw)

    # If census_block missing, compute via spatial join
    if ("census_block" not in si.columns) or si["census_block"].isna().any():
        import geopandas as gpd
        from shapely.geometry import Point

        pts = si.drop_duplicates(subset=["lat", "lon"])[["lon", "lat"]].copy()
        pts_gdf = gpd.GeoDataFrame(
            pts,
            geometry=[Point(xy) for xy in zip(pts["lon"], pts["lat"])],
            crs="EPSG:4326",
        )

        blocks_wgs84 = gpd.read_file(str(census_blocks_path)).to_crs(epsg=4326)
        if blocks_id_col not in blocks_wgs84.columns:
            raise ValueError(f"Block shapefile missing '{blocks_id_col}' for station geocoding.")

        joined = gpd.sjoin(pts_gdf, blocks_wgs84[[blocks_id_col, "geometry"]], how="left", predicate="within")
        joined = joined.rename(columns={blocks_id_col: "census_block"})
        joined["census_block"] = joined["census_block"].astype(str)
        joined["census_tract"] = _tract_from_block(joined["census_block"]).astype(str)

        si = si.merge(joined[["lon", "lat", "census_block", "census_tract"]], on=["lon", "lat"], how="left", suffixes=("", "_sj"))

        # fill from spatial join columns if needed
        if "census_block_sj" in si.columns:
            if "census_block" not in si.columns:
                si["census_block"] = si["census_block_sj"]
            else:
                si["census_block"] = si["census_block"].fillna(si["census_block_sj"])
            si = si.drop(columns=["census_block_sj"], errors="ignore")

        if "census_tract_sj" in si.columns:
            if "census_tract" not in si.columns:
                si["census_tract"] = si["census_tract_sj"]
            else:
                si["census_tract"] = si["census_tract"].fillna(si["census_tract_sj"])
            si = si.drop(columns=["census_tract_sj"], errors="ignore")

        # Census API fill for missing
        if fill_missing_with_census_api and ("census_block" in si.columns) and si["census_block"].isna().any():
            miss = si[si["census_block"].isna()][["lat", "lon"]].drop_duplicates().copy()
            miss["census_block_new"] = [
                _census_api_block(lat, lon, census_geocoder_benchmark, census_geocoder_vintage)
                for lat, lon in zip(miss["lat"], miss["lon"])
            ]
            miss["census_tract_new"] = _tract_from_block(miss["census_block_new"]).astype(str)

            si = si.merge(miss, on=["lat", "lon"], how="left")
            si["census_block"] = si["census_block"].fillna(si["census_block_new"])
            if "census_tract" not in si.columns:
                si["census_tract"] = pd.NA
            si["census_tract"] = si["census_tract"].fillna(si["census_tract_new"])
            si = si.drop(columns=["census_block_new", "census_tract_new"], errors="ignore")

    si["census_block"] = si["census_block"].astype(str)
    if "census_tract" not in si.columns or si["census_tract"].isna().any():
        si["census_tract"] = _tract_from_block(si["census_block"]).astype(str)
    else:
        si["census_tract"] = si["census_tract"].astype(str)

    results["station_info_done"] = si
    if save_outputs:
        si.to_csv(out_dir / f"{city_key.lower()}_station_information_done.csv", index=False)

    # Merge blocks into station status
    ss_done = ss_df.merge(si[["station_id", "census_block"]], on="station_id", how="left")
    ss_done["census_block"] = ss_done["census_block"].astype(str)
    ss_done["census_tract"] = _tract_from_block(ss_done["census_block"]).astype(str)

    # --------------------------------------------------
    # Availability column standardization (tolerant to missing ebikes)
    # --------------------------------------------------
    if "num_bikes_available" not in ss_done.columns:
        raise ValueError(
            "station_status_done is missing 'num_bikes_available'. "
            f"Available columns: {list(ss_done.columns)[:200]}"
        )
    if "num_ebikes_available" not in ss_done.columns:
        ss_done["num_ebikes_available"] = 0

    if "num_docks_available" not in ss_done.columns:
        alt = None
        for c in ["docks_available", "num_docks_free", "num_docks_open"]:
            if c in ss_done.columns:
                alt = c
                break
        if alt is None:
            raise ValueError(
                "station_status_done is missing 'num_docks_available' and no alternative dock column was found. "
                f"Available columns: {list(ss_done.columns)[:200]}"
            )
        ss_done["num_docks_available"] = pd.to_numeric(ss_done[alt], errors="coerce").fillna(0).astype(int)

    ss_done["num_bikes_available"] = pd.to_numeric(ss_done["num_bikes_available"], errors="coerce").fillna(0).astype(int)
    ss_done["num_ebikes_available"] = pd.to_numeric(ss_done["num_ebikes_available"], errors="coerce").fillna(0).astype(int)
    ss_done["num_docks_available"] = pd.to_numeric(ss_done["num_docks_available"], errors="coerce").fillna(0).astype(int)
    ss_done["total_vehicle_available"] = ss_done["num_bikes_available"] + ss_done["num_ebikes_available"]

    results["station_status_done"] = ss_done
    if save_outputs:
        ss_done.to_csv(out_dir / f"{city_key.lower()}_station_status_done.csv", index=False)

    # ==================================================
    # AVAILABILITY
    # ==================================================
    required_cols = {"timestamp", "num_bikes_available", "num_docks_available", "census_block", "census_tract"}
    miss = required_cols - set(ss_done.columns)
    if miss:
        raise ValueError(f"station_status_done missing required columns for availability: {miss}")

    ss_av = ss_done.copy()
    ss_av["timestamp"] = pd.to_datetime(ss_av["timestamp"], errors="coerce")
    if ss_av["timestamp"].isna().any():
        raise ValueError("Found unparseable timestamps in station_status_done.")

    data_min = ss_av["timestamp"].min()
    data_max = ss_av["timestamp"].max()

    if availability_time_start is not None:
        ss_av = ss_av[ss_av["timestamp"] >= availability_time_start].copy()
    if availability_time_end is not None:
        ss_av = ss_av[ss_av["timestamp"] < availability_time_end].copy()
    if ss_av.empty:
        raise ValueError(f"No availability rows after time filter. Data range: [{data_min}, {data_max}]")

    def _availability_for_level(level: str) -> Dict[str, pd.DataFrame]:
        geo_key = "census_block" if level == "block" else "census_tract"

        temp = ss_av.copy()
        temp["time_slot"] = temp["timestamp"].dt.floor(availability_time_granularity)

        geo_ts = (
            temp.groupby([geo_key, "timestamp"], as_index=False)
            .agg(
                num_bikes_available=("num_bikes_available", "sum"),
                num_ebikes_available=("num_ebikes_available", "sum"),
                num_docks_available=("num_docks_available", "sum"),
                total_vehicle_available=("total_vehicle_available", "sum"),
            )
        )
        geo_ts["time_slot"] = geo_ts["timestamp"].dt.floor(availability_time_granularity)

        raw_df = (
            geo_ts.groupby([geo_key, "time_slot"], as_index=False)
            .mean(numeric_only=True)
            .round(0)
            .sort_values([geo_key, "time_slot"])
            .reset_index(drop=True)
        )

        for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
            raw_df[c] = raw_df[c].astype(int)

        if level == "tract" and external_tract_prefix:
            raw_df = raw_df[~raw_df["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

        norm_df = raw_df.copy()
        for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
            norm_df[c + "_norm"] = _minmax_norm(norm_df[c]).round(5)

        if save_outputs:
            raw_df.to_csv(out_dir / f"availability__raw__{level}.csv", index=False)
            norm_df.to_csv(out_dir / f"availability__norm__{level}.csv", index=False)

        return {"raw": raw_df, "norm": norm_df}

    availability_out: Dict[str, Any] = {"meta": {
        "time_start": availability_time_start,
        "time_end": availability_time_end,
        "data_min": data_min,
        "data_max": data_max,
        "time_granularity": availability_time_granularity,
        "group_level": availability_group_level,
    }}

    if availability_group_level in {"block", "both"}:
        blk = _availability_for_level("block")
        availability_out["availability_block_raw"] = blk["raw"]
        availability_out["availability_block_norm"] = blk["norm"]

    if availability_group_level in {"tract", "both"}:
        tr = _availability_for_level("tract")
        availability_out["availability_tract_raw"] = tr["raw"]
        availability_out["availability_tract_norm"] = tr["norm"]

    if "availability_tract_norm" not in availability_out:
        tr = _availability_for_level("tract")
        availability_out["availability_tract_raw"] = tr["raw"]
        availability_out["availability_tract_norm"] = tr["norm"]

    results["availability"] = availability_out

    availability_df_for_capacity = availability_out["availability_tract_norm"].copy()
    availability_df_for_capacity["census_tract"] = availability_df_for_capacity["census_tract"].astype(str)
    availability_df_for_capacity["time_slot"] = pd.to_datetime(availability_df_for_capacity["time_slot"])

    # ==================================================
    # CAPACITY (centroids computed from tract shapefile)
    # ==================================================
    import geopandas as gpd

    tracts_gdf = gpd.read_file(str(tracts_path))
    if tracts_gdf.crs is None:
        raise ValueError(f"Tracts shapefile CRS is missing: {tracts_path}")
    if tract_id_col not in tracts_gdf.columns:
        raise ValueError(f"Tract shapefile missing '{tract_id_col}' column.")

    if cfg.get("has_boro_logic", False):
        tracts_gdf["county_fips"] = tracts_gdf["STATEFP"].astype(str) + tracts_gdf["COUNTYFP"].astype(str)
        county_to_boro = {
            "36005": "The Bronx",
            "36047": "Brooklyn",
            "36061": "Manhattan",
            "36081": "Queens",
            "36085": "Staten Island",
        }
        tracts_gdf["boro"] = tracts_gdf["county_fips"].map(county_to_boro)
        if drop_staten_island:
            tracts_gdf = tracts_gdf[tracts_gdf["boro"] != "Staten Island"].copy()

    centroids_wgs84 = gpd.GeoSeries(tracts_gdf.geometry.centroid, crs=tracts_gdf.crs).to_crs(epsg=4326)
    tract_centroids_df = pd.DataFrame({
        "census_tract": tracts_gdf[tract_id_col].astype(str).values,
        "centroid_lon": centroids_wgs84.x.values,
        "centroid_lat": centroids_wgs84.y.values,
    })
    if "boro" in tracts_gdf.columns:
        tract_centroids_df["boro"] = tracts_gdf["boro"].values

    results["tract_centroids_df"] = tract_centroids_df
    if save_outputs:
        tract_centroids_df.to_csv(out_dir / "centroid_tract_computed.csv", index=False)

    station_info = si.copy()
    station_info["census_block"] = station_info["census_block"].astype(str)
    station_info["census_tract"] = _tract_from_block(station_info["census_block"]).astype(str)

    capacity_block = (
        station_info.groupby("census_block", as_index=False)
        .agg(total_capacity=("capacity", "sum"), num_station=("station_id", "count"))
    )
    results["capacity_block"] = capacity_block
    if save_outputs:
        capacity_block.to_csv(out_dir / "capacity_block.csv", index=False)

    tract_capacity = (
        station_info.groupby("census_tract", as_index=False)
        .agg(total_capacity=("capacity", "sum"), num_station=("station_id", "count"))
    )

    capacity_tract = tract_centroids_df[["census_tract"]].drop_duplicates().merge(
        tract_capacity, on="census_tract", how="left"
    )
    capacity_tract[["total_capacity", "num_station"]] = capacity_tract[["total_capacity", "num_station"]].fillna(0)

    if external_tract_prefix:
        capacity_tract = capacity_tract[~capacity_tract["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    capacity_tract = capacity_tract.sort_values("census_tract").reset_index(drop=True)
    capacity_tract["total_capacity_norm"] = _minmax_norm(capacity_tract["total_capacity"]).round(5)
    capacity_tract["num_station_norm"] = _minmax_norm(capacity_tract["num_station"]).round(5)

    results["capacity_tract_norm"] = capacity_tract
    if save_outputs:
        capacity_tract.to_csv(out_dir / "capacity_tract_norm.csv", index=False)

    avail = availability_df_for_capacity.copy()
    if external_tract_prefix:
        avail = avail[~avail["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    if peak_time_slot is None:
        peak_series = avail.groupby("time_slot")[peak_metric].sum()
        if peak_series.empty:
            raise ValueError("No availability data after filters; cannot find peak time_slot.")
        peak_time_slot = peak_series.idxmax()
    peak_time_slot = pd.to_datetime(peak_time_slot)

    peak_df = avail[avail["time_slot"] == peak_time_slot].copy()

    vehicle_capacity = (
        peak_df.groupby("census_tract", as_index=False)["total_vehicle_available"]
        .sum()
        .rename(columns={"total_vehicle_available": "vehicle_capacity"})
    )
    dock_capacity = (
        peak_df.groupby("census_tract", as_index=False)["num_docks_available"]
        .sum()
        .rename(columns={"num_docks_available": "dock_capacity"})
    )

    capacity_df = (
        capacity_tract.merge(vehicle_capacity, on="census_tract", how="left")
        .merge(dock_capacity, on="census_tract", how="left")
    )
    capacity_df["vehicle_capacity"] = capacity_df["vehicle_capacity"].fillna(0)
    capacity_df["dock_capacity"] = capacity_df["dock_capacity"].fillna(0)

    capacity_df["vehicle_capacity_norm"] = _minmax_norm(capacity_df["vehicle_capacity"]).round(5)
    capacity_df["dock_capacity_norm"] = _minmax_norm(capacity_df["dock_capacity"]).round(5)

    capacity_df["occupancy_rate"] = capacity_df.apply(
        lambda r: (r["vehicle_capacity"] / r["total_capacity"]) if r["total_capacity"] else 0.0, axis=1
    )
    capacity_df["return_pressure"] = capacity_df.apply(
        lambda r: 1.0 - (r["dock_capacity"] / r["total_capacity"]) if r["total_capacity"] else 0.0, axis=1
    )

    results["capacity_df"] = capacity_df
    results["peak_time_slot"] = peak_time_slot
    results["peak_metric"] = peak_metric
    if save_outputs:
        capacity_df.to_csv(out_dir / "capacity_tract_with_vehicle_and_docks_norm.csv", index=False)

    # ==================================================
    # SAFETY (CRS-safe: uses blocks CRS)
    # ==================================================
    blocks_gdf = gpd.read_file(str(census_blocks_path))
    if blocks_gdf.crs is None:
        raise ValueError(f"Blocks shapefile CRS is missing: {census_blocks_path}")
    if blocks_id_col not in blocks_gdf.columns:
        raise ValueError(f"Blocks shapefile missing '{blocks_id_col}' column.")

    target_crs = blocks_gdf.crs
    blocks_gdf = blocks_gdf.to_crs(target_crs)

    safety_kind = str(cfg.get("safety_input_kind", "shp"))
    streets_gdf = _read_lines_layer(
        centerline_path,
        kind=safety_kind,
        target_crs=target_crs,
        csv_wkt_candidates=tuple(cfg.get("safety_csv_wkt_candidates", ("the_geom", "wkt"))),
        csv_crs=str(cfg.get("safety_csv_crs", "EPSG:4326")),
    )
    bike_gdf = _read_lines_layer(
        bike_lanes_path,
        kind=safety_kind,
        target_crs=target_crs,
        csv_wkt_candidates=tuple(cfg.get("safety_csv_wkt_candidates", ("the_geom", "wkt"))),
        csv_crs=str(cfg.get("safety_csv_crs", "EPSG:4326")),
    )

    street_len = _overlay_length_by_block(streets_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="streets_leng")
    bike_len = _overlay_length_by_block(bike_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="total_bike_lane_length")

    protected_rule = cfg.get("protected_lane_rule", {})
    facility_col, protected_gdf = _protected_filter(bike_gdf, protected_rule)

    if facility_col is None:
        prot_len = pd.DataFrame({"census_block": street_len["census_block"].astype(str).unique()})
        prot_len["protected_bike_lane_length"] = 0.0
        results["safety_note"] = (
            f"[{city_key}] No protected-lane column found. Protected length set to 0. "
            f"Tried: {protected_rule.get('col_candidates')}"
        )
    else:
        prot_len = _overlay_length_by_block(protected_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="protected_bike_lane_length")
        results["safety_protected_lane_col_used"] = facility_col
        results["safety_protected_lane_value_used"] = protected_rule.get("match_value")

    safety_block = (
        street_len.merge(bike_len, on="census_block", how="left")
        .merge(prot_len, on="census_block", how="left")
    )
    safety_block["total_bike_lane_length"] = safety_block["total_bike_lane_length"].fillna(0)
    safety_block["protected_bike_lane_length"] = safety_block["protected_bike_lane_length"].fillna(0)
    safety_block["bike_lane_ratio"] = _safe_ratio(safety_block["total_bike_lane_length"], safety_block["streets_leng"]).round(3)
    safety_block["protected_bike_lane_ratio"] = _safe_ratio(safety_block["protected_bike_lane_length"], safety_block["streets_leng"]).round(3)
    safety_block["census_tract"] = _tract_from_block(safety_block["census_block"]).astype(str)

    safety_tract = (
        safety_block.groupby("census_tract", as_index=False)[
            ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]
        ].sum()
    )

    centroids_csv_df = pd.read_csv(centroid_tract_csv)
    centroids_csv_df["census_tract"] = centroids_csv_df["census_tract"].astype(str)
    region_cols = ["census_tract"]
    for c in ["boro", "county_name", "county", "state"]:
        if c in centroids_csv_df.columns:
            region_cols.append(c)

    safety_tract = centroids_csv_df[region_cols].merge(safety_tract, on="census_tract", how="left")
    for c in ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]:
        safety_tract[c] = safety_tract[c].fillna(0)

    safety_tract["bike_lane_ratio"] = _safe_ratio(safety_tract["total_bike_lane_length"], safety_tract["streets_leng"])
    safety_tract["protected_bike_lane_ratio"] = _safe_ratio(safety_tract["protected_bike_lane_length"], safety_tract["streets_leng"])
    safety_tract = safety_tract.sort_values("census_tract").reset_index(drop=True)

    service_tracts = set(capacity_df["census_tract"].astype(str).unique().tolist())
    safety_service = safety_tract[safety_tract["census_tract"].astype(str).isin(service_tracts)].reset_index(drop=True)

    safety_norm = safety_service.copy()
    safety_norm["bike_lane_ratio_norm"] = _minmax_norm(safety_norm["bike_lane_ratio"]).round(5)
    safety_norm["protected_bike_lane_ratio_norm"] = _minmax_norm(safety_norm["protected_bike_lane_ratio"]).round(5)

    results["safety_bike_lane_block"] = safety_block
    results["safety_bike_lane_tract"] = safety_tract
    results["safety_bike_lane_service_area"] = safety_service
    results["safety_bike_lane_norm_tract"] = safety_norm

    if save_outputs:
        safety_block.to_csv(out_dir / "safety_bike_lane_block.csv", index=False)
        safety_tract.to_csv(out_dir / "safety_bike_lane_tract.csv", index=False)
        safety_service.to_csv(out_dir / "safety_bike_lane_service_area.csv", index=False)
        safety_norm.to_csv(out_dir / "safety_bike_lane_norm_tract.csv", index=False)

    # ==================================================
    # TRIPS — load + standardize + compute start/end census blocks if missing
    # ==================================================
    trip_raw = pd.read_csv(trip_csv)
    trip_df = _standardize_trips(trip_raw)

    # parse dates
    trip_df["started_at"] = pd.to_datetime(trip_df["started_at"], errors="coerce")
    trip_df["ended_at"] = pd.to_datetime(trip_df["ended_at"], errors="coerce")

    # compute start/end census blocks if missing
    if ("start_census_block" not in trip_df.columns) or ("end_census_block" not in trip_df.columns):
        if not {"start_lat", "start_lng", "end_lat", "end_lng"}.issubset(trip_df.columns):
            raise ValueError(
                "Trip CSV does not have start/end census blocks AND does not have start/end lat/lng to compute them. "
                f"Found: {list(trip_df.columns)[:200]}"
            )

        start_map = _geocode_points_to_blocks_spatial(
            trip_df, lon_col="start_lng", lat_col="start_lat",
            blocks_path=census_blocks_path, blocks_id_col=blocks_id_col
        ).rename(columns={"census_block": "start_census_block"})

        end_map = _geocode_points_to_blocks_spatial(
            trip_df, lon_col="end_lng", lat_col="end_lat",
            blocks_path=census_blocks_path, blocks_id_col=blocks_id_col
        ).rename(columns={"census_block": "end_census_block"})

        trip_df = trip_df.merge(
            start_map, left_on=["start_lng", "start_lat"], right_on=["lon", "lat"], how="left"
        ).drop(columns=["lon", "lat"], errors="ignore")

        trip_df = trip_df.merge(
            end_map, left_on=["end_lng", "end_lat"], right_on=["lon", "lat"], how="left"
        ).drop(columns=["lon", "lat"], errors="ignore")

    required_trip_cols = {"started_at", "ended_at", "start_census_block", "end_census_block"}
    miss_trip = required_trip_cols - set(trip_df.columns)
    if miss_trip:
        raise ValueError(
            f"Trip CSV is missing required columns after standardization: {miss_trip}. "
            f"Available columns: {list(trip_df.columns)[:200]}"
        )

    trip_df["start_census_block"] = trip_df["start_census_block"].astype(str)
    trip_df["end_census_block"] = trip_df["end_census_block"].astype(str)

    # default usage/idle windows
    if usage_time_start is None:
        usage_time_start = availability_time_start
    if usage_time_end is None:
        usage_time_end = availability_time_end
    if idle_time_start is None:
        idle_time_start = availability_time_start
    if idle_time_end is None:
        idle_time_end = availability_time_end

    if usage_time_start is None:
        usage_time_start = trip_df["started_at"].min()
    if usage_time_end is None:
        usage_time_end = trip_df["started_at"].max()
    if idle_time_start is None:
        idle_time_start = trip_df["started_at"].min()
    if idle_time_end is None:
        idle_time_end = trip_df["started_at"].max()

    usage_time_start = pd.to_datetime(usage_time_start)
    usage_time_end = pd.to_datetime(usage_time_end)
    idle_time_start = pd.to_datetime(idle_time_start)
    idle_time_end = pd.to_datetime(idle_time_end)

    # ==================================================
    # USAGE (5-min, hourly, tract-hourly norm)
    # ==================================================
    trips = trip_df.copy()
    trips = trips[(trips["started_at"] >= usage_time_start) & (trips["started_at"] <= usage_time_end)].copy()
    trips["trip_duration"] = (trips["ended_at"] - trips["started_at"]).dt.total_seconds() / 60.0
    trips = trips[trips["trip_duration"] <= 240].copy()

    trips["time_slot_start"] = trips["started_at"].dt.floor("5min")
    trips["time_slot_end"] = trips["ended_at"].dt.floor("5min")

    trips_starting_5 = trips.groupby(["start_census_block", "time_slot_start"]).size().reset_index(name="trips_starting")
    trips_ending_5 = trips.groupby(["end_census_block", "time_slot_end"]).size().reset_index(name="trips_ending")

    trips_starting_5 = trips_starting_5.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})
    trips_ending_5 = trips_ending_5.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})

    all_blocks = pd.unique(pd.concat([trips["start_census_block"], trips["end_census_block"]]).astype(str))
    all_blocks = [b for b in all_blocks if b not in ("nan", "0")]

    all_time_slots_5 = pd.date_range(start=usage_time_start.floor("5min"), end=usage_time_end.ceil("5min"), freq="5min")
    grid_5 = pd.MultiIndex.from_product([all_blocks, all_time_slots_5], names=["census_block", "time_slot"]).to_frame(index=False)

    usage_5min_block = (
        grid_5.merge(trips_starting_5, on=["census_block", "time_slot"], how="left")
             .merge(trips_ending_5, on=["census_block", "time_slot"], how="left")
    )
    usage_5min_block[["trips_starting", "trips_ending"]] = usage_5min_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
    usage_5min_block = usage_5min_block.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

    if save_outputs:
        usage_5min_block.to_csv(out_dir / "usage_5min_block.csv", index=False)

    trips["hour_start"] = trips["started_at"].dt.floor("H")
    trips["hour_end"] = trips["ended_at"].dt.floor("H")

    trips_starting_h = trips.groupby(["start_census_block", "hour_start"]).size().reset_index(name="trips_starting")
    trips_ending_h = trips.groupby(["end_census_block", "hour_end"]).size().reset_index(name="trips_ending")

    trips_starting_h = trips_starting_h.rename(columns={"start_census_block": "census_block", "hour_start": "hour"})
    trips_ending_h = trips_ending_h.rename(columns={"end_census_block": "census_block", "hour_end": "hour"})

    all_hours = pd.date_range(start=usage_time_start.floor("H"), end=usage_time_end.ceil("H"), freq="H")
    grid_h = pd.MultiIndex.from_product([all_blocks, all_hours], names=["census_block", "hour"]).to_frame(index=False)

    usage_hourly_block = (
        grid_h.merge(trips_starting_h, on=["census_block", "hour"], how="left")
             .merge(trips_ending_h, on=["census_block", "hour"], how="left")
    )
    usage_hourly_block[["trips_starting", "trips_ending"]] = usage_hourly_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
    usage_hourly_block = usage_hourly_block.sort_values(["census_block", "hour"]).reset_index(drop=True)

    if save_outputs:
        usage_hourly_block.to_csv(out_dir / "usage_hourly_block.csv", index=False)

    usage_hourly = usage_hourly_block.copy()
    usage_hourly["census_block"] = usage_hourly["census_block"].astype(str)
    usage_hourly["census_tract"] = _tract_from_block(usage_hourly["census_block"]).astype(str)

    if external_tract_prefix:
        usage_hourly = usage_hourly[~usage_hourly["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    usage_hourly["time_slot"] = pd.to_datetime(usage_hourly["hour"])
    usage_hourly = usage_hourly.drop(columns=["hour"])

    usage_hourly_tract = (
        usage_hourly.groupby(["census_tract", "time_slot"], as_index=False)
        .agg(trips_starting=("trips_starting", "sum"), trips_ending=("trips_ending", "sum"))
        .reset_index(drop=True)
    )
    usage_hourly_tract_raw = usage_hourly_tract.copy()

    if tracts_to_remove:
        usage_hourly_tract = usage_hourly_tract[~usage_hourly_tract["census_tract"].isin(tracts_to_remove)].reset_index(drop=True)

    usage_hourly_tract["trips_starting_norm"] = _minmax_norm(usage_hourly_tract["trips_starting"]).round(5)
    usage_hourly_tract["trips_ending_norm"] = _minmax_norm(usage_hourly_tract["trips_ending"]).round(5)

    if save_outputs:
        usage_hourly_tract_raw.to_csv(out_dir / "usage_hourly_tract_raw.csv", index=False)
        usage_hourly_tract.to_csv(out_dir / "usage_norm_hourly_tract.csv", index=False)

    results["usage_5min_block"] = usage_5min_block
    results["usage_hourly_block"] = usage_hourly_block
    results["usage_hourly_tract_raw"] = usage_hourly_tract_raw
    results["usage_norm_hourly_tract"] = usage_hourly_tract

    # ==================================================
    # IDLE TIME (optional; skip if no usable bike-type)
    # ==================================================
    if not compute_idle_time:
        results["idle_note"] = "Idle time skipped (compute_idle_time=False)."
        return results

    if "rideable_type" not in trip_df.columns:
        results["idle_note"] = "Idle time skipped: trip file has no rideable_type."
        return results

    # CitiBike-style idle expects classic_bike/electric_bike; PIT 'Rider Type' isn't that.
    # If values don't look like bike types, skip (prevents false results).
    vals = pd.Series(trip_df["rideable_type"].dropna().astype(str).unique())
    looks_like_bike_type = vals.str.contains("bike|electric|classic|ebike", case=False, na=False).any()
    if not looks_like_bike_type:
        results["idle_note"] = f"Idle time skipped: rideable_type values don't look like bike types. Sample: {vals.head(10).tolist()}"
        return results

    from collections import deque

    ss_idle = ss_done.copy()
    ss_idle["timestamp"] = pd.to_datetime(ss_idle["timestamp"])
    ss_idle["census_block"] = ss_idle["census_block"].astype(str)
    ss_idle["census_tract"] = ss_idle["census_tract"].astype(str)
    ss_idle["total_available"] = ss_idle["total_vehicle_available"]
    ss_idle["time_5min"] = ss_idle["timestamp"].dt.floor("5min")

    trips_idle = trip_df.copy()
    trips_idle["started_at"] = pd.to_datetime(trips_idle["started_at"])
    trips_idle["ended_at"] = pd.to_datetime(trips_idle["ended_at"])
    trips_idle["start_census_block"] = trips_idle["start_census_block"].astype(str)
    trips_idle["end_census_block"] = trips_idle["end_census_block"].astype(str)

    trips_idle = trips_idle[(trips_idle["started_at"] >= idle_time_start) & (trips_idle["started_at"] <= idle_time_end)].copy()
    trips_idle["time_slot_start"] = trips_idle["started_at"].dt.floor("5min")
    trips_idle["time_slot_end"] = trips_idle["ended_at"].dt.floor("5min")

    ts_start = (
        trips_idle.groupby(["start_census_block", "time_slot_start", "rideable_type"])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )
    ts_start = ts_start.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})

    ts_end = (
        trips_idle.groupby(["end_census_block", "time_slot_end", "rideable_type"])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )
    ts_end = ts_end.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})

    # Force classic/electric columns if present; otherwise treat all as generic (still usable)
    for col in ["classic_bike", "electric_bike"]:
        if col not in ts_start.columns:
            ts_start[col] = 0
        if col not in ts_end.columns:
            ts_end[col] = 0

    ts_start = ts_start.rename(columns={"classic_bike": "trips_starting_bike", "electric_bike": "trips_starting_ebike"})
    ts_end = ts_end.rename(columns={"classic_bike": "trips_ending_bike", "electric_bike": "trips_ending_ebike"})

    all_blocks_idle = pd.unique(pd.concat([trips_idle["start_census_block"], trips_idle["end_census_block"]]))
    all_blocks_idle = pd.Series(all_blocks_idle).dropna().astype(str).unique()
    all_time_slots_idle = pd.date_range(start=idle_time_start.floor("5min"), end=idle_time_end.ceil("5min"), freq="5min")

    grid_idle = pd.MultiIndex.from_product([all_blocks_idle, all_time_slots_idle], names=["census_block", "time_slot"]).to_frame(index=False)

    result_complete = (
        grid_idle.merge(ts_start[["census_block", "time_slot", "trips_starting_bike", "trips_starting_ebike"]],
                        on=["census_block", "time_slot"], how="left")
                .merge(ts_end[["census_block", "time_slot", "trips_ending_bike", "trips_ending_ebike"]],
                       on=["census_block", "time_slot"], how="left")
    )
    for c in ["trips_starting_bike", "trips_starting_ebike", "trips_ending_bike", "trips_ending_ebike"]:
        result_complete[c] = result_complete[c].fillna(0).astype(int)

    result_complete["trips_starting"] = result_complete["trips_starting_bike"] + result_complete["trips_starting_ebike"]
    result_complete["trips_ending"] = result_complete["trips_ending_bike"] + result_complete["trips_ending_ebike"]
    result_complete = result_complete.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

    station_availability_df = (
        ss_idle.groupby(["census_block", "time_5min"], as_index=False)[["total_available"]].sum()
    ).rename(columns={"time_5min": "time_slot"})
    station_availability_df["time_slot"] = station_availability_df["time_slot"] - pd.Timedelta(minutes=5)

    merged_idle = station_availability_df.merge(result_complete, on=["census_block", "time_slot"], how="left")
    merged_idle[["trips_starting", "trips_ending"]] = merged_idle[["trips_starting", "trips_ending"]].fillna(0).astype(int)
    merged_idle["vehicles_moved"] = merged_idle["trips_starting"] + merged_idle["trips_ending"]
    merged_idle["idle_vehicles"] = (merged_idle["total_available"] - merged_idle["vehicles_moved"]).clip(lower=0)

    merged_idle["time_slot"] = pd.to_datetime(merged_idle["time_slot"])
    merged_idle = merged_idle.sort_values(["census_block", "time_slot"])
    merged_idle["hour"] = merged_idle["time_slot"].dt.floor("H")

    idle_results = []
    for block, block_df in merged_idle.groupby("census_block"):
        for hour, hour_df in block_df.groupby("hour"):
            hour_df = hour_df.sort_values("time_slot")
            virtual_vehicles = deque()
            durations = []

            for _, row in hour_df.iterrows():
                current_time = row["time_slot"]
                current_idle = int(row["idle_vehicles"])

                if current_idle > len(virtual_vehicles):
                    for _ in range(current_idle - len(virtual_vehicles)):
                        virtual_vehicles.append(current_time)
                elif current_idle < len(virtual_vehicles):
                    for _ in range(len(virtual_vehicles) - current_idle):
                        start_t = virtual_vehicles.popleft()
                        durations.append((current_time - start_t).total_seconds() / 60.0)

            final_time = hour + pd.Timedelta(hours=1)
            while virtual_vehicles:
                start_t = virtual_vehicles.popleft()
                durations.append((final_time - start_t).total_seconds() / 60.0)

            avg_idle_time = round(sum(durations) / len(durations), 2) if durations else 0.0
            idle_results.append({"census_block": block, "time_slot": hour, "avg_idle_time": avg_idle_time})

    idle_time_block = pd.DataFrame(idle_results)
    idle_time_block["census_block"] = idle_time_block["census_block"].astype(str)
    idle_time_block["census_tract"] = _tract_from_block(idle_time_block["census_block"]).astype(str)

    idle_time_tract = idle_time_block.groupby(["census_tract", "time_slot"], as_index=False)["avg_idle_time"].mean()
    if external_tract_prefix:
        idle_time_tract = idle_time_tract[~idle_time_tract["census_tract"].astype(str).str.startswith(external_tract_prefix)].reset_index(drop=True)

    idle_time_tract["avg_idle_time_norm"] = _minmax_norm(idle_time_tract["avg_idle_time"]).round(5)

    results["idle_merged_5min"] = merged_idle
    results["idle_time_block"] = idle_time_block
    results["idle_time_tract_raw"] = idle_time_tract.drop(columns=["avg_idle_time_norm"])
    results["idle_time_norm_tract"] = idle_time_tract

    if save_outputs:
        merged_idle.to_csv(out_dir / "idle_merged_5min.csv", index=False)
        idle_time_block.to_csv(out_dir / "idle_time_block.csv", index=False)
        results["idle_time_tract_raw"].to_csv(out_dir / "idle_time_tract_raw.csv", index=False)
        idle_time_tract.to_csv(out_dir / "idle_time_norm_tract.csv", index=False)

    return results


In [20]:
from __future__ import annotations

from pathlib import Path
from typing import Optional, Union, Dict, Any, Sequence, Tuple
import json
import pandas as pd


# ==================================================
# CITY CONFIG (hardcoded assets for NYC + NJ + PITT + SF)
# ==================================================
CITY_CONFIG: Dict[str, Dict[str, Any]] = {
    "NYC": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NYC\tl_2024_36_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NYC\tl_2024_36_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Capacity_NYC\centroid_tract_computed.csv"
        ),
        # safety (NYC = CSV WKT)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\CSCL_PlowNYC_20250619.csv"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\New_York_City_Bike_Routes_20250619.csv"
        ),
        "safety_input_kind": "csv_wkt",
        "safety_csv_wkt_candidates": ("geometry", "the_geom", "wkt", "geometry_wkt", "WKT", "geom", "shape", "line"),
        "safety_csv_crs": "EPSG:4326",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": "34",  # drop NJ tracts for NYC results
        "has_boro_logic": True,
        "drop_staten_island_default": True,
        "protected_lane_rule": {
            "col_candidates": ("facilitycl", "facility", "class", "ft", "type"),
            "match_type": "equals",
            "match_value": "I",
        },
    },
    "NJ": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\tl_2024_34_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\tl_2024_34_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv"
        ),
        # safety (NJ = SHP)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\Tran_road.shp"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp"
        ),
        "safety_input_kind": "shp",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        "protected_lane_rule": {
            "col_candidates": ("type", "facility", "facilitycl", "class", "lane_type"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
    "PITT": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\tl_2024_42_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\tl_2024_42_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\centroid_tract_pa.csv"
        ),
        # safety (PITT = SHP)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Pittsburgh_Street_Centerline.shp"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Bike Map\Bike Lanes.shp"
        ),
        "safety_input_kind": "shp",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        "protected_lane_rule": {
            "col_candidates": ("facility", "type", "class", "status", "lane_type", "BIKE_FACIL", "CATEGORY"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
    "SF": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Usage\San_Fran_Baywheels\tl_2024_06_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\San_Fran_Baywheels\tl_2024_06_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\San_Fran_Baywheels\centroid_tract_ca.csv"
        ),
        # safety (SF = CSV WKT)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Streets___Active_and_Retired_20250626 (1).csv"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Bikelane.csv"
        ),
        "safety_input_kind": "csv_wkt",
        "safety_csv_wkt_candidates": ("geometry", "shape", "line", "the_geom", "wkt", "geometry_wkt", "WKT", "geom"),
        "safety_csv_crs": "EPSG:4326",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        # protected heuristic: non-empty BARRIER OR keywords in FACILITY_T
        "protected_lane_rule": {
            "col_candidates": ("BARRIER", "FACILITY_T", "BUFFERED", "RAISED", "SYMBOLOGY"),
            "match_type": "not_empty_or_contains",
            "match_value": r"PROTECT|SEPARAT|CYCLETRACK|BARRIER|RAISED|BUFFER",
        },
    },
}


# ==================================================
# Helpers
# ==================================================
def _ensure_exists(p: Path, label: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"{label} not found: {p}")


def _minmax_norm(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    mn = s.min(skipna=True)
    mx = s.max(skipna=True)
    if pd.isna(mn) or pd.isna(mx) or mx <= mn:
        return pd.Series(0.0, index=s.index)
    return (s - mn) / (mx - mn)


def _safe_ratio(numer: pd.Series, denom: pd.Series) -> pd.Series:
    return numer.div(denom.where(denom > 0, other=pd.NA)).fillna(0)


def _pick_col(df: pd.DataFrame, candidates: Sequence[str], required: bool, label: str) -> Optional[str]:
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    if required:
        raise ValueError(
            f"Could not find required column for {label}. Tried {list(candidates)}. "
            f"Found columns (sample): {list(df.columns)[:200]}"
        )
    return None


def _tract_from_block(block_series: pd.Series) -> pd.Series:
    s = block_series.astype(str)
    # If it looks like a 15-digit block GEOID -> tract is first 11 OR strip last 4
    # We use strip-last-4 which matches your previous logic.
    return s.where(s.str.len() < 15, s.str[:-4])


def _standardize_station_info(df: pd.DataFrame) -> pd.DataFrame:
    sid = _pick_col(df, ["station_id", "stationId", "id", "Station ID", "Station Id"], True, "station_id")
    lat = _pick_col(df, ["lat", "latitude", "station_lat", "y"], True, "lat")
    lon = _pick_col(df, ["lon", "lng", "longitude", "station_lon", "x"], True, "lon")
    cap = _pick_col(df, ["capacity", "dock_count", "num_docks", "total_docks"], True, "capacity")

    out = df.copy().rename(columns={sid: "station_id", lat: "lat", lon: "lon", cap: "capacity"})
    out["station_id"] = out["station_id"].astype(str).str.strip()

    cb = _pick_col(out, ["census_block", "block_geoid", "GEOID20"], False, "census_block")
    if cb and cb != "census_block":
        out = out.rename(columns={cb: "census_block"})

    ct = _pick_col(out, ["census_tract", "tract_geoid", "GEOID"], False, "census_tract")
    if ct and ct != "census_tract":
        out = out.rename(columns={ct: "census_tract"})

    return out


def _standardize_trips(df: pd.DataFrame) -> pd.DataFrame:
    started = _pick_col(
        df,
        ["started_at", "start_time", "trip_start_time", "starttime", "Start Date", "start_date", "StartDate"],
        True,
        "started_at",
    )
    ended = _pick_col(
        df,
        ["ended_at", "end_time", "trip_end_time", "stoptime", "End Date", "end_date", "EndDate"],
        True,
        "ended_at",
    )
    out = df.copy().rename(columns={started: "started_at", ended: "ended_at"})

    rt = _pick_col(out, ["rideable_type", "bike_type", "vehicle_type", "Rider Type", "rider_type"], False, "rideable_type")
    if rt and rt != "rideable_type":
        out = out.rename(columns={rt: "rideable_type"})

    # precomputed blocks if present
    sb = _pick_col(out, ["start_census_block", "start_block", "start_block_geoid"], False, "start_census_block")
    eb = _pick_col(out, ["end_census_block", "end_block", "end_block_geoid"], False, "end_census_block")
    if sb and sb != "start_census_block":
        out = out.rename(columns={sb: "start_census_block"})
    if eb and eb != "end_census_block":
        out = out.rename(columns={eb: "end_census_block"})

    # lat/lon fallbacks
    slat = _pick_col(out, ["start_lat", "startLatitude", "StartLat"], False, "start_lat")
    slng = _pick_col(out, ["start_lng", "start_lon", "startLongitude", "StartLng"], False, "start_lng")
    elat = _pick_col(out, ["end_lat", "endLatitude", "EndLat"], False, "end_lat")
    elng = _pick_col(out, ["end_lng", "end_lon", "endLongitude", "EndLng"], False, "end_lng")
    if slat and slat != "start_lat":
        out = out.rename(columns={slat: "start_lat"})
    if slng and slng != "start_lng":
        out = out.rename(columns={slng: "start_lng"})
    if elat and elat != "end_lat":
        out = out.rename(columns={elat: "end_lat"})
    if elng and elng != "end_lng":
        out = out.rename(columns={elng: "end_lng"})

    # station ids (optional)
    ssid = _pick_col(out, ["start_station_id", "Start Station Id", "Start Station ID"], False, "start_station_id")
    esid = _pick_col(out, ["end_station_id", "End Station Id", "End Station ID"], False, "end_station_id")
    if ssid and ssid != "start_station_id":
        out = out.rename(columns={ssid: "start_station_id"})
    if esid and esid != "end_station_id":
        out = out.rename(columns={esid: "end_station_id"})

    return out


def _read_lines_layer(
    path: Path,
    *,
    kind: str,
    target_crs,
    csv_wkt_candidates: Tuple[str, ...],
    csv_crs: str,
):
    """
    Robust for SF/NYC CSV-WKT:
      - auto-picks the WKT column by sampling values
      - safe parses WKT (drops non-string/NaN)
    """
    import geopandas as gpd
    from shapely import wkt

    if kind == "shp":
        gdf = gpd.read_file(str(path))
        if gdf.crs is None:
            raise ValueError(f"CRS missing for {path}. Define CRS before projecting.")
        return gdf.to_crs(target_crs)

    df = pd.read_csv(path)

    def _looks_like_wkt(v: Any) -> bool:
        if not isinstance(v, str):
            return False
        s = v.strip().upper()
        return s.startswith(("LINESTRING", "MULTILINESTRING", "GEOMETRYCOLLECTION"))

    wkt_col = None
    for c in csv_wkt_candidates:
        if c in df.columns:
            sample = df[c].dropna().astype(str).head(80)
            if not sample.empty and sample.map(_looks_like_wkt).mean() >= 0.2:
                wkt_col = c
                break

    if wkt_col is None:
        raise ValueError(
            f"Could not find a usable WKT column in {path.name}. "
            f"Tried candidates: {list(csv_wkt_candidates)}. "
            f"Found columns: {list(df.columns)[:120]}"
        )

    def _safe_wkt_load(x: Any):
        if not isinstance(x, str):
            return None
        x = x.strip()
        if not x:
            return None
        try:
            return wkt.loads(x)
        except Exception:
            return None

    df = df.copy()
    df["geometry"] = df[wkt_col].apply(_safe_wkt_load)
    df = df[df["geometry"].notna()].copy()
    if df.empty:
        raise ValueError(
            f"After parsing WKT, no valid geometries remained for {path.name}. "
            f"Chosen WKT column: '{wkt_col}'."
        )

    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=csv_crs)
    return gdf.to_crs(target_crs)


def _overlay_length_by_block(lines_gdf, blocks_gdf, *, blocks_id_col: str, out_col: str) -> pd.DataFrame:
    import geopandas as gpd

    blocks = blocks_gdf[[blocks_id_col, "geometry"]].copy()
    lines = lines_gdf[["geometry"]].copy()

    inter = gpd.overlay(lines, blocks, how="intersection", keep_geom_type=False)
    if inter.empty:
        return pd.DataFrame({"census_block": pd.Series([], dtype=str), out_col: pd.Series([], dtype=float)})

    inter["seg_len"] = inter.geometry.length
    inter = inter.rename(columns={blocks_id_col: "census_block"})
    out = inter.groupby("census_block", as_index=False)["seg_len"].sum().rename(columns={"seg_len": out_col})
    out[out_col] = out[out_col].round(3)
    out["census_block"] = out["census_block"].astype(str)
    return out


def _protected_filter(lines_gdf, rule: Dict[str, Any]):
    """
    Supported match_type:
      - equals
      - contains
      - not_empty
      - not_empty_or_contains
    """
    col = _pick_col(lines_gdf, list(rule.get("col_candidates", [])), required=False, label="protected lane col")
    if col is None:
        return None, lines_gdf

    s = lines_gdf[col].astype(str)
    match_type = str(rule.get("match_type", "equals")).lower()
    match_value = str(rule.get("match_value", ""))

    if match_type == "equals":
        mask = s.str.upper() == match_value.upper()
    elif match_type == "contains":
        mask = s.str.upper().str.contains(match_value.upper(), na=False)
    elif match_type == "not_empty":
        s2 = lines_gdf[col]
        mask = s2.notna() & (s.astype(str).str.strip() != "") & (~s.astype(str).str.lower().isin(["0", "false", "none", "nan"]))
    elif match_type == "not_empty_or_contains":
        s2 = lines_gdf[col]
        non_empty = s2.notna() & (s.astype(str).str.strip() != "") & (~s.astype(str).str.lower().isin(["0", "false", "none", "nan"]))
        contains = s.astype(str).str.contains(match_value, case=False, na=False, regex=True)
        mask = non_empty | contains
    else:
        raise ValueError(f"Unsupported match_type='{match_type}' in protected_lane_rule")

    return col, lines_gdf[mask].copy()


def _geocode_points_to_blocks_spatial(
    df: pd.DataFrame,
    *,
    lon_col: str,
    lat_col: str,
    blocks_path: Path,
    blocks_id_col: str,
) -> pd.DataFrame:
    import geopandas as gpd
    from shapely.geometry import Point

    blocks = gpd.read_file(str(blocks_path)).to_crs(epsg=4326)
    if blocks_id_col not in blocks.columns:
        raise ValueError(f"Block shapefile missing '{blocks_id_col}' for spatial geocoding.")

    pts = df[[lon_col, lat_col]].dropna().drop_duplicates().copy()
    pts = pts.rename(columns={lon_col: "lon", lat_col: "lat"})

    gdf = gpd.GeoDataFrame(
        pts,
        geometry=[Point(xy) for xy in zip(pts["lon"], pts["lat"])],
        crs="EPSG:4326",
    )
    joined = gpd.sjoin(gdf, blocks[[blocks_id_col, "geometry"]], how="left", predicate="within")
    joined = joined.rename(columns={blocks_id_col: "census_block"})
    out = joined[["lon", "lat", "census_block"]].copy()
    out["census_block"] = out["census_block"].astype(str)
    return out


def _census_api_block(lat: float, lon: float, benchmark: str, vintage: str) -> Optional[str]:
    import requests
    try:
        url = (
            "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
            f"?x={lon}&y={lat}"
            f"&benchmark={benchmark}"
            f"&vintage={vintage}"
            "&format=json"
        )
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        js = r.json()
        blocks = js["result"]["geographies"].get("2020 Census Blocks", [])
        if not blocks:
            return None
        return blocks[0].get("GEOID")
    except Exception:
        return None


def _map_rideable_type_to_two_columns(trips: pd.DataFrame) -> pd.DataFrame:
    """
    Create columns:
      - trips_starting_bike, trips_starting_ebike
      - trips_ending_bike, trips_ending_ebike
    using rideable_type values like classic_bike/electric_bike (or similar strings).
    """
    df = trips.copy()
    if "rideable_type" not in df.columns:
        return df

    t = df["rideable_type"].astype(str).str.lower()
    df["_is_ebike"] = t.str.contains("electric|ebike|e-bike|assist", na=False)

    return df


# ==================================================
# MAIN FUNCTION
# ==================================================
def run_docked_all_utilities_single_function(
    *,
    city: str,
    station_status_txt: Union[str, Path],
    station_information_csv: Union[str, Path],
    trip_csv: Union[str, Path],
    output_dir: Optional[Union[str, Path]] = None,
    save_outputs: bool = True,
    compute_idle_time: bool = True,
    availability_time_start: Optional[Union[str, pd.Timestamp]] = None,
    availability_time_end: Optional[Union[str, pd.Timestamp]] = None,
    availability_time_granularity: str = "1H",
    availability_group_level: str = "both",
    peak_time_slot: Optional[Union[str, pd.Timestamp]] = None,
    peak_metric: str = "total_vehicle_available",
    drop_staten_island: Optional[bool] = None,
    usage_time_start: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_end: Optional[Union[str, pd.Timestamp]] = None,
    tracts_to_remove: Optional[Sequence[str]] = None,
    idle_time_start: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_end: Optional[Union[str, pd.Timestamp]] = None,
    remove_tz_suffix: str = " EDT",
    drop_cols: tuple[str, ...] = (
        "num_scooters_available",
        "num_scooters_unavailable",
        "is_installed",
        "is_returning",
        "is_renting",
    ),
    fill_missing_with_census_api: bool = True,
    census_geocoder_benchmark: str = "Public_AR_Census2020",
    census_geocoder_vintage: str = "Census2020_Current",
) -> Dict[str, Any]:
    """
    One-function multi-city docked pipeline (NYC/NJ/PITT/SF), including idle simulation.
    """

    # Normalize city keys
    city_key = city.strip().upper()
    if city_key in {"PITTSBURGH", "PIT"}:
        city_key = "PITT"
    if city_key in {"SANFRAN", "SAN_FRAN", "SAN FRANCISCO", "BAYWHEELS", "BAY WHEELS"}:
        city_key = "SF"

    if city_key not in CITY_CONFIG:
        raise ValueError(f"Unsupported city='{city}'. Supported: {list(CITY_CONFIG.keys())}")

    cfg = CITY_CONFIG[city_key]

    station_status_txt = Path(station_status_txt)
    station_information_csv = Path(station_information_csv)
    trip_csv = Path(trip_csv)

    census_blocks_path = Path(cfg["census_blocks_path"])
    tracts_path = Path(cfg["tracts_path"])
    centroid_tract_csv = Path(cfg["centroid_tract_csv"])
    centerline_path = Path(cfg["centerline_path"])
    bike_lanes_path = Path(cfg["bike_lanes_path"])

    blocks_id_col = str(cfg["blocks_id_col"])
    tract_id_col = str(cfg["tract_id_col"])
    external_tract_prefix = cfg.get("external_tract_prefix", None)

    if drop_staten_island is None:
        drop_staten_island = bool(cfg.get("drop_staten_island_default", False))

    out_dir = Path(output_dir) if output_dir is not None else Path(f"./{city_key}_ALL_IN_ONE_OUTPUTS")
    out_dir.mkdir(parents=True, exist_ok=True)

    # existence checks
    _ensure_exists(station_status_txt, "station_status_txt")
    _ensure_exists(station_information_csv, "station_information_csv")
    _ensure_exists(trip_csv, "trip_csv")
    _ensure_exists(census_blocks_path, "census_blocks_path")
    _ensure_exists(tracts_path, "tracts_path")
    _ensure_exists(centroid_tract_csv, "centroid_tract_csv")
    _ensure_exists(centerline_path, "centerline_path")
    _ensure_exists(bike_lanes_path, "bike_lanes_path")

    # normalize times
    availability_time_start = pd.to_datetime(availability_time_start) if availability_time_start is not None else None
    availability_time_end = pd.to_datetime(availability_time_end) if availability_time_end is not None else None
    usage_time_start = pd.to_datetime(usage_time_start) if usage_time_start is not None else None
    usage_time_end = pd.to_datetime(usage_time_end) if usage_time_end is not None else None
    idle_time_start = pd.to_datetime(idle_time_start) if idle_time_start is not None else None
    idle_time_end = pd.to_datetime(idle_time_end) if idle_time_end is not None else None

    availability_group_level = availability_group_level.lower().strip()
    if availability_group_level not in {"block", "tract", "both"}:
        raise ValueError("availability_group_level must be one of: 'block', 'tract', 'both'")

    if peak_metric not in {"total_vehicle_available", "num_docks_available"}:
        raise ValueError("peak_metric must be 'total_vehicle_available' or 'num_docks_available'")

    results: Dict[str, Any] = {
        "city": city_key,
        "system_type": "docked",
        "output_dir": str(out_dir),
        "city_assets": {
            "census_blocks_path": str(census_blocks_path),
            "tracts_path": str(tracts_path),
            "centroid_tract_csv": str(centroid_tract_csv),
            "centerline_path": str(centerline_path),
            "bike_lanes_path": str(bike_lanes_path),
        },
    }

    # ==================================================
    # STEP 0 — station_status JSONL flatten
    # ==================================================
    with station_status_txt.open("r", encoding="utf-8") as f:
        lines = f.readlines()

    di_list = []
    for item in lines:
        if not item.strip():
            continue
        js = json.loads(item)
        timestamp = list(js.keys())[0]
        for entry in js[timestamp]:
            for vehicle_type in entry.get("vehicle_types_available", []):
                entry[f"vehicle_type_{vehicle_type['vehicle_type_id']}_count"] = vehicle_type["count"]
            if "vehicle_types_available" in entry:
                del entry["vehicle_types_available"]
            entry["timestamp"] = timestamp
            di_list.append(entry)

    station_status_flat = pd.DataFrame(di_list)
    results["station_status_flat"] = station_status_flat
    if save_outputs:
        station_status_flat.to_csv(out_dir / f"{city_key.lower()}_station_status_flat.csv", index=False)

    ss_df = station_status_flat.copy()
    ss_df["timestamp"] = ss_df["timestamp"].astype(str).str.replace(remove_tz_suffix, "", regex=False)
    ss_df["timestamp"] = pd.to_datetime(ss_df["timestamp"], errors="coerce")
    ss_df["hour"] = ss_df["timestamp"].dt.hour
    ss_df["date"] = ss_df["timestamp"].dt.date
    cols_to_drop = [c for c in drop_cols if c in ss_df.columns]
    if cols_to_drop:
        ss_df = ss_df.drop(cols_to_drop, axis=1)

    if "station_id" not in ss_df.columns:
        raise ValueError("station_status data missing station_id after flattening.")
    ss_df["station_id"] = ss_df["station_id"].astype(str).str.strip()

    results["station_status_clean"] = ss_df
    if save_outputs:
        ss_df.to_csv(out_dir / f"{city_key.lower()}_station_status_clean.csv", index=False)

    # ==================================================
    # STEP 0 — station_information -> standardized + census block/tract
    # ==================================================
    si_raw = pd.read_csv(station_information_csv)
    results["station_info_raw"] = si_raw
    if save_outputs:
        si_raw.to_csv(out_dir / f"{city_key.lower()}_station_information_raw.csv", index=False)

    si = _standardize_station_info(si_raw)

    # If census_block missing, compute via spatial join
    if ("census_block" not in si.columns) or si["census_block"].isna().any():
        import geopandas as gpd
        from shapely.geometry import Point

        pts = si.drop_duplicates(subset=["lat", "lon"])[["lon", "lat"]].copy()
        pts_gdf = gpd.GeoDataFrame(
            pts,
            geometry=[Point(xy) for xy in zip(pts["lon"], pts["lat"])],
            crs="EPSG:4326",
        )

        blocks_wgs84 = gpd.read_file(str(census_blocks_path)).to_crs(epsg=4326)
        if blocks_id_col not in blocks_wgs84.columns:
            raise ValueError(f"Block shapefile missing '{blocks_id_col}' for station geocoding.")

        joined = gpd.sjoin(pts_gdf, blocks_wgs84[[blocks_id_col, "geometry"]], how="left", predicate="within")
        joined = joined.rename(columns={blocks_id_col: "census_block"})
        joined["census_block"] = joined["census_block"].astype(str)
        joined["census_tract"] = _tract_from_block(joined["census_block"]).astype(str)

        si = si.merge(joined[["lon", "lat", "census_block", "census_tract"]], on=["lon", "lat"], how="left", suffixes=("", "_sj"))

        if "census_block_sj" in si.columns:
            si["census_block"] = si.get("census_block").fillna(si["census_block_sj"]) if "census_block" in si.columns else si["census_block_sj"]
            si = si.drop(columns=["census_block_sj"], errors="ignore")

        if "census_tract_sj" in si.columns:
            if "census_tract" not in si.columns:
                si["census_tract"] = pd.NA
            si["census_tract"] = si["census_tract"].fillna(si["census_tract_sj"])
            si = si.drop(columns=["census_tract_sj"], errors="ignore")

        # Census API fill for remaining missing (optional)
        if fill_missing_with_census_api and ("census_block" in si.columns) and si["census_block"].isna().any():
            miss = si[si["census_block"].isna()][["lat", "lon"]].drop_duplicates().copy()
            miss["census_block_new"] = [
                _census_api_block(lat, lon, census_geocoder_benchmark, census_geocoder_vintage)
                for lat, lon in zip(miss["lat"], miss["lon"])
            ]
            miss["census_tract_new"] = _tract_from_block(miss["census_block_new"]).astype(str)

            si = si.merge(miss, on=["lat", "lon"], how="left")
            si["census_block"] = si["census_block"].fillna(si["census_block_new"])
            if "census_tract" not in si.columns:
                si["census_tract"] = pd.NA
            si["census_tract"] = si["census_tract"].fillna(si["census_tract_new"])
            si = si.drop(columns=["census_block_new", "census_tract_new"], errors="ignore")

    si["census_block"] = si["census_block"].astype(str)
    if "census_tract" not in si.columns or si["census_tract"].isna().any():
        si["census_tract"] = _tract_from_block(si["census_block"]).astype(str)
    else:
        si["census_tract"] = si["census_tract"].astype(str)

    results["station_info_done"] = si
    if save_outputs:
        si.to_csv(out_dir / f"{city_key.lower()}_station_information_done.csv", index=False)

    # Merge blocks into station status
    ss_done = ss_df.merge(si[["station_id", "census_block"]], on="station_id", how="left")
    ss_done["census_block"] = ss_done["census_block"].astype(str)
    ss_done["census_tract"] = _tract_from_block(ss_done["census_block"]).astype(str)

    # Availability column standardization (tolerant to missing ebikes)
    if "num_bikes_available" not in ss_done.columns:
        raise ValueError(
            "station_status_done is missing 'num_bikes_available'. "
            f"Available columns: {list(ss_done.columns)[:200]}"
        )

    if "num_ebikes_available" not in ss_done.columns:
        ss_done["num_ebikes_available"] = 0

    if "num_docks_available" not in ss_done.columns:
        alt = None
        for c in ["docks_available", "num_docks_free", "num_docks_open"]:
            if c in ss_done.columns:
                alt = c
                break
        if alt is None:
            raise ValueError(
                "station_status_done is missing 'num_docks_available' and no alternative dock column was found. "
                f"Available columns: {list(ss_done.columns)[:200]}"
            )
        ss_done["num_docks_available"] = pd.to_numeric(ss_done[alt], errors="coerce").fillna(0)

    ss_done["num_bikes_available"] = pd.to_numeric(ss_done["num_bikes_available"], errors="coerce").fillna(0).astype(int)
    ss_done["num_ebikes_available"] = pd.to_numeric(ss_done["num_ebikes_available"], errors="coerce").fillna(0).astype(int)
    ss_done["num_docks_available"] = pd.to_numeric(ss_done["num_docks_available"], errors="coerce").fillna(0).astype(int)
    ss_done["total_vehicle_available"] = ss_done["num_bikes_available"] + ss_done["num_ebikes_available"]

    results["station_status_done"] = ss_done
    if save_outputs:
        ss_done.to_csv(out_dir / f"{city_key.lower()}_station_status_done.csv", index=False)

    # ==================================================
    # AVAILABILITY
    # ==================================================
    ss_av = ss_done.copy()
    ss_av["timestamp"] = pd.to_datetime(ss_av["timestamp"], errors="coerce")
    if ss_av["timestamp"].isna().any():
        raise ValueError("Found unparseable timestamps in station_status_done.")

    data_min = ss_av["timestamp"].min()
    data_max = ss_av["timestamp"].max()

    if availability_time_start is not None:
        ss_av = ss_av[ss_av["timestamp"] >= availability_time_start].copy()
    if availability_time_end is not None:
        ss_av = ss_av[ss_av["timestamp"] < availability_time_end].copy()

    if ss_av.empty:
        raise ValueError(f"No availability rows after time filter. Data range: [{data_min}, {data_max}]")

    def _availability_for_level(level: str) -> Dict[str, pd.DataFrame]:
        geo_key = "census_block" if level == "block" else "census_tract"

        temp = ss_av.copy()
        temp["time_slot"] = temp["timestamp"].dt.floor(availability_time_granularity)

        geo_ts = (
            temp.groupby([geo_key, "timestamp"], as_index=False)
            .agg(
                num_bikes_available=("num_bikes_available", "sum"),
                num_ebikes_available=("num_ebikes_available", "sum"),
                num_docks_available=("num_docks_available", "sum"),
                total_vehicle_available=("total_vehicle_available", "sum"),
            )
        )
        geo_ts["time_slot"] = geo_ts["timestamp"].dt.floor(availability_time_granularity)

        raw_df = (
            geo_ts.groupby([geo_key, "time_slot"], as_index=False)
            .mean(numeric_only=True)
            .round(0)
            .sort_values([geo_key, "time_slot"])
            .reset_index(drop=True)
        )

        for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
            raw_df[c] = raw_df[c].astype(int)

        if level == "tract" and external_tract_prefix:
            raw_df = raw_df[~raw_df["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

        norm_df = raw_df.copy()
        for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
            norm_df[c + "_norm"] = _minmax_norm(norm_df[c]).round(5)

        if save_outputs:
            raw_df.to_csv(out_dir / f"availability__raw__{level}.csv", index=False)
            norm_df.to_csv(out_dir / f"availability__norm__{level}.csv", index=False)

        return {"raw": raw_df, "norm": norm_df}

    availability_out: Dict[str, Any] = {"meta": {
        "time_start": availability_time_start,
        "time_end": availability_time_end,
        "data_min": data_min,
        "data_max": data_max,
        "time_granularity": availability_time_granularity,
        "group_level": availability_group_level,
    }}

    if availability_group_level in {"block", "both"}:
        blk = _availability_for_level("block")
        availability_out["availability_block_raw"] = blk["raw"]
        availability_out["availability_block_norm"] = blk["norm"]

    if availability_group_level in {"tract", "both"}:
        tr = _availability_for_level("tract")
        availability_out["availability_tract_raw"] = tr["raw"]
        availability_out["availability_tract_norm"] = tr["norm"]

    if "availability_tract_norm" not in availability_out:
        tr = _availability_for_level("tract")
        availability_out["availability_tract_raw"] = tr["raw"]
        availability_out["availability_tract_norm"] = tr["norm"]

    results["availability"] = availability_out

    availability_df_for_capacity = availability_out["availability_tract_norm"].copy()
    availability_df_for_capacity["census_tract"] = availability_df_for_capacity["census_tract"].astype(str)
    availability_df_for_capacity["time_slot"] = pd.to_datetime(availability_df_for_capacity["time_slot"])

    # ==================================================
    # CAPACITY
    # ==================================================
    import geopandas as gpd

    tracts_gdf = gpd.read_file(str(tracts_path))
    if tracts_gdf.crs is None:
        raise ValueError(f"Tracts shapefile CRS is missing: {tracts_path}")
    if tract_id_col not in tracts_gdf.columns:
        raise ValueError(f"Tract shapefile missing '{tract_id_col}' column.")

    if cfg.get("has_boro_logic", False):
        tracts_gdf["county_fips"] = tracts_gdf["STATEFP"].astype(str) + tracts_gdf["COUNTYFP"].astype(str)
        county_to_boro = {
            "36005": "The Bronx",
            "36047": "Brooklyn",
            "36061": "Manhattan",
            "36081": "Queens",
            "36085": "Staten Island",
        }
        tracts_gdf["boro"] = tracts_gdf["county_fips"].map(county_to_boro)
        if drop_staten_island:
            tracts_gdf = tracts_gdf[tracts_gdf["boro"] != "Staten Island"].copy()

    centroids_wgs84 = gpd.GeoSeries(tracts_gdf.geometry.centroid, crs=tracts_gdf.crs).to_crs(epsg=4326)
    tract_centroids_df = pd.DataFrame({
        "census_tract": tracts_gdf[tract_id_col].astype(str).values,
        "centroid_lon": centroids_wgs84.x.values,
        "centroid_lat": centroids_wgs84.y.values,
    })
    if "boro" in tracts_gdf.columns:
        tract_centroids_df["boro"] = tracts_gdf["boro"].values

    results["tract_centroids_df"] = tract_centroids_df
    if save_outputs:
        tract_centroids_df.to_csv(out_dir / "centroid_tract_computed.csv", index=False)

    station_info = si.copy()
    station_info["census_block"] = station_info["census_block"].astype(str)
    station_info["census_tract"] = _tract_from_block(station_info["census_block"]).astype(str)

    capacity_block = (
        station_info.groupby("census_block", as_index=False)
        .agg(total_capacity=("capacity", "sum"), num_station=("station_id", "count"))
    )
    results["capacity_block"] = capacity_block
    if save_outputs:
        capacity_block.to_csv(out_dir / "capacity_block.csv", index=False)

    tract_capacity = (
        station_info.groupby("census_tract", as_index=False)
        .agg(total_capacity=("capacity", "sum"), num_station=("station_id", "count"))
    )

    capacity_tract = tract_centroids_df[["census_tract"]].drop_duplicates().merge(
        tract_capacity, on="census_tract", how="left"
    )
    capacity_tract[["total_capacity", "num_station"]] = capacity_tract[["total_capacity", "num_station"]].fillna(0)

    if external_tract_prefix:
        capacity_tract = capacity_tract[~capacity_tract["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    capacity_tract = capacity_tract.sort_values("census_tract").reset_index(drop=True)
    capacity_tract["total_capacity_norm"] = _minmax_norm(capacity_tract["total_capacity"]).round(5)
    capacity_tract["num_station_norm"] = _minmax_norm(capacity_tract["num_station"]).round(5)

    results["capacity_tract_norm"] = capacity_tract
    if save_outputs:
        capacity_tract.to_csv(out_dir / "capacity_tract_norm.csv", index=False)

    avail = availability_df_for_capacity.copy()
    if external_tract_prefix:
        avail = avail[~avail["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    if peak_time_slot is None:
        peak_series = avail.groupby("time_slot")[peak_metric].sum()
        if peak_series.empty:
            raise ValueError("No availability data after filters; cannot find peak time_slot.")
        peak_time_slot = peak_series.idxmax()
    peak_time_slot = pd.to_datetime(peak_time_slot)

    peak_df = avail[avail["time_slot"] == peak_time_slot].copy()

    vehicle_capacity = (
        peak_df.groupby("census_tract", as_index=False)["total_vehicle_available"]
        .sum()
        .rename(columns={"total_vehicle_available": "vehicle_capacity"})
    )
    dock_capacity = (
        peak_df.groupby("census_tract", as_index=False)["num_docks_available"]
        .sum()
        .rename(columns={"num_docks_available": "dock_capacity"})
    )

    capacity_df = (
        capacity_tract.merge(vehicle_capacity, on="census_tract", how="left")
        .merge(dock_capacity, on="census_tract", how="left")
    )
    capacity_df["vehicle_capacity"] = capacity_df["vehicle_capacity"].fillna(0)
    capacity_df["dock_capacity"] = capacity_df["dock_capacity"].fillna(0)

    capacity_df["vehicle_capacity_norm"] = _minmax_norm(capacity_df["vehicle_capacity"]).round(5)
    capacity_df["dock_capacity_norm"] = _minmax_norm(capacity_df["dock_capacity"]).round(5)

    capacity_df["occupancy_rate"] = capacity_df.apply(
        lambda r: (r["vehicle_capacity"] / r["total_capacity"]) if r["total_capacity"] else 0.0, axis=1
    )
    capacity_df["return_pressure"] = capacity_df.apply(
        lambda r: 1.0 - (r["dock_capacity"] / r["total_capacity"]) if r["total_capacity"] else 0.0, axis=1
    )

    results["capacity_df"] = capacity_df
    results["peak_time_slot"] = peak_time_slot
    results["peak_metric"] = peak_metric
    if save_outputs:
        capacity_df.to_csv(out_dir / "capacity_tract_with_vehicle_and_docks_norm.csv", index=False)

    # ==================================================
    # SAFETY (robust WKT; CRS-safe using blocks CRS)
    # ==================================================
    blocks_gdf = gpd.read_file(str(census_blocks_path))
    if blocks_gdf.crs is None:
        raise ValueError(f"Blocks shapefile CRS is missing: {census_blocks_path}")
    if blocks_id_col not in blocks_gdf.columns:
        raise ValueError(f"Blocks shapefile missing '{blocks_id_col}' column.")

    target_crs = blocks_gdf.crs
    safety_kind = str(cfg.get("safety_input_kind", "shp"))

    streets_gdf = _read_lines_layer(
        centerline_path,
        kind=safety_kind,
        target_crs=target_crs,
        csv_wkt_candidates=tuple(cfg.get("safety_csv_wkt_candidates", ("geometry", "the_geom", "wkt", "shape", "line"))),
        csv_crs=str(cfg.get("safety_csv_crs", "EPSG:4326")),
    )
    bike_gdf = _read_lines_layer(
        bike_lanes_path,
        kind=safety_kind,
        target_crs=target_crs,
        csv_wkt_candidates=tuple(cfg.get("safety_csv_wkt_candidates", ("geometry", "the_geom", "wkt", "shape", "line"))),
        csv_crs=str(cfg.get("safety_csv_crs", "EPSG:4326")),
    )

    street_len = _overlay_length_by_block(streets_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="streets_leng")
    bike_len = _overlay_length_by_block(bike_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="total_bike_lane_length")

    protected_rule = cfg.get("protected_lane_rule", {})
    facility_col, protected_gdf = _protected_filter(bike_gdf, protected_rule)

    if facility_col is None:
        prot_len = pd.DataFrame({"census_block": street_len["census_block"].astype(str).unique()})
        prot_len["protected_bike_lane_length"] = 0.0
        results["safety_note"] = (
            f"[{city_key}] No protected-lane column found. Protected length set to 0. "
            f"Tried: {protected_rule.get('col_candidates')}"
        )
    else:
        prot_len = _overlay_length_by_block(
            protected_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="protected_bike_lane_length"
        )
        results["safety_protected_lane_col_used"] = facility_col
        results["safety_protected_lane_rule"] = protected_rule

    safety_block = (
        street_len.merge(bike_len, on="census_block", how="left")
        .merge(prot_len, on="census_block", how="left")
    )
    safety_block["total_bike_lane_length"] = safety_block["total_bike_lane_length"].fillna(0)
    safety_block["protected_bike_lane_length"] = safety_block["protected_bike_lane_length"].fillna(0)
    safety_block["bike_lane_ratio"] = _safe_ratio(safety_block["total_bike_lane_length"], safety_block["streets_leng"]).round(3)
    safety_block["protected_bike_lane_ratio"] = _safe_ratio(safety_block["protected_bike_lane_length"], safety_block["streets_leng"]).round(3)
    safety_block["census_tract"] = _tract_from_block(safety_block["census_block"]).astype(str)

    # tract safety using centroid file (for region cols)
    centroids_csv_df = pd.read_csv(centroid_tract_csv)
    centroids_csv_df["census_tract"] = centroids_csv_df["census_tract"].astype(str)
    region_cols = ["census_tract"]
    for c in ["boro", "county_name", "county", "state"]:
        if c in centroids_csv_df.columns:
            region_cols.append(c)

    safety_tract = (
        safety_block.groupby("census_tract", as_index=False)[
            ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]
        ].sum()
    )
    safety_tract = centroids_csv_df[region_cols].merge(safety_tract, on="census_tract", how="left")
    for c in ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]:
        safety_tract[c] = safety_tract[c].fillna(0)

    safety_tract["bike_lane_ratio"] = _safe_ratio(safety_tract["total_bike_lane_length"], safety_tract["streets_leng"])
    safety_tract["protected_bike_lane_ratio"] = _safe_ratio(safety_tract["protected_bike_lane_length"], safety_tract["streets_leng"])
    safety_tract = safety_tract.sort_values("census_tract").reset_index(drop=True)

    service_tracts = set(capacity_df["census_tract"].astype(str).unique().tolist())
    safety_service = safety_tract[safety_tract["census_tract"].astype(str).isin(service_tracts)].reset_index(drop=True)

    safety_norm = safety_service.copy()
    safety_norm["bike_lane_ratio_norm"] = _minmax_norm(safety_norm["bike_lane_ratio"]).round(5)
    safety_norm["protected_bike_lane_ratio_norm"] = _minmax_norm(safety_norm["protected_bike_lane_ratio"]).round(5)

    results["safety_bike_lane_block"] = safety_block
    results["safety_bike_lane_tract"] = safety_tract
    results["safety_bike_lane_service_area"] = safety_service
    results["safety_bike_lane_norm_tract"] = safety_norm

    if save_outputs:
        safety_block.to_csv(out_dir / "safety_bike_lane_block.csv", index=False)
        safety_tract.to_csv(out_dir / "safety_bike_lane_tract.csv", index=False)
        safety_service.to_csv(out_dir / "safety_bike_lane_service_area.csv", index=False)
        safety_norm.to_csv(out_dir / "safety_bike_lane_norm_tract.csv", index=False)

    # ==================================================
    # TRIPS — load + standardize + compute start/end census blocks if missing
    # ==================================================
    trip_raw = pd.read_csv(trip_csv)
    trip_df = _standardize_trips(trip_raw)

    trip_df["started_at"] = pd.to_datetime(trip_df["started_at"], errors="coerce")
    trip_df["ended_at"] = pd.to_datetime(trip_df["ended_at"], errors="coerce")

    # Compute blocks from lat/lng if not present
    if ("start_census_block" not in trip_df.columns) or ("end_census_block" not in trip_df.columns):
        if not {"start_lat", "start_lng", "end_lat", "end_lng"}.issubset(trip_df.columns):
            raise ValueError(
                "Trip CSV does not have start/end census blocks AND does not have start/end lat/lng to compute them. "
                f"Found: {list(trip_df.columns)[:200]}"
            )

        start_map = _geocode_points_to_blocks_spatial(
            trip_df, lon_col="start_lng", lat_col="start_lat",
            blocks_path=census_blocks_path, blocks_id_col=blocks_id_col
        ).rename(columns={"census_block": "start_census_block"})

        end_map = _geocode_points_to_blocks_spatial(
            trip_df, lon_col="end_lng", lat_col="end_lat",
            blocks_path=census_blocks_path, blocks_id_col=blocks_id_col
        ).rename(columns={"census_block": "end_census_block"})

        trip_df = trip_df.merge(
            start_map, left_on=["start_lng", "start_lat"], right_on=["lon", "lat"], how="left"
        ).drop(columns=["lon", "lat"], errors="ignore")

        trip_df = trip_df.merge(
            end_map, left_on=["end_lng", "end_lat"], right_on=["lon", "lat"], how="left"
        ).drop(columns=["lon", "lat"], errors="ignore")

    required_trip_cols = {"started_at", "ended_at", "start_census_block", "end_census_block"}
    miss_trip = required_trip_cols - set(trip_df.columns)
    if miss_trip:
        raise ValueError(
            f"Trip CSV is missing required columns after standardization: {miss_trip}. "
            f"Available columns: {list(trip_df.columns)[:200]}"
        )

    trip_df["start_census_block"] = trip_df["start_census_block"].astype(str)
    trip_df["end_census_block"] = trip_df["end_census_block"].astype(str)

    # default usage/idle windows
    if usage_time_start is None:
        usage_time_start = availability_time_start
    if usage_time_end is None:
        usage_time_end = availability_time_end
    if idle_time_start is None:
        idle_time_start = availability_time_start
    if idle_time_end is None:
        idle_time_end = availability_time_end

    if usage_time_start is None:
        usage_time_start = trip_df["started_at"].min()
    if usage_time_end is None:
        usage_time_end = trip_df["started_at"].max()
    if idle_time_start is None:
        idle_time_start = trip_df["started_at"].min()
    if idle_time_end is None:
        idle_time_end = trip_df["started_at"].max()

    usage_time_start = pd.to_datetime(usage_time_start)
    usage_time_end = pd.to_datetime(usage_time_end)
    idle_time_start = pd.to_datetime(idle_time_start)
    idle_time_end = pd.to_datetime(idle_time_end)

    # ==================================================
    # USAGE (5-min, hourly, tract-hourly norm)
    # ==================================================
    trips = trip_df.copy()
    trips = trips[(trips["started_at"] >= usage_time_start) & (trips["started_at"] <= usage_time_end)].copy()
    trips["trip_duration"] = (trips["ended_at"] - trips["started_at"]).dt.total_seconds() / 60.0
    trips = trips[trips["trip_duration"] <= 240].copy()

    # 5-min
    trips["time_slot_start"] = trips["started_at"].dt.floor("5min")
    trips["time_slot_end"] = trips["ended_at"].dt.floor("5min")

    trips_starting_5 = trips.groupby(["start_census_block", "time_slot_start"]).size().reset_index(name="trips_starting")
    trips_ending_5 = trips.groupby(["end_census_block", "time_slot_end"]).size().reset_index(name="trips_ending")

    trips_starting_5 = trips_starting_5.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})
    trips_ending_5 = trips_ending_5.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})

    all_blocks = pd.unique(pd.concat([trips["start_census_block"], trips["end_census_block"]]).astype(str))
    all_blocks = [b for b in all_blocks if b not in ("nan", "0")]

    all_time_slots_5 = pd.date_range(start=usage_time_start.floor("5min"), end=usage_time_end.ceil("5min"), freq="5min")
    grid_5 = pd.MultiIndex.from_product([all_blocks, all_time_slots_5], names=["census_block", "time_slot"]).to_frame(index=False)

    usage_5min_block = (
        grid_5.merge(trips_starting_5, on=["census_block", "time_slot"], how="left")
             .merge(trips_ending_5, on=["census_block", "time_slot"], how="left")
    )
    usage_5min_block[["trips_starting", "trips_ending"]] = usage_5min_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
    usage_5min_block = usage_5min_block.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

    if save_outputs:
        usage_5min_block.to_csv(out_dir / "usage_5min_block.csv", index=False)

    # hourly block
    trips["hour_start"] = trips["started_at"].dt.floor("H")
    trips["hour_end"] = trips["ended_at"].dt.floor("H")

    trips_starting_h = trips.groupby(["start_census_block", "hour_start"]).size().reset_index(name="trips_starting")
    trips_ending_h = trips.groupby(["end_census_block", "hour_end"]).size().reset_index(name="trips_ending")

    trips_starting_h = trips_starting_h.rename(columns={"start_census_block": "census_block", "hour_start": "hour"})
    trips_ending_h = trips_ending_h.rename(columns={"end_census_block": "census_block", "hour_end": "hour"})

    all_hours = pd.date_range(start=usage_time_start.floor("H"), end=usage_time_end.ceil("H"), freq="H")
    grid_h = pd.MultiIndex.from_product([all_blocks, all_hours], names=["census_block", "hour"]).to_frame(index=False)

    usage_hourly_block = (
        grid_h.merge(trips_starting_h, on=["census_block", "hour"], how="left")
             .merge(trips_ending_h, on=["census_block", "hour"], how="left")
    )
    usage_hourly_block[["trips_starting", "trips_ending"]] = usage_hourly_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
    usage_hourly_block = usage_hourly_block.sort_values(["census_block", "hour"]).reset_index(drop=True)

    if save_outputs:
        usage_hourly_block.to_csv(out_dir / "usage_hourly_block.csv", index=False)

    # hourly tract
    usage_hourly = usage_hourly_block.copy()
    usage_hourly["census_block"] = usage_hourly["census_block"].astype(str)
    usage_hourly["census_tract"] = _tract_from_block(usage_hourly["census_block"]).astype(str)
    if external_tract_prefix:
        usage_hourly = usage_hourly[~usage_hourly["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    usage_hourly["time_slot"] = pd.to_datetime(usage_hourly["hour"])
    usage_hourly = usage_hourly.drop(columns=["hour"])

    usage_hourly_tract = (
        usage_hourly.groupby(["census_tract", "time_slot"], as_index=False)
        .agg(trips_starting=("trips_starting", "sum"), trips_ending=("trips_ending", "sum"))
        .reset_index(drop=True)
    )
    usage_hourly_tract_raw = usage_hourly_tract.copy()

    if tracts_to_remove:
        usage_hourly_tract = usage_hourly_tract[~usage_hourly_tract["census_tract"].isin(tracts_to_remove)].reset_index(drop=True)

    usage_hourly_tract["trips_starting_norm"] = _minmax_norm(usage_hourly_tract["trips_starting"]).round(5)
    usage_hourly_tract["trips_ending_norm"] = _minmax_norm(usage_hourly_tract["trips_ending"]).round(5)

    if save_outputs:
        usage_hourly_tract_raw.to_csv(out_dir / "usage_hourly_tract_raw.csv", index=False)
        usage_hourly_tract.to_csv(out_dir / "usage_norm_hourly_tract.csv", index=False)

    results["usage_5min_block"] = usage_5min_block
    results["usage_hourly_block"] = usage_hourly_block
    results["usage_hourly_tract_raw"] = usage_hourly_tract_raw
    results["usage_norm_hourly_tract"] = usage_hourly_tract

    # ==================================================
    # IDLE TIME (full simulation, same logic as your earlier notebook)
    # ==================================================
    if not compute_idle_time:
        results["idle_note"] = "Idle time skipped (compute_idle_time=False)."
        return results

    # Need rideable_type to split bike vs ebike; if missing -> still compute total moved (no split)
    has_rideable = "rideable_type" in trip_df.columns

    # GBFS input for idle (has vehicles)
    ss_idle = ss_done.copy()
    ss_idle["timestamp"] = pd.to_datetime(ss_idle["timestamp"])
    ss_idle["census_block"] = ss_idle["census_block"].astype(str)
    ss_idle["census_tract"] = ss_idle["census_tract"].astype(str)
    ss_idle["total_available"] = ss_idle["num_bikes_available"] + ss_idle["num_ebikes_available"]
    ss_idle["time_5min"] = ss_idle["timestamp"].dt.floor("5min")

    trips_idle = trip_df.copy()
    trips_idle["start_census_block"] = trips_idle["start_census_block"].astype(str)
    trips_idle["end_census_block"] = trips_idle["end_census_block"].astype(str)
    trips_idle["started_at"] = pd.to_datetime(trips_idle["started_at"])
    trips_idle["ended_at"] = pd.to_datetime(trips_idle["ended_at"])

    # drop external if requested (NYC drops NJ)
    if external_tract_prefix:
        start_tract = _tract_from_block(trips_idle["start_census_block"]).astype(str)
        end_tract = _tract_from_block(trips_idle["end_census_block"]).astype(str)
        trips_idle = trips_idle[
            (~start_tract.str.startswith(external_tract_prefix)) & (~end_tract.str.startswith(external_tract_prefix))
        ].copy()

    # time filter
    trips_idle = trips_idle[(trips_idle["started_at"] >= idle_time_start) & (trips_idle["started_at"] <= idle_time_end)].copy()

    # 5-min buckets
    trips_idle["time_slot_start"] = trips_idle["started_at"].dt.floor("5min")
    trips_idle["time_slot_end"] = trips_idle["ended_at"].dt.floor("5min")

    # If rideable_type exists, compute starting/ending by type (bike vs ebike)
    if has_rideable:
        trips_idle = _map_rideable_type_to_two_columns(trips_idle)

        # starting: bike/ebike counts
        ts_start = (
            trips_idle.groupby(["start_census_block", "time_slot_start", "_is_ebike"])
            .size()
            .unstack(fill_value=0)
            .reset_index()
        )
        ts_start = ts_start.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})
        if True not in ts_start.columns:
            ts_start[True] = 0
        if False not in ts_start.columns:
            ts_start[False] = 0
        ts_start = ts_start.rename(columns={False: "trips_starting_bike", True: "trips_starting_ebike"})

        # ending: bike/ebike counts
        ts_end = (
            trips_idle.groupby(["end_census_block", "time_slot_end", "_is_ebike"])
            .size()
            .unstack(fill_value=0)
            .reset_index()
        )
        ts_end = ts_end.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})
        if True not in ts_end.columns:
            ts_end[True] = 0
        if False not in ts_end.columns:
            ts_end[False] = 0
        ts_end = ts_end.rename(columns={False: "trips_ending_bike", True: "trips_ending_ebike"})

    else:
        # No rideable_type: only total starting/ending
        ts_start = trips_idle.groupby(["start_census_block", "time_slot_start"]).size().reset_index(name="trips_starting")
        ts_end = trips_idle.groupby(["end_census_block", "time_slot_end"]).size().reset_index(name="trips_ending")
        ts_start = ts_start.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})
        ts_end = ts_end.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})

    # full grid
    all_blocks_idle = pd.unique(pd.concat([trips_idle["start_census_block"], trips_idle["end_census_block"]]))
    all_blocks_idle = pd.Series(all_blocks_idle).dropna().astype(str).unique()

    all_time_slots_idle = pd.date_range(
        start=idle_time_start.floor("5min"),
        end=idle_time_end.ceil("5min"),
        freq="5min",
    )
    grid_idle = pd.MultiIndex.from_product(
        [all_blocks_idle, all_time_slots_idle],
        names=["census_block", "time_slot"],
    ).to_frame(index=False)

    if has_rideable:
        result_complete = grid_idle.merge(ts_start, on=["census_block", "time_slot"], how="left").merge(
            ts_end, on=["census_block", "time_slot"], how="left"
        )
        cols = ["trips_starting_bike", "trips_starting_ebike", "trips_ending_bike", "trips_ending_ebike"]
        for c in cols:
            if c not in result_complete.columns:
                result_complete[c] = 0
            result_complete[c] = result_complete[c].fillna(0).astype(int)

        result_complete["trips_starting"] = result_complete["trips_starting_bike"] + result_complete["trips_starting_ebike"]
        result_complete["trips_ending"] = result_complete["trips_ending_bike"] + result_complete["trips_ending_ebike"]

    else:
        result_complete = grid_idle.merge(ts_start, on=["census_block", "time_slot"], how="left").merge(
            ts_end, on=["census_block", "time_slot"], how="left"
        )
        for c in ["trips_starting", "trips_ending"]:
            if c not in result_complete.columns:
                result_complete[c] = 0
            result_complete[c] = result_complete[c].fillna(0).astype(int)

    result_complete = result_complete.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

    # station availability by block 5-min
    station_availability_df = (
        ss_idle.groupby(["census_block", "time_5min"], as_index=False)[
            ["num_bikes_available", "num_ebikes_available", "total_available"]
        ].sum()
    )
    station_availability_df = station_availability_df.rename(columns={"time_5min": "time_slot"})
    station_availability_df["time_slot"] = station_availability_df["time_slot"] - pd.Timedelta(minutes=5)

    merged_idle = station_availability_df.merge(result_complete, on=["census_block", "time_slot"], how="left")
    if has_rideable:
        cols = ["trips_starting_bike", "trips_starting_ebike", "trips_ending_bike", "trips_ending_ebike", "trips_starting", "trips_ending"]
        for c in cols:
            if c not in merged_idle.columns:
                merged_idle[c] = 0
            merged_idle[c] = merged_idle[c].fillna(0).astype(int)

        merged_idle["vehicles_moved_bike"] = merged_idle["trips_starting_bike"] + merged_idle["trips_ending_bike"]
        merged_idle["vehicles_moved_ebike"] = merged_idle["trips_starting_ebike"] + merged_idle["trips_ending_ebike"]
        merged_idle["vehicles_moved"] = merged_idle["trips_starting"] + merged_idle["trips_ending"]
    else:
        for c in ["trips_starting", "trips_ending"]:
            if c not in merged_idle.columns:
                merged_idle[c] = 0
            merged_idle[c] = merged_idle[c].fillna(0).astype(int)
        merged_idle["vehicles_moved"] = merged_idle["trips_starting"] + merged_idle["trips_ending"]

    merged_idle["idle_vehicles"] = (merged_idle["total_available"] - merged_idle["vehicles_moved"]).clip(lower=0)
    merged_idle["time_slot"] = pd.to_datetime(merged_idle["time_slot"])
    merged_idle = merged_idle.sort_values(["census_block", "time_slot"])
    merged_idle["hour"] = merged_idle["time_slot"].dt.floor("H")

    # hourly idle simulation per block
    from collections import deque

    idle_results = []
    for block, block_df in merged_idle.groupby("census_block"):
        for hour, hour_df in block_df.groupby("hour"):
            hour_df = hour_df.sort_values("time_slot")
            virtual_vehicles = deque()
            finalized_durations = []

            for _, row in hour_df.iterrows():
                current_time = row["time_slot"]
                current_idle = int(row["idle_vehicles"])

                if current_idle > len(virtual_vehicles):
                    for _ in range(current_idle - len(virtual_vehicles)):
                        virtual_vehicles.append(current_time)
                elif current_idle < len(virtual_vehicles):
                    for _ in range(len(virtual_vehicles) - current_idle):
                        start_t = virtual_vehicles.popleft()
                        duration = (current_time - start_t).total_seconds() / 60.0
                        finalized_durations.append(duration)

            final_time = hour + pd.Timedelta(hours=1)
            while virtual_vehicles:
                start_t = virtual_vehicles.popleft()
                duration = (final_time - start_t).total_seconds() / 60.0
                finalized_durations.append(duration)

            avg_idle_time = round(sum(finalized_durations) / len(finalized_durations), 2) if finalized_durations else 0.0
            idle_results.append({"census_block": block, "hour": hour, "avg_idle_time": avg_idle_time})

    idle_time_block = pd.DataFrame(idle_results)

    # tract aggregation + normalize
    idle_time_block["census_block"] = idle_time_block["census_block"].astype(str)
    idle_time_block["census_tract"] = _tract_from_block(idle_time_block["census_block"]).astype(str)

    idle_time_tract = idle_time_block.groupby(["census_tract", "hour"], as_index=False)["avg_idle_time"].mean()

    if external_tract_prefix:
        idle_time_tract = idle_time_tract[
            ~idle_time_tract["census_tract"].astype(str).str.startswith(external_tract_prefix)
        ].reset_index(drop=True)

    idle_time_tract["avg_idle_time_norm"] = _minmax_norm(idle_time_tract["avg_idle_time"]).round(5)
    idle_time_tract = idle_time_tract.rename(columns={"hour": "time_slot"})

    results["idle_merged_5min"] = merged_idle
    results["idle_time_block"] = idle_time_block
    results["idle_time_tract_raw"] = idle_time_tract.drop(columns=["avg_idle_time_norm"])
    results["idle_time_norm_tract"] = idle_time_tract

    if save_outputs:
        merged_idle.to_csv(out_dir / "idle_merged_5min.csv", index=False)
        idle_time_block.to_csv(out_dir / "idle_time_block.csv", index=False)
        results["idle_time_tract_raw"].to_csv(out_dir / "idle_time_tract_raw.csv", index=False)
        idle_time_tract.to_csv(out_dir / "idle_time_norm_tract.csv", index=False)

    return results


In [ ]:
out = run_docked_all_utilities_single_function(
    city="SF", # city_key="NJ" / "NYC" / "PITT" / "SF"
    station_status_txt=r"D:\Research Fellowship\Summer Research Stuff\Collected Data\Week 1\09-June\san_fran_baywheels_docked_station_status_6_9.txt",
    station_information_csv=r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\San_Fran_Baywheels\San_fran_Baywheels station information 06_09.csv",
    trip_csv=r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Usage\San_Fran_Baywheels\202505-baywheels-tripdata.csv",
    output_dir="San_Fran_Baywheels_outputs",
)

C:\Users\Vedant\AppData\Local\Temp\ipykernel_10600\2635220460.py:720: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  temp["time_slot"] = temp["timestamp"].dt.floor(availability_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_10600\2635220460.py:731: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  geo_ts["time_slot"] = geo_ts["timestamp"].dt.floor(availability_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_10600\2635220460.py:720: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  temp["time_slot"] = temp["timestamp"].dt.floor(availability_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_10600\2635220460.py:731: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  geo_ts["time_slot"] = geo_ts["timestamp"].dt.floor(availability_time_granularity)


In [1]:
from __future__ import annotations

from pathlib import Path
from typing import Optional, Union, Dict, Any, Sequence, Tuple
import json
import pandas as pd


# ==================================================
# CITY CONFIG (hardcoded assets for NYC + NJ + PITT + SF)
# ==================================================
CITY_CONFIG: Dict[str, Dict[str, Any]] = {
    "NYC": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NYC\tl_2024_36_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NYC\tl_2024_36_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Capacity_NYC\centroid_tract_computed.csv"
        ),
        # safety (NYC = CSV WKT)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\CSCL_PlowNYC_20250619.csv"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\New_York_City_Bike_Routes_20250619.csv"
        ),
        "safety_input_kind": "csv_wkt",
        "safety_csv_wkt_candidates": ("geometry", "the_geom", "wkt", "geometry_wkt", "WKT", "geom", "shape", "line"),
        "safety_csv_crs": "EPSG:4326",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": "34",  # drop NJ tracts for NYC results
        "has_boro_logic": True,
        "drop_staten_island_default": True,
        "protected_lane_rule": {
            "col_candidates": ("facilitycl", "facility", "class", "ft", "type"),
            "match_type": "equals",
            "match_value": "I",
        },
    },
    "NJ": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\tl_2024_34_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\tl_2024_34_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv"
        ),
        # safety (NJ = SHP)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\Tran_road.shp"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp"
        ),
        "safety_input_kind": "shp",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        "protected_lane_rule": {
            "col_candidates": ("type", "facility", "facilitycl", "class", "lane_type"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
    "PITT": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\tl_2024_42_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\tl_2024_42_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\centroid_tract_pa.csv"
        ),
        # safety (PITT = SHP)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Pittsburgh_Street_Centerline.shp"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Bike Map\Bike Lanes.shp"
        ),
        "safety_input_kind": "shp",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        "protected_lane_rule": {
            "col_candidates": ("facility", "type", "class", "status", "lane_type", "BIKE_FACIL", "CATEGORY"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
    "SF": {
        "census_blocks_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Usage\San_Fran_Baywheels\tl_2024_06_tabblock20.shp"
        ),
        "tracts_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\San_Fran_Baywheels\tl_2024_06_tract.shp"
        ),
        "centroid_tract_csv": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\San_Fran_Baywheels\centroid_tract_ca.csv"
        ),
        # safety (SF = CSV WKT)
        "centerline_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Streets___Active_and_Retired_20250626 (1).csv"
        ),
        "bike_lanes_path": Path(
            r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Bikelane.csv"
        ),
        "safety_input_kind": "csv_wkt",
        "safety_csv_wkt_candidates": ("geometry", "shape", "line", "the_geom", "wkt", "geometry_wkt", "WKT", "geom"),
        "safety_csv_crs": "EPSG:4326",
        "blocks_id_col": "GEOID20",
        "tract_id_col": "GEOID",
        "external_tract_prefix": None,
        "has_boro_logic": False,
        "drop_staten_island_default": False,
        # protected heuristic: non-empty BARRIER OR keywords in FACILITY_T
        "protected_lane_rule": {
            "col_candidates": ("BARRIER", "FACILITY_T", "BUFFERED", "RAISED", "SYMBOLOGY"),
            "match_type": "not_empty_or_contains",
            "match_value": r"PROTECT|SEPARAT|CYCLETRACK|BARRIER|RAISED|BUFFER",
        },
    },
}


# ==================================================
# Helpers
# ==================================================
def _ensure_exists(p: Path, label: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"{label} not found: {p}")


def _minmax_norm(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    mn = s.min(skipna=True)
    mx = s.max(skipna=True)
    if pd.isna(mn) or pd.isna(mx) or mx <= mn:
        return pd.Series(0.0, index=s.index)
    return (s - mn) / (mx - mn)


def _safe_ratio(numer: pd.Series, denom: pd.Series) -> pd.Series:
    return numer.div(denom.where(denom > 0, other=pd.NA)).fillna(0)


def _pick_col(df: pd.DataFrame, candidates: Sequence[str], required: bool, label: str) -> Optional[str]:
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    if required:
        raise ValueError(
            f"Could not find required column for {label}. Tried {list(candidates)}. "
            f"Found columns (sample): {list(df.columns)[:200]}"
        )
    return None


def _tract_from_block(block_series: pd.Series) -> pd.Series:
    s = block_series.astype(str)
    # If it looks like a 15-digit block GEOID -> tract is first 11 OR strip last 4
    return s.where(s.str.len() < 15, s.str[:-4])


def _standardize_station_info(df: pd.DataFrame) -> pd.DataFrame:
    sid = _pick_col(df, ["station_id", "stationId", "id", "Station ID", "Station Id"], True, "station_id")
    lat = _pick_col(df, ["lat", "latitude", "station_lat", "y"], True, "lat")
    lon = _pick_col(df, ["lon", "lng", "longitude", "station_lon", "x"], True, "lon")
    cap = _pick_col(df, ["capacity", "dock_count", "num_docks", "total_docks"], True, "capacity")

    out = df.copy().rename(columns={sid: "station_id", lat: "lat", lon: "lon", cap: "capacity"})
    out["station_id"] = out["station_id"].astype(str).str.strip()

    cb = _pick_col(out, ["census_block", "block_geoid", "GEOID20"], False, "census_block")
    if cb and cb != "census_block":
        out = out.rename(columns={cb: "census_block"})

    ct = _pick_col(out, ["census_tract", "tract_geoid", "GEOID"], False, "census_tract")
    if ct and ct != "census_tract":
        out = out.rename(columns={ct: "census_tract"})

    return out


def _standardize_trips(df: pd.DataFrame) -> pd.DataFrame:
    started = _pick_col(
        df,
        ["started_at", "start_time", "trip_start_time", "starttime", "Start Date", "start_date", "StartDate"],
        True,
        "started_at",
    )
    ended = _pick_col(
        df,
        ["ended_at", "end_time", "trip_end_time", "stoptime", "End Date", "end_date", "EndDate"],
        True,
        "ended_at",
    )
    out = df.copy().rename(columns={started: "started_at", ended: "ended_at"})

    rt = _pick_col(out, ["rideable_type", "bike_type", "vehicle_type", "Rider Type", "rider_type"], False, "rideable_type")
    if rt and rt != "rideable_type":
        out = out.rename(columns={rt: "rideable_type"})

    # precomputed blocks if present
    sb = _pick_col(out, ["start_census_block", "start_block", "start_block_geoid"], False, "start_census_block")
    eb = _pick_col(out, ["end_census_block", "end_block", "end_block_geoid"], False, "end_census_block")
    if sb and sb != "start_census_block":
        out = out.rename(columns={sb: "start_census_block"})
    if eb and eb != "end_census_block":
        out = out.rename(columns={eb: "end_census_block"})

    # lat/lon fallbacks
    slat = _pick_col(out, ["start_lat", "startLatitude", "StartLat"], False, "start_lat")
    slng = _pick_col(out, ["start_lng", "start_lon", "startLongitude", "StartLng"], False, "start_lng")
    elat = _pick_col(out, ["end_lat", "endLatitude", "EndLat"], False, "end_lat")
    elng = _pick_col(out, ["end_lng", "end_lon", "endLongitude", "EndLng"], False, "end_lng")
    if slat and slat != "start_lat":
        out = out.rename(columns={slat: "start_lat"})
    if slng and slng != "start_lng":
        out = out.rename(columns={slng: "start_lng"})
    if elat and elat != "end_lat":
        out = out.rename(columns={elat: "end_lat"})
    if elng and elng != "end_lng":
        out = out.rename(columns={elng: "end_lng"})

    # station ids (optional)
    ssid = _pick_col(out, ["start_station_id", "Start Station Id", "Start Station ID"], False, "start_station_id")
    esid = _pick_col(out, ["end_station_id", "End Station Id", "End Station ID"], False, "end_station_id")
    if ssid and ssid != "start_station_id":
        out = out.rename(columns={ssid: "start_station_id"})
    if esid and esid != "end_station_id":
        out = out.rename(columns={esid: "end_station_id"})

    return out


def _read_lines_layer(
    path: Path,
    *,
    kind: str,
    target_crs,
    csv_wkt_candidates: Tuple[str, ...],
    csv_crs: str,
):
    """
    Robust for SF/NYC CSV-WKT:
      - auto-picks the WKT column by sampling values
      - safe parses WKT (drops non-string/NaN)
    """
    import geopandas as gpd
    from shapely import wkt

    if kind == "shp":
        gdf = gpd.read_file(str(path))
        if gdf.crs is None:
            raise ValueError(f"CRS missing for {path}. Define CRS before projecting.")
        return gdf.to_crs(target_crs)

    df = pd.read_csv(path)

    def _looks_like_wkt(v: Any) -> bool:
        if not isinstance(v, str):
            return False
        s = v.strip().upper()
        return s.startswith(("LINESTRING", "MULTILINESTRING", "GEOMETRYCOLLECTION"))

    wkt_col = None
    for c in csv_wkt_candidates:
        if c in df.columns:
            sample = df[c].dropna().astype(str).head(80)
            if not sample.empty and sample.map(_looks_like_wkt).mean() >= 0.2:
                wkt_col = c
                break

    if wkt_col is None:
        raise ValueError(
            f"Could not find a usable WKT column in {path.name}. "
            f"Tried candidates: {list(csv_wkt_candidates)}. "
            f"Found columns: {list(df.columns)[:120]}"
        )

    def _safe_wkt_load(x: Any):
        if not isinstance(x, str):
            return None
        x = x.strip()
        if not x:
            return None
        try:
            return wkt.loads(x)
        except Exception:
            return None

    df = df.copy()
    df["geometry"] = df[wkt_col].apply(_safe_wkt_load)
    df = df[df["geometry"].notna()].copy()
    if df.empty:
        raise ValueError(
            f"After parsing WKT, no valid geometries remained for {path.name}. "
            f"Chosen WKT column: '{wkt_col}'."
        )

    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=csv_crs)
    return gdf.to_crs(target_crs)


def _overlay_length_by_block(lines_gdf, blocks_gdf, *, blocks_id_col: str, out_col: str) -> pd.DataFrame:
    import geopandas as gpd

    blocks = blocks_gdf[[blocks_id_col, "geometry"]].copy()
    lines = lines_gdf[["geometry"]].copy()

    inter = gpd.overlay(lines, blocks, how="intersection", keep_geom_type=False)
    if inter.empty:
        return pd.DataFrame({"census_block": pd.Series([], dtype=str), out_col: pd.Series([], dtype=float)})

    inter["seg_len"] = inter.geometry.length
    inter = inter.rename(columns={blocks_id_col: "census_block"})
    out = inter.groupby("census_block", as_index=False)["seg_len"].sum().rename(columns={"seg_len": out_col})
    out[out_col] = out[out_col].round(3)
    out["census_block"] = out["census_block"].astype(str)
    return out


def _protected_filter(lines_gdf, rule: Dict[str, Any]):
    """
    Supported match_type:
      - equals
      - contains
      - not_empty
      - not_empty_or_contains
    """
    col = _pick_col(lines_gdf, list(rule.get("col_candidates", [])), required=False, label="protected lane col")
    if col is None:
        return None, lines_gdf

    s = lines_gdf[col].astype(str)
    match_type = str(rule.get("match_type", "equals")).lower()
    match_value = str(rule.get("match_value", ""))

    if match_type == "equals":
        mask = s.str.upper() == match_value.upper()
    elif match_type == "contains":
        mask = s.str.upper().str.contains(match_value.upper(), na=False)
    elif match_type == "not_empty":
        s2 = lines_gdf[col]
        mask = s2.notna() & (s.astype(str).str.strip() != "") & (~s.astype(str).str.lower().isin(["0", "false", "none", "nan"]))
    elif match_type == "not_empty_or_contains":
        s2 = lines_gdf[col]
        non_empty = s2.notna() & (s.astype(str).str.strip() != "") & (~s.astype(str).str.lower().isin(["0", "false", "none", "nan"]))
        contains = s.astype(str).str.contains(match_value, case=False, na=False, regex=True)
        mask = non_empty | contains
    else:
        raise ValueError(f"Unsupported match_type='{match_type}' in protected_lane_rule")

    return col, lines_gdf[mask].copy()


def _geocode_points_to_blocks_spatial(
    df: pd.DataFrame,
    *,
    lon_col: str,
    lat_col: str,
    blocks_path: Path,
    blocks_id_col: str,
) -> pd.DataFrame:
    import geopandas as gpd
    from shapely.geometry import Point

    blocks = gpd.read_file(str(blocks_path)).to_crs(epsg=4326)
    if blocks_id_col not in blocks.columns:
        raise ValueError(f"Block shapefile missing '{blocks_id_col}' for spatial geocoding.")

    pts = df[[lon_col, lat_col]].dropna().drop_duplicates().copy()
    pts = pts.rename(columns={lon_col: "lon", lat_col: "lat"})

    gdf = gpd.GeoDataFrame(
        pts,
        geometry=[Point(xy) for xy in zip(pts["lon"], pts["lat"])],
        crs="EPSG:4326",
    )
    joined = gpd.sjoin(gdf, blocks[[blocks_id_col, "geometry"]], how="left", predicate="within")
    joined = joined.rename(columns={blocks_id_col: "census_block"})
    out = joined[["lon", "lat", "census_block"]].copy()
    out["census_block"] = out["census_block"].astype(str)
    return out


def _census_api_block(lat: float, lon: float, benchmark: str, vintage: str) -> Optional[str]:
    import requests
    try:
        url = (
            "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
            f"?x={lon}&y={lat}"
            f"&benchmark={benchmark}"
            f"&vintage={vintage}"
            "&format=json"
        )
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        js = r.json()
        blocks = js["result"]["geographies"].get("2020 Census Blocks", [])
        if not blocks:
            return None
        return blocks[0].get("GEOID")
    except Exception:
        return None


def _map_rideable_type_to_two_columns(trips: pd.DataFrame) -> pd.DataFrame:
    """
    Create columns:
      - trips_starting_bike, trips_starting_ebike
      - trips_ending_bike, trips_ending_ebike
    using rideable_type values like classic_bike/electric_bike (or similar strings).
    """
    df = trips.copy()
    if "rideable_type" not in df.columns:
        return df

    t = df["rideable_type"].astype(str).str.lower()
    df["_is_ebike"] = t.str.contains("electric|ebike|e-bike|assist", na=False)

    return df


# ==================================================
# MAIN FUNCTION
# ==================================================
def run_docked_all_utilities_single_function(
    *,
    city: str,
    station_status_txt: Union[str, Path],
    station_information_csv: Union[str, Path],
    trip_csv: Union[str, Path],
    output_dir: Optional[Union[str, Path]] = None,
    save_outputs: bool = True,
    
    # ------------------------------------------------------------
    # TIME WINDOW (unified)
    # ------------------------------------------------------------
    time_start: Optional[Union[str, pd.Timestamp]] = None,
    time_end: Optional[Union[str, pd.Timestamp]] = None,
    
    compute_idle_time: bool = True,
    
    # Specific overrides (will inherit from time_start/time_end if None)
    availability_time_start: Optional[Union[str, pd.Timestamp]] = None,
    availability_time_end: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_start: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_end: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_start: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_end: Optional[Union[str, pd.Timestamp]] = None,
    
    availability_time_granularity: str = "1H",
    availability_group_level: str = "both",
    peak_time_slot: Optional[Union[str, pd.Timestamp]] = None,
    peak_metric: str = "total_vehicle_available",
    drop_staten_island: Optional[bool] = None,
    tracts_to_remove: Optional[Sequence[str]] = None,
    remove_tz_suffix: str = " EDT",
    drop_cols: tuple[str, ...] = (
        "num_scooters_available",
        "num_scooters_unavailable",
        "is_installed",
        "is_returning",
        "is_renting",
    ),
    fill_missing_with_census_api: bool = True,
    census_geocoder_benchmark: str = "Public_AR_Census2020",
    census_geocoder_vintage: str = "Census2020_Current",
) -> Dict[str, Any]:
    """
    One-function multi-city docked pipeline (NYC/NJ/PITT/SF), including idle simulation.
    Accepts global time_start/time_end which propagates to all utilities.
    """

    # Normalize city keys
    city_key = city.strip().upper()
    if city_key in {"PITTSBURGH", "PIT"}:
        city_key = "PITT"
    if city_key in {"SANFRAN", "SAN_FRAN", "SAN FRANCISCO", "BAYWHEELS", "BAY WHEELS"}:
        city_key = "SF"

    if city_key not in CITY_CONFIG:
        raise ValueError(f"Unsupported city='{city}'. Supported: {list(CITY_CONFIG.keys())}")

    cfg = CITY_CONFIG[city_key]

    station_status_txt = Path(station_status_txt)
    station_information_csv = Path(station_information_csv)
    trip_csv = Path(trip_csv)

    census_blocks_path = Path(cfg["census_blocks_path"])
    tracts_path = Path(cfg["tracts_path"])
    centroid_tract_csv = Path(cfg["centroid_tract_csv"])
    centerline_path = Path(cfg["centerline_path"])
    bike_lanes_path = Path(cfg["bike_lanes_path"])

    blocks_id_col = str(cfg["blocks_id_col"])
    tract_id_col = str(cfg["tract_id_col"])
    external_tract_prefix = cfg.get("external_tract_prefix", None)

    if drop_staten_island is None:
        drop_staten_island = bool(cfg.get("drop_staten_island_default", False))

    out_dir = Path(output_dir) if output_dir is not None else Path(f"./{city_key}_ALL_IN_ONE_OUTPUTS")
    out_dir.mkdir(parents=True, exist_ok=True)

    # existence checks
    _ensure_exists(station_status_txt, "station_status_txt")
    _ensure_exists(station_information_csv, "station_information_csv")
    _ensure_exists(trip_csv, "trip_csv")
    _ensure_exists(census_blocks_path, "census_blocks_path")
    _ensure_exists(tracts_path, "tracts_path")
    _ensure_exists(centroid_tract_csv, "centroid_tract_csv")
    _ensure_exists(centerline_path, "centerline_path")
    _ensure_exists(bike_lanes_path, "bike_lanes_path")

    # -------------------------------------------------------------------------
    # TIME WINDOW LOGIC
    # -------------------------------------------------------------------------
    # 1. Normalize global inputs
    ts_start = pd.to_datetime(time_start) if time_start is not None else None
    ts_end = pd.to_datetime(time_end) if time_end is not None else None

    # 2. Propagate to specific params if they are None
    if availability_time_start is None:
        availability_time_start = ts_start
    if availability_time_end is None:
        availability_time_end = ts_end
        
    if usage_time_start is None:
        usage_time_start = ts_start
    if usage_time_end is None:
        usage_time_end = ts_end
        
    if idle_time_start is None:
        idle_time_start = ts_start
    if idle_time_end is None:
        idle_time_end = ts_end

    # 3. Ensure they are datetime objects
    availability_time_start = pd.to_datetime(availability_time_start) if availability_time_start else None
    availability_time_end = pd.to_datetime(availability_time_end) if availability_time_end else None
    usage_time_start = pd.to_datetime(usage_time_start) if usage_time_start else None
    usage_time_end = pd.to_datetime(usage_time_end) if usage_time_end else None
    idle_time_start = pd.to_datetime(idle_time_start) if idle_time_start else None
    idle_time_end = pd.to_datetime(idle_time_end) if idle_time_end else None

    availability_group_level = availability_group_level.lower().strip()
    if availability_group_level not in {"block", "tract", "both"}:
        raise ValueError("availability_group_level must be one of: 'block', 'tract', 'both'")

    if peak_metric not in {"total_vehicle_available", "num_docks_available"}:
        raise ValueError("peak_metric must be 'total_vehicle_available' or 'num_docks_available'")

    results: Dict[str, Any] = {
        "city": city_key,
        "system_type": "docked",
        "output_dir": str(out_dir),
        "time_window": {
            "global_start": str(ts_start),
            "global_end": str(ts_end)
        },
        "city_assets": {
            "census_blocks_path": str(census_blocks_path),
            "tracts_path": str(tracts_path),
            "centroid_tract_csv": str(centroid_tract_csv),
            "centerline_path": str(centerline_path),
            "bike_lanes_path": str(bike_lanes_path),
        },
    }

    # ==================================================
    # STEP 0 — station_status JSONL flatten
    # ==================================================
    with station_status_txt.open("r", encoding="utf-8") as f:
        lines = f.readlines()

    di_list = []
    for item in lines:
        if not item.strip():
            continue
        js = json.loads(item)
        timestamp = list(js.keys())[0]
        for entry in js[timestamp]:
            for vehicle_type in entry.get("vehicle_types_available", []):
                entry[f"vehicle_type_{vehicle_type['vehicle_type_id']}_count"] = vehicle_type["count"]
            if "vehicle_types_available" in entry:
                del entry["vehicle_types_available"]
            entry["timestamp"] = timestamp
            di_list.append(entry)

    station_status_flat = pd.DataFrame(di_list)
    results["station_status_flat"] = station_status_flat
    if save_outputs:
        station_status_flat.to_csv(out_dir / f"{city_key.lower()}_station_status_flat.csv", index=False)

    ss_df = station_status_flat.copy()
    ss_df["timestamp"] = ss_df["timestamp"].astype(str).str.replace(remove_tz_suffix, "", regex=False)
    ss_df["timestamp"] = pd.to_datetime(ss_df["timestamp"], errors="coerce")
    ss_df["hour"] = ss_df["timestamp"].dt.hour
    ss_df["date"] = ss_df["timestamp"].dt.date
    cols_to_drop = [c for c in drop_cols if c in ss_df.columns]
    if cols_to_drop:
        ss_df = ss_df.drop(cols_to_drop, axis=1)

    if "station_id" not in ss_df.columns:
        raise ValueError("station_status data missing station_id after flattening.")
    ss_df["station_id"] = ss_df["station_id"].astype(str).str.strip()

    results["station_status_clean"] = ss_df
    if save_outputs:
        ss_df.to_csv(out_dir / f"{city_key.lower()}_station_status_clean.csv", index=False)

    # ==================================================
    # STEP 0 — station_information -> standardized + census block/tract
    # ==================================================
    si_raw = pd.read_csv(station_information_csv)
    results["station_info_raw"] = si_raw
    if save_outputs:
        si_raw.to_csv(out_dir / f"{city_key.lower()}_station_information_raw.csv", index=False)

    si = _standardize_station_info(si_raw)

    # If census_block missing, compute via spatial join
    if ("census_block" not in si.columns) or si["census_block"].isna().any():
        import geopandas as gpd
        from shapely.geometry import Point

        pts = si.drop_duplicates(subset=["lat", "lon"])[["lon", "lat"]].copy()
        pts_gdf = gpd.GeoDataFrame(
            pts,
            geometry=[Point(xy) for xy in zip(pts["lon"], pts["lat"])],
            crs="EPSG:4326",
        )

        blocks_wgs84 = gpd.read_file(str(census_blocks_path)).to_crs(epsg=4326)
        if blocks_id_col not in blocks_wgs84.columns:
            raise ValueError(f"Block shapefile missing '{blocks_id_col}' for station geocoding.")

        joined = gpd.sjoin(pts_gdf, blocks_wgs84[[blocks_id_col, "geometry"]], how="left", predicate="within")
        joined = joined.rename(columns={blocks_id_col: "census_block"})
        joined["census_block"] = joined["census_block"].astype(str)
        joined["census_tract"] = _tract_from_block(joined["census_block"]).astype(str)

        si = si.merge(joined[["lon", "lat", "census_block", "census_tract"]], on=["lon", "lat"], how="left", suffixes=("", "_sj"))

        if "census_block_sj" in si.columns:
            si["census_block"] = si.get("census_block").fillna(si["census_block_sj"]) if "census_block" in si.columns else si["census_block_sj"]
            si = si.drop(columns=["census_block_sj"], errors="ignore")

        if "census_tract_sj" in si.columns:
            if "census_tract" not in si.columns:
                si["census_tract"] = pd.NA
            si["census_tract"] = si["census_tract"].fillna(si["census_tract_sj"])
            si = si.drop(columns=["census_tract_sj"], errors="ignore")

        # Census API fill for remaining missing (optional)
        if fill_missing_with_census_api and ("census_block" in si.columns) and si["census_block"].isna().any():
            miss = si[si["census_block"].isna()][["lat", "lon"]].drop_duplicates().copy()
            miss["census_block_new"] = [
                _census_api_block(lat, lon, census_geocoder_benchmark, census_geocoder_vintage)
                for lat, lon in zip(miss["lat"], miss["lon"])
            ]
            miss["census_tract_new"] = _tract_from_block(miss["census_block_new"]).astype(str)

            si = si.merge(miss, on=["lat", "lon"], how="left")
            si["census_block"] = si["census_block"].fillna(si["census_block_new"])
            if "census_tract" not in si.columns:
                si["census_tract"] = pd.NA
            si["census_tract"] = si["census_tract"].fillna(si["census_tract_new"])
            si = si.drop(columns=["census_block_new", "census_tract_new"], errors="ignore")

    si["census_block"] = si["census_block"].astype(str)
    if "census_tract" not in si.columns or si["census_tract"].isna().any():
        si["census_tract"] = _tract_from_block(si["census_block"]).astype(str)
    else:
        si["census_tract"] = si["census_tract"].astype(str)

    results["station_info_done"] = si
    if save_outputs:
        si.to_csv(out_dir / f"{city_key.lower()}_station_information_done.csv", index=False)

    # Merge blocks into station status
    ss_done = ss_df.merge(si[["station_id", "census_block"]], on="station_id", how="left")
    ss_done["census_block"] = ss_done["census_block"].astype(str)
    ss_done["census_tract"] = _tract_from_block(ss_done["census_block"]).astype(str)

    # Availability column standardization (tolerant to missing ebikes)
    if "num_bikes_available" not in ss_done.columns:
        raise ValueError(
            "station_status_done is missing 'num_bikes_available'. "
            f"Available columns: {list(ss_done.columns)[:200]}"
        )

    if "num_ebikes_available" not in ss_done.columns:
        ss_done["num_ebikes_available"] = 0

    if "num_docks_available" not in ss_done.columns:
        alt = None
        for c in ["docks_available", "num_docks_free", "num_docks_open"]:
            if c in ss_done.columns:
                alt = c
                break
        if alt is None:
            raise ValueError(
                "station_status_done is missing 'num_docks_available' and no alternative dock column was found. "
                f"Available columns: {list(ss_done.columns)[:200]}"
            )
        ss_done["num_docks_available"] = pd.to_numeric(ss_done[alt], errors="coerce").fillna(0)

    ss_done["num_bikes_available"] = pd.to_numeric(ss_done["num_bikes_available"], errors="coerce").fillna(0).astype(int)
    ss_done["num_ebikes_available"] = pd.to_numeric(ss_done["num_ebikes_available"], errors="coerce").fillna(0).astype(int)
    ss_done["num_docks_available"] = pd.to_numeric(ss_done["num_docks_available"], errors="coerce").fillna(0).astype(int)
    ss_done["total_vehicle_available"] = ss_done["num_bikes_available"] + ss_done["num_ebikes_available"]

    results["station_status_done"] = ss_done
    if save_outputs:
        ss_done.to_csv(out_dir / f"{city_key.lower()}_station_status_done.csv", index=False)

    # ==================================================
    # AVAILABILITY
    # ==================================================
    ss_av = ss_done.copy()
    ss_av["timestamp"] = pd.to_datetime(ss_av["timestamp"], errors="coerce")
    if ss_av["timestamp"].isna().any():
        raise ValueError("Found unparseable timestamps in station_status_done.")

    data_min = ss_av["timestamp"].min()
    data_max = ss_av["timestamp"].max()

    if availability_time_start is not None:
        ss_av = ss_av[ss_av["timestamp"] >= availability_time_start].copy()
    if availability_time_end is not None:
        ss_av = ss_av[ss_av["timestamp"] < availability_time_end].copy()

    if ss_av.empty:
        raise ValueError(f"No availability rows after time filter. Data range: [{data_min}, {data_max}]")

    def _availability_for_level(level: str) -> Dict[str, pd.DataFrame]:
        geo_key = "census_block" if level == "block" else "census_tract"

        temp = ss_av.copy()
        temp["time_slot"] = temp["timestamp"].dt.floor(availability_time_granularity)

        geo_ts = (
            temp.groupby([geo_key, "timestamp"], as_index=False)
            .agg(
                num_bikes_available=("num_bikes_available", "sum"),
                num_ebikes_available=("num_ebikes_available", "sum"),
                num_docks_available=("num_docks_available", "sum"),
                total_vehicle_available=("total_vehicle_available", "sum"),
            )
        )
        geo_ts["time_slot"] = geo_ts["timestamp"].dt.floor(availability_time_granularity)

        raw_df = (
            geo_ts.groupby([geo_key, "time_slot"], as_index=False)
            .mean(numeric_only=True)
            .round(0)
            .sort_values([geo_key, "time_slot"])
            .reset_index(drop=True)
        )

        for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
            raw_df[c] = raw_df[c].astype(int)

        if level == "tract" and external_tract_prefix:
            raw_df = raw_df[~raw_df["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

        norm_df = raw_df.copy()
        for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
            norm_df[c + "_norm"] = _minmax_norm(norm_df[c]).round(5)

        if save_outputs:
            raw_df.to_csv(out_dir / f"availability__raw__{level}.csv", index=False)
            norm_df.to_csv(out_dir / f"availability__norm__{level}.csv", index=False)

        return {"raw": raw_df, "norm": norm_df}

    availability_out: Dict[str, Any] = {"meta": {
        "time_start": availability_time_start,
        "time_end": availability_time_end,
        "data_min": data_min,
        "data_max": data_max,
        "time_granularity": availability_time_granularity,
        "group_level": availability_group_level,
    }}

    if availability_group_level in {"block", "both"}:
        blk = _availability_for_level("block")
        availability_out["availability_block_raw"] = blk["raw"]
        availability_out["availability_block_norm"] = blk["norm"]

    if availability_group_level in {"tract", "both"}:
        tr = _availability_for_level("tract")
        availability_out["availability_tract_raw"] = tr["raw"]
        availability_out["availability_tract_norm"] = tr["norm"]

    if "availability_tract_norm" not in availability_out:
        tr = _availability_for_level("tract")
        availability_out["availability_tract_raw"] = tr["raw"]
        availability_out["availability_tract_norm"] = tr["norm"]

    results["availability"] = availability_out

    availability_df_for_capacity = availability_out["availability_tract_norm"].copy()
    availability_df_for_capacity["census_tract"] = availability_df_for_capacity["census_tract"].astype(str)
    availability_df_for_capacity["time_slot"] = pd.to_datetime(availability_df_for_capacity["time_slot"])

    # ==================================================
    # CAPACITY
    # ==================================================
    import geopandas as gpd

    tracts_gdf = gpd.read_file(str(tracts_path))
    if tracts_gdf.crs is None:
        raise ValueError(f"Tracts shapefile CRS is missing: {tracts_path}")
    if tract_id_col not in tracts_gdf.columns:
        raise ValueError(f"Tract shapefile missing '{tract_id_col}' column.")

    if cfg.get("has_boro_logic", False):
        tracts_gdf["county_fips"] = tracts_gdf["STATEFP"].astype(str) + tracts_gdf["COUNTYFP"].astype(str)
        county_to_boro = {
            "36005": "The Bronx",
            "36047": "Brooklyn",
            "36061": "Manhattan",
            "36081": "Queens",
            "36085": "Staten Island",
        }
        tracts_gdf["boro"] = tracts_gdf["county_fips"].map(county_to_boro)
        if drop_staten_island:
            tracts_gdf = tracts_gdf[tracts_gdf["boro"] != "Staten Island"].copy()

    centroids_wgs84 = gpd.GeoSeries(tracts_gdf.geometry.centroid, crs=tracts_gdf.crs).to_crs(epsg=4326)
    tract_centroids_df = pd.DataFrame({
        "census_tract": tracts_gdf[tract_id_col].astype(str).values,
        "centroid_lon": centroids_wgs84.x.values,
        "centroid_lat": centroids_wgs84.y.values,
    })
    if "boro" in tracts_gdf.columns:
        tract_centroids_df["boro"] = tracts_gdf["boro"].values

    results["tract_centroids_df"] = tract_centroids_df
    if save_outputs:
        tract_centroids_df.to_csv(out_dir / "centroid_tract_computed.csv", index=False)

    station_info = si.copy()
    station_info["census_block"] = station_info["census_block"].astype(str)
    station_info["census_tract"] = _tract_from_block(station_info["census_block"]).astype(str)

    capacity_block = (
        station_info.groupby("census_block", as_index=False)
        .agg(total_capacity=("capacity", "sum"), num_station=("station_id", "count"))
    )
    results["capacity_block"] = capacity_block
    if save_outputs:
        capacity_block.to_csv(out_dir / "capacity_block.csv", index=False)

    tract_capacity = (
        station_info.groupby("census_tract", as_index=False)
        .agg(total_capacity=("capacity", "sum"), num_station=("station_id", "count"))
    )

    capacity_tract = tract_centroids_df[["census_tract"]].drop_duplicates().merge(
        tract_capacity, on="census_tract", how="left"
    )
    capacity_tract[["total_capacity", "num_station"]] = capacity_tract[["total_capacity", "num_station"]].fillna(0)

    if external_tract_prefix:
        capacity_tract = capacity_tract[~capacity_tract["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    capacity_tract = capacity_tract.sort_values("census_tract").reset_index(drop=True)
    capacity_tract["total_capacity_norm"] = _minmax_norm(capacity_tract["total_capacity"]).round(5)
    capacity_tract["num_station_norm"] = _minmax_norm(capacity_tract["num_station"]).round(5)

    results["capacity_tract_norm"] = capacity_tract
    if save_outputs:
        capacity_tract.to_csv(out_dir / "capacity_tract_norm.csv", index=False)

    avail = availability_df_for_capacity.copy()
    if external_tract_prefix:
        avail = avail[~avail["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    if peak_time_slot is None:
        peak_series = avail.groupby("time_slot")[peak_metric].sum()
        if peak_series.empty:
            # Fallback if no data, though we checked earlier
            peak_time_slot = avail["time_slot"].min()
        else:
            peak_time_slot = peak_series.idxmax()
    peak_time_slot = pd.to_datetime(peak_time_slot)

    peak_df = avail[avail["time_slot"] == peak_time_slot].copy()

    vehicle_capacity = (
        peak_df.groupby("census_tract", as_index=False)["total_vehicle_available"]
        .sum()
        .rename(columns={"total_vehicle_available": "vehicle_capacity"})
    )
    dock_capacity = (
        peak_df.groupby("census_tract", as_index=False)["num_docks_available"]
        .sum()
        .rename(columns={"num_docks_available": "dock_capacity"})
    )

    capacity_df = (
        capacity_tract.merge(vehicle_capacity, on="census_tract", how="left")
        .merge(dock_capacity, on="census_tract", how="left")
    )
    capacity_df["vehicle_capacity"] = capacity_df["vehicle_capacity"].fillna(0)
    capacity_df["dock_capacity"] = capacity_df["dock_capacity"].fillna(0)

    capacity_df["vehicle_capacity_norm"] = _minmax_norm(capacity_df["vehicle_capacity"]).round(5)
    capacity_df["dock_capacity_norm"] = _minmax_norm(capacity_df["dock_capacity"]).round(5)

    capacity_df["occupancy_rate"] = capacity_df.apply(
        lambda r: (r["vehicle_capacity"] / r["total_capacity"]) if r["total_capacity"] else 0.0, axis=1
    )
    capacity_df["return_pressure"] = capacity_df.apply(
        lambda r: 1.0 - (r["dock_capacity"] / r["total_capacity"]) if r["total_capacity"] else 0.0, axis=1
    )

    results["capacity_df"] = capacity_df
    results["peak_time_slot"] = peak_time_slot
    results["peak_metric"] = peak_metric
    if save_outputs:
        capacity_df.to_csv(out_dir / "capacity_tract_with_vehicle_and_docks_norm.csv", index=False)

    # ==================================================
    # SAFETY (robust WKT; CRS-safe using blocks CRS)
    # ==================================================
    blocks_gdf = gpd.read_file(str(census_blocks_path))
    if blocks_gdf.crs is None:
        raise ValueError(f"Blocks shapefile CRS is missing: {census_blocks_path}")
    if blocks_id_col not in blocks_gdf.columns:
        raise ValueError(f"Blocks shapefile missing '{blocks_id_col}' column.")

    target_crs = blocks_gdf.crs
    safety_kind = str(cfg.get("safety_input_kind", "shp"))

    streets_gdf = _read_lines_layer(
        centerline_path,
        kind=safety_kind,
        target_crs=target_crs,
        csv_wkt_candidates=tuple(cfg.get("safety_csv_wkt_candidates", ("geometry", "the_geom", "wkt", "shape", "line"))),
        csv_crs=str(cfg.get("safety_csv_crs", "EPSG:4326")),
    )
    bike_gdf = _read_lines_layer(
        bike_lanes_path,
        kind=safety_kind,
        target_crs=target_crs,
        csv_wkt_candidates=tuple(cfg.get("safety_csv_wkt_candidates", ("geometry", "the_geom", "wkt", "shape", "line"))),
        csv_crs=str(cfg.get("safety_csv_crs", "EPSG:4326")),
    )

    street_len = _overlay_length_by_block(streets_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="streets_leng")
    bike_len = _overlay_length_by_block(bike_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="total_bike_lane_length")

    protected_rule = cfg.get("protected_lane_rule", {})
    facility_col, protected_gdf = _protected_filter(bike_gdf, protected_rule)

    if facility_col is None:
        prot_len = pd.DataFrame({"census_block": street_len["census_block"].astype(str).unique()})
        prot_len["protected_bike_lane_length"] = 0.0
        results["safety_note"] = (
            f"[{city_key}] No protected-lane column found. Protected length set to 0. "
            f"Tried: {protected_rule.get('col_candidates')}"
        )
    else:
        prot_len = _overlay_length_by_block(
            protected_gdf, blocks_gdf, blocks_id_col=blocks_id_col, out_col="protected_bike_lane_length"
        )
        results["safety_protected_lane_col_used"] = facility_col
        results["safety_protected_lane_rule"] = protected_rule

    safety_block = (
        street_len.merge(bike_len, on="census_block", how="left")
        .merge(prot_len, on="census_block", how="left")
    )
    safety_block["total_bike_lane_length"] = safety_block["total_bike_lane_length"].fillna(0)
    safety_block["protected_bike_lane_length"] = safety_block["protected_bike_lane_length"].fillna(0)
    safety_block["bike_lane_ratio"] = _safe_ratio(safety_block["total_bike_lane_length"], safety_block["streets_leng"]).round(3)
    safety_block["protected_bike_lane_ratio"] = _safe_ratio(safety_block["protected_bike_lane_length"], safety_block["streets_leng"]).round(3)
    safety_block["census_tract"] = _tract_from_block(safety_block["census_block"]).astype(str)

    # tract safety using centroid file (for region cols)
    centroids_csv_df = pd.read_csv(centroid_tract_csv)
    centroids_csv_df["census_tract"] = centroids_csv_df["census_tract"].astype(str)
    region_cols = ["census_tract"]
    for c in ["boro", "county_name", "county", "state"]:
        if c in centroids_csv_df.columns:
            region_cols.append(c)

    safety_tract = (
        safety_block.groupby("census_tract", as_index=False)[
            ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]
        ].sum()
    )
    safety_tract = centroids_csv_df[region_cols].merge(safety_tract, on="census_tract", how="left")
    for c in ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]:
        safety_tract[c] = safety_tract[c].fillna(0)

    safety_tract["bike_lane_ratio"] = _safe_ratio(safety_tract["total_bike_lane_length"], safety_tract["streets_leng"])
    safety_tract["protected_bike_lane_ratio"] = _safe_ratio(safety_tract["protected_bike_lane_length"], safety_tract["streets_leng"])
    safety_tract = safety_tract.sort_values("census_tract").reset_index(drop=True)

    service_tracts = set(capacity_df["census_tract"].astype(str).unique().tolist())
    safety_service = safety_tract[safety_tract["census_tract"].astype(str).isin(service_tracts)].reset_index(drop=True)

    safety_norm = safety_service.copy()
    safety_norm["bike_lane_ratio_norm"] = _minmax_norm(safety_norm["bike_lane_ratio"]).round(5)
    safety_norm["protected_bike_lane_ratio_norm"] = _minmax_norm(safety_norm["protected_bike_lane_ratio"]).round(5)

    results["safety_bike_lane_block"] = safety_block
    results["safety_bike_lane_tract"] = safety_tract
    results["safety_bike_lane_service_area"] = safety_service
    results["safety_bike_lane_norm_tract"] = safety_norm

    if save_outputs:
        safety_block.to_csv(out_dir / "safety_bike_lane_block.csv", index=False)
        safety_tract.to_csv(out_dir / "safety_bike_lane_tract.csv", index=False)
        safety_service.to_csv(out_dir / "safety_bike_lane_service_area.csv", index=False)
        safety_norm.to_csv(out_dir / "safety_bike_lane_norm_tract.csv", index=False)

    # ==================================================
    # TRIPS — load + standardize + compute start/end census blocks if missing
    # ==================================================
    trip_raw = pd.read_csv(trip_csv)
    trip_df = _standardize_trips(trip_raw)

    trip_df["started_at"] = pd.to_datetime(trip_df["started_at"], errors="coerce")
    trip_df["ended_at"] = pd.to_datetime(trip_df["ended_at"], errors="coerce")

    # Compute blocks from lat/lng if not present
    if ("start_census_block" not in trip_df.columns) or ("end_census_block" not in trip_df.columns):
        if not {"start_lat", "start_lng", "end_lat", "end_lng"}.issubset(trip_df.columns):
            raise ValueError(
                "Trip CSV does not have start/end census blocks AND does not have start/end lat/lng to compute them. "
                f"Found: {list(trip_df.columns)[:200]}"
            )

        start_map = _geocode_points_to_blocks_spatial(
            trip_df, lon_col="start_lng", lat_col="start_lat",
            blocks_path=census_blocks_path, blocks_id_col=blocks_id_col
        ).rename(columns={"census_block": "start_census_block"})

        end_map = _geocode_points_to_blocks_spatial(
            trip_df, lon_col="end_lng", lat_col="end_lat",
            blocks_path=census_blocks_path, blocks_id_col=blocks_id_col
        ).rename(columns={"census_block": "end_census_block"})

        trip_df = trip_df.merge(
            start_map, left_on=["start_lng", "start_lat"], right_on=["lon", "lat"], how="left"
        ).drop(columns=["lon", "lat"], errors="ignore")

        trip_df = trip_df.merge(
            end_map, left_on=["end_lng", "end_lat"], right_on=["lon", "lat"], how="left"
        ).drop(columns=["lon", "lat"], errors="ignore")

    required_trip_cols = {"started_at", "ended_at", "start_census_block", "end_census_block"}
    miss_trip = required_trip_cols - set(trip_df.columns)
    if miss_trip:
        raise ValueError(
            f"Trip CSV is missing required columns after standardization: {miss_trip}. "
            f"Available columns: {list(trip_df.columns)[:200]}"
        )

    trip_df["start_census_block"] = trip_df["start_census_block"].astype(str)
    trip_df["end_census_block"] = trip_df["end_census_block"].astype(str)

    # default usage/idle windows fallback to avail or trip min/max
    if usage_time_start is None:
        usage_time_start = trip_df["started_at"].min()
    if usage_time_end is None:
        usage_time_end = trip_df["started_at"].max()
    if idle_time_start is None:
        idle_time_start = trip_df["started_at"].min()
    if idle_time_end is None:
        idle_time_end = trip_df["started_at"].max()

    usage_time_start = pd.to_datetime(usage_time_start)
    usage_time_end = pd.to_datetime(usage_time_end)
    idle_time_start = pd.to_datetime(idle_time_start)
    idle_time_end = pd.to_datetime(idle_time_end)

    # ==================================================
    # USAGE (5-min, hourly, tract-hourly norm)
    # ==================================================
    trips = trip_df.copy()
    trips = trips[(trips["started_at"] >= usage_time_start) & (trips["started_at"] <= usage_time_end)].copy()
    trips["trip_duration"] = (trips["ended_at"] - trips["started_at"]).dt.total_seconds() / 60.0
    trips = trips[trips["trip_duration"] <= 240].copy()

    # 5-min
    trips["time_slot_start"] = trips["started_at"].dt.floor("5min")
    trips["time_slot_end"] = trips["ended_at"].dt.floor("5min")

    trips_starting_5 = trips.groupby(["start_census_block", "time_slot_start"]).size().reset_index(name="trips_starting")
    trips_ending_5 = trips.groupby(["end_census_block", "time_slot_end"]).size().reset_index(name="trips_ending")

    trips_starting_5 = trips_starting_5.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})
    trips_ending_5 = trips_ending_5.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})

    all_blocks = pd.unique(pd.concat([trips["start_census_block"], trips["end_census_block"]]).astype(str))
    all_blocks = [b for b in all_blocks if b not in ("nan", "0")]

    all_time_slots_5 = pd.date_range(start=usage_time_start.floor("5min"), end=usage_time_end.ceil("5min"), freq="5min")
    grid_5 = pd.MultiIndex.from_product([all_blocks, all_time_slots_5], names=["census_block", "time_slot"]).to_frame(index=False)

    usage_5min_block = (
        grid_5.merge(trips_starting_5, on=["census_block", "time_slot"], how="left")
             .merge(trips_ending_5, on=["census_block", "time_slot"], how="left")
    )
    usage_5min_block[["trips_starting", "trips_ending"]] = usage_5min_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
    usage_5min_block = usage_5min_block.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

    if save_outputs:
        usage_5min_block.to_csv(out_dir / "usage_5min_block.csv", index=False)

    # hourly block
    trips["hour_start"] = trips["started_at"].dt.floor("H")
    trips["hour_end"] = trips["ended_at"].dt.floor("H")

    trips_starting_h = trips.groupby(["start_census_block", "hour_start"]).size().reset_index(name="trips_starting")
    trips_ending_h = trips.groupby(["end_census_block", "hour_end"]).size().reset_index(name="trips_ending")

    trips_starting_h = trips_starting_h.rename(columns={"start_census_block": "census_block", "hour_start": "hour"})
    trips_ending_h = trips_ending_h.rename(columns={"end_census_block": "census_block", "hour_end": "hour"})

    all_hours = pd.date_range(start=usage_time_start.floor("H"), end=usage_time_end.ceil("H"), freq="H")
    grid_h = pd.MultiIndex.from_product([all_blocks, all_hours], names=["census_block", "hour"]).to_frame(index=False)

    usage_hourly_block = (
        grid_h.merge(trips_starting_h, on=["census_block", "hour"], how="left")
             .merge(trips_ending_h, on=["census_block", "hour"], how="left")
    )
    usage_hourly_block[["trips_starting", "trips_ending"]] = usage_hourly_block[["trips_starting", "trips_ending"]].fillna(0).astype(int)
    usage_hourly_block = usage_hourly_block.sort_values(["census_block", "hour"]).reset_index(drop=True)

    if save_outputs:
        usage_hourly_block.to_csv(out_dir / "usage_hourly_block.csv", index=False)

    # hourly tract
    usage_hourly = usage_hourly_block.copy()
    usage_hourly["census_block"] = usage_hourly["census_block"].astype(str)
    usage_hourly["census_tract"] = _tract_from_block(usage_hourly["census_block"]).astype(str)
    if external_tract_prefix:
        usage_hourly = usage_hourly[~usage_hourly["census_tract"].astype(str).str.startswith(external_tract_prefix)].copy()

    usage_hourly["time_slot"] = pd.to_datetime(usage_hourly["hour"])
    usage_hourly = usage_hourly.drop(columns=["hour"])

    usage_hourly_tract = (
        usage_hourly.groupby(["census_tract", "time_slot"], as_index=False)
        .agg(trips_starting=("trips_starting", "sum"), trips_ending=("trips_ending", "sum"))
        .reset_index(drop=True)
    )
    usage_hourly_tract_raw = usage_hourly_tract.copy()

    if tracts_to_remove:
        usage_hourly_tract = usage_hourly_tract[~usage_hourly_tract["census_tract"].isin(tracts_to_remove)].reset_index(drop=True)

    usage_hourly_tract["trips_starting_norm"] = _minmax_norm(usage_hourly_tract["trips_starting"]).round(5)
    usage_hourly_tract["trips_ending_norm"] = _minmax_norm(usage_hourly_tract["trips_ending"]).round(5)

    if save_outputs:
        usage_hourly_tract_raw.to_csv(out_dir / "usage_hourly_tract_raw.csv", index=False)
        usage_hourly_tract.to_csv(out_dir / "usage_norm_hourly_tract.csv", index=False)

    results["usage_5min_block"] = usage_5min_block
    results["usage_hourly_block"] = usage_hourly_block
    results["usage_hourly_tract_raw"] = usage_hourly_tract_raw
    results["usage_norm_hourly_tract"] = usage_hourly_tract

    # ==================================================
    # IDLE TIME (full simulation, same logic as your earlier notebook)
    # ==================================================
    if not compute_idle_time:
        results["idle_note"] = "Idle time skipped (compute_idle_time=False)."
        return results

    # Need rideable_type to split bike vs ebike; if missing -> still compute total moved (no split)
    has_rideable = "rideable_type" in trip_df.columns

    # GBFS input for idle (has vehicles)
    ss_idle = ss_done.copy()
    ss_idle["timestamp"] = pd.to_datetime(ss_idle["timestamp"])
    ss_idle["census_block"] = ss_idle["census_block"].astype(str)
    ss_idle["census_tract"] = ss_idle["census_tract"].astype(str)
    ss_idle["total_available"] = ss_idle["num_bikes_available"] + ss_idle["num_ebikes_available"]
    ss_idle["time_5min"] = ss_idle["timestamp"].dt.floor("5min")

    trips_idle = trip_df.copy()
    trips_idle["start_census_block"] = trips_idle["start_census_block"].astype(str)
    trips_idle["end_census_block"] = trips_idle["end_census_block"].astype(str)
    trips_idle["started_at"] = pd.to_datetime(trips_idle["started_at"])
    trips_idle["ended_at"] = pd.to_datetime(trips_idle["ended_at"])

    # drop external if requested (NYC drops NJ)
    if external_tract_prefix:
        start_tract = _tract_from_block(trips_idle["start_census_block"]).astype(str)
        end_tract = _tract_from_block(trips_idle["end_census_block"]).astype(str)
        trips_idle = trips_idle[
            (~start_tract.str.startswith(external_tract_prefix)) & (~end_tract.str.startswith(external_tract_prefix))
        ].copy()

    # time filter
    trips_idle = trips_idle[(trips_idle["started_at"] >= idle_time_start) & (trips_idle["started_at"] <= idle_time_end)].copy()

    # 5-min buckets
    trips_idle["time_slot_start"] = trips_idle["started_at"].dt.floor("5min")
    trips_idle["time_slot_end"] = trips_idle["ended_at"].dt.floor("5min")

    # If rideable_type exists, compute starting/ending by type (bike vs ebike)
    if has_rideable:
        trips_idle = _map_rideable_type_to_two_columns(trips_idle)

        # starting: bike/ebike counts
        ts_start = (
            trips_idle.groupby(["start_census_block", "time_slot_start", "_is_ebike"])
            .size()
            .unstack(fill_value=0)
            .reset_index()
        )
        ts_start = ts_start.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})
        if True not in ts_start.columns:
            ts_start[True] = 0
        if False not in ts_start.columns:
            ts_start[False] = 0
        ts_start = ts_start.rename(columns={False: "trips_starting_bike", True: "trips_starting_ebike"})

        # ending: bike/ebike counts
        ts_end = (
            trips_idle.groupby(["end_census_block", "time_slot_end", "_is_ebike"])
            .size()
            .unstack(fill_value=0)
            .reset_index()
        )
        ts_end = ts_end.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})
        if True not in ts_end.columns:
            ts_end[True] = 0
        if False not in ts_end.columns:
            ts_end[False] = 0
        ts_end = ts_end.rename(columns={False: "trips_ending_bike", True: "trips_ending_ebike"})

    else:
        # No rideable_type: only total starting/ending
        ts_start = trips_idle.groupby(["start_census_block", "time_slot_start"]).size().reset_index(name="trips_starting")
        ts_end = trips_idle.groupby(["end_census_block", "time_slot_end"]).size().reset_index(name="trips_ending")
        ts_start = ts_start.rename(columns={"start_census_block": "census_block", "time_slot_start": "time_slot"})
        ts_end = ts_end.rename(columns={"end_census_block": "census_block", "time_slot_end": "time_slot"})

    # full grid
    all_blocks_idle = pd.unique(pd.concat([trips_idle["start_census_block"], trips_idle["end_census_block"]]))
    all_blocks_idle = pd.Series(all_blocks_idle).dropna().astype(str).unique()

    all_time_slots_idle = pd.date_range(
        start=idle_time_start.floor("5min"),
        end=idle_time_end.ceil("5min"),
        freq="5min",
    )
    grid_idle = pd.MultiIndex.from_product(
        [all_blocks_idle, all_time_slots_idle],
        names=["census_block", "time_slot"],
    ).to_frame(index=False)

    if has_rideable:
        result_complete = grid_idle.merge(ts_start, on=["census_block", "time_slot"], how="left").merge(
            ts_end, on=["census_block", "time_slot"], how="left"
        )
        cols = ["trips_starting_bike", "trips_starting_ebike", "trips_ending_bike", "trips_ending_ebike"]
        for c in cols:
            if c not in result_complete.columns:
                result_complete[c] = 0
            result_complete[c] = result_complete[c].fillna(0).astype(int)

        result_complete["trips_starting"] = result_complete["trips_starting_bike"] + result_complete["trips_starting_ebike"]
        result_complete["trips_ending"] = result_complete["trips_ending_bike"] + result_complete["trips_ending_ebike"]

    else:
        result_complete = grid_idle.merge(ts_start, on=["census_block", "time_slot"], how="left").merge(
            ts_end, on=["census_block", "time_slot"], how="left"
        )
        for c in ["trips_starting", "trips_ending"]:
            if c not in result_complete.columns:
                result_complete[c] = 0
            result_complete[c] = result_complete[c].fillna(0).astype(int)

    result_complete = result_complete.sort_values(["census_block", "time_slot"]).reset_index(drop=True)

    # station availability by block 5-min
    station_availability_df = (
        ss_idle.groupby(["census_block", "time_5min"], as_index=False)[
            ["num_bikes_available", "num_ebikes_available", "total_available"]
        ].sum()
    )
    station_availability_df = station_availability_df.rename(columns={"time_5min": "time_slot"})
    station_availability_df["time_slot"] = station_availability_df["time_slot"] - pd.Timedelta(minutes=5)

    merged_idle = station_availability_df.merge(result_complete, on=["census_block", "time_slot"], how="left")
    if has_rideable:
        cols = ["trips_starting_bike", "trips_starting_ebike", "trips_ending_bike", "trips_ending_ebike", "trips_starting", "trips_ending"]
        for c in cols:
            if c not in merged_idle.columns:
                merged_idle[c] = 0
            merged_idle[c] = merged_idle[c].fillna(0).astype(int)

        merged_idle["vehicles_moved_bike"] = merged_idle["trips_starting_bike"] + merged_idle["trips_ending_bike"]
        merged_idle["vehicles_moved_ebike"] = merged_idle["trips_starting_ebike"] + merged_idle["trips_ending_ebike"]
        merged_idle["vehicles_moved"] = merged_idle["trips_starting"] + merged_idle["trips_ending"]
    else:
        for c in ["trips_starting", "trips_ending"]:
            if c not in merged_idle.columns:
                merged_idle[c] = 0
            merged_idle[c] = merged_idle[c].fillna(0).astype(int)
        merged_idle["vehicles_moved"] = merged_idle["trips_starting"] + merged_idle["trips_ending"]

    merged_idle["idle_vehicles"] = (merged_idle["total_available"] - merged_idle["vehicles_moved"]).clip(lower=0)
    merged_idle["time_slot"] = pd.to_datetime(merged_idle["time_slot"])
    merged_idle = merged_idle.sort_values(["census_block", "time_slot"])
    merged_idle["hour"] = merged_idle["time_slot"].dt.floor("H")

    # hourly idle simulation per block
    from collections import deque

    idle_results = []
    for block, block_df in merged_idle.groupby("census_block"):
        for hour, hour_df in block_df.groupby("hour"):
            hour_df = hour_df.sort_values("time_slot")
            virtual_vehicles = deque()
            finalized_durations = []

            for _, row in hour_df.iterrows():
                current_time = row["time_slot"]
                current_idle = int(row["idle_vehicles"])

                if current_idle > len(virtual_vehicles):
                    for _ in range(current_idle - len(virtual_vehicles)):
                        virtual_vehicles.append(current_time)
                elif current_idle < len(virtual_vehicles):
                    for _ in range(len(virtual_vehicles) - current_idle):
                        start_t = virtual_vehicles.popleft()
                        duration = (current_time - start_t).total_seconds() / 60.0
                        finalized_durations.append(duration)

            final_time = hour + pd.Timedelta(hours=1)
            while virtual_vehicles:
                start_t = virtual_vehicles.popleft()
                duration = (final_time - start_t).total_seconds() / 60.0
                finalized_durations.append(duration)

            avg_idle_time = round(sum(finalized_durations) / len(finalized_durations), 2) if finalized_durations else 0.0
            idle_results.append({"census_block": block, "hour": hour, "avg_idle_time": avg_idle_time})

    idle_time_block = pd.DataFrame(idle_results)

    # tract aggregation + normalize
    idle_time_block["census_block"] = idle_time_block["census_block"].astype(str)
    idle_time_block["census_tract"] = _tract_from_block(idle_time_block["census_block"]).astype(str)

    idle_time_tract = idle_time_block.groupby(["census_tract", "hour"], as_index=False)["avg_idle_time"].mean()

    if external_tract_prefix:
        idle_time_tract = idle_time_tract[
            ~idle_time_tract["census_tract"].astype(str).str.startswith(external_tract_prefix)
        ].reset_index(drop=True)

    idle_time_tract["avg_idle_time_norm"] = _minmax_norm(idle_time_tract["avg_idle_time"]).round(5)
    idle_time_tract = idle_time_tract.rename(columns={"hour": "time_slot"})

    results["idle_merged_5min"] = merged_idle
    results["idle_time_block"] = idle_time_block
    results["idle_time_tract_raw"] = idle_time_tract.drop(columns=["avg_idle_time_norm"])
    results["idle_time_norm_tract"] = idle_time_tract

    if save_outputs:
        merged_idle.to_csv(out_dir / "idle_merged_5min.csv", index=False)
        idle_time_block.to_csv(out_dir / "idle_time_block.csv", index=False)
        results["idle_time_tract_raw"].to_csv(out_dir / "idle_time_tract_raw.csv", index=False)
        idle_time_tract.to_csv(out_dir / "idle_time_norm_tract.csv", index=False)

    return results

In [2]:
# Example Call for San Francisco Docked
out_docked = run_docked_all_utilities_single_function(
    city="SF",
    
    # Inputs
    station_status_txt=r"D:\Research Fellowship\Summer Research Stuff\Collected Data\Week 1\09-June\san_fran_baywheels_docked_station_status_6_9.txt",
    station_information_csv=r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\San_Fran_Baywheels\San_fran_Baywheels station information 06_09.csv",
    trip_csv=r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Usage\San_Fran_Baywheels\202505-baywheels-tripdata.csv",
    
    # Output Location
    output_dir="San_Fran_Baywheels_Docked_Output",
    
    # Global Time Window (Propagates to all utilities)
    time_start="2025-06-09 06:00:00",
    time_end="2025-06-09 12:00:00",
    
    # Optional: Disable idle simulation if it takes too long
    compute_idle_time=True
)

C:\Users\Vedant\AppData\Local\Temp\ipykernel_23924\1654986023.py:757: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  temp["time_slot"] = temp["timestamp"].dt.floor(availability_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_23924\1654986023.py:768: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  geo_ts["time_slot"] = geo_ts["timestamp"].dt.floor(availability_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_23924\1654986023.py:757: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  temp["time_slot"] = temp["timestamp"].dt.floor(availability_time_granularity)
C:\Users\Vedant\AppData\Local\Temp\ipykernel_23924\1654986023.py:768: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  geo_ts["time_slot"] = geo_ts["timestamp"].dt.floor(availability_time_granularity)


In [31]:
from __future__ import annotations

import json
import warnings
from collections import deque
from pathlib import Path
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from shapely import wkt
from shapely.geometry import Point
from tqdm import tqdm

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# ==================================================
# 1. CITY CONFIGURATION
#    Added "metric_crs" to ensure length calculations are in Meters, not Degrees.
# ==================================================
CITY_CONFIG: Dict[str, Dict[str, Any]] = {
    "NYC": {
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NYC\tl_2024_36_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NYC\tl_2024_36_tract.shp",
            "centroid_csv": r"D:\Research Fellowship\Capacity_NYC\centroid_tract_computed.csv",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\CSCL_PlowNYC_20250619.csv",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\New_York_City_Bike_Routes_20250619.csv",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "metric_crs": "EPSG:32618",  # UTM Zone 18N (Meters)
            "safety_type": "csv_wkt",
            "wkt_candidates": ("geometry", "the_geom", "wkt", "geometry_wkt", "WKT", "geom", "shape", "line"),
            "external_tract_prefix": "34",
            "drop_staten_island": True,
        },
        "safety_rule": {
            "col_candidates": ("facilitycl", "facility", "class", "ft", "type"),
            "match_type": "equals",
            "match_value": "I",
        },
    },
    "NJ": {
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\tl_2024_34_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\tl_2024_34_tract.shp",
            "centroid_csv": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\Tran_road.shp",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "metric_crs": "EPSG:32618",  # UTM Zone 18N (Meters)
            "safety_type": "shp",
            "wkt_candidates": (),
            "external_tract_prefix": None,
            "drop_staten_island": False,
        },
        "safety_rule": {
            "col_candidates": ("type", "facility", "facilitycl", "class", "lane_type"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
    "PITT": {
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\tl_2024_42_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\tl_2024_42_tract.shp",
            "centroid_csv": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\centroid_tract_pa.csv",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Pittsburgh_Street_Centerline.shp",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Bike Map\Bike Lanes.shp",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "metric_crs": "EPSG:32617",  # UTM Zone 17N (Meters)
            "safety_type": "shp",
            "wkt_candidates": (),
            "external_tract_prefix": None,
            "drop_staten_island": False,
        },
        "safety_rule": {
            "col_candidates": ("facility", "type", "class", "status", "lane_type", "BIKE_FACIL", "CATEGORY"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
    "SF": {
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Usage\San_Fran_Baywheels\tl_2024_06_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\San_Fran_Baywheels\tl_2024_06_tract.shp",
            "centroid_csv": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\San_Fran_Baywheels\centroid_tract_ca.csv",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Streets___Active_and_Retired_20250626 (1).csv",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Bikelane.csv",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "metric_crs": "EPSG:26910",  # UTM Zone 10N (Meters) - Crucial for SF
            "safety_type": "csv_wkt",
            "wkt_candidates": ("geometry", "shape", "line", "the_geom", "wkt", "geometry_wkt", "WKT", "geom"),
            "external_tract_prefix": None,
            "drop_staten_island": False,
        },
        "safety_rule": {
            "col_candidates": ("BARRIER", "FACILITY_T", "BUFFERED", "RAISED", "SYMBOLOGY"),
            "match_type": "not_empty_or_contains",
            "match_value": r"PROTECT|SEPARAT|CYCLETRACK|BARRIER|RAISED|BUFFER",
        },
    },
}


# ==================================================
# 2. UTILITY CLASSES
# ==================================================
class _Standardizer:
    @staticmethod
    def pick_col(df: pd.DataFrame, candidates: Sequence[str], required: bool = False, label: str = "") -> Optional[str]:
        cols = set(df.columns)
        for c in candidates:
            if c in cols: return c
        if required:
            raise ValueError(f"Missing required col for {label}. Tried: {candidates}")
        return None

    @staticmethod
    def normalize_tract(s: pd.Series) -> pd.Series:
        """Ensures tract IDs are 11 digits (pads with leading zeros)."""
        return s.astype(str).str.split('.').str[0].str.zfill(11)

    @staticmethod
    def tract_from_block(s: pd.Series) -> pd.Series:
        s = s.astype(str).str.split('.').str[0]
        return s.str[:11]

    @staticmethod
    def normalize_id(s: pd.Series) -> pd.Series:
        """Standardizes IDs: to string, uppercase, stripped, no .0 decimals."""
        return s.astype(str).str.strip().str.upper().str.replace(r"\.0$", "", regex=True)

    @classmethod
    def stations(cls, df: pd.DataFrame) -> pd.DataFrame:
        mapping = {
            cls.pick_col(df, ["station_id", "stationId", "id", "Station ID"], True, "station_id"): "station_id",
            cls.pick_col(df, ["lat", "latitude", "y"], True, "lat"): "lat",
            cls.pick_col(df, ["lon", "lng", "longitude", "x"], True, "lon"): "lon",
            cls.pick_col(df, ["capacity", "dock_count", "num_docks"], True, "capacity"): "capacity"
        }
        out = df.rename(columns=mapping)
        out["station_id"] = cls.normalize_id(out["station_id"])
        
        cb = cls.pick_col(out, ["census_block", "block_geoid", "GEOID20"])
        if cb and cb != "census_block": out = out.rename(columns={cb: "census_block"})
        
        ct = cls.pick_col(out, ["census_tract", "tract_geoid", "GEOID"])
        if ct and ct != "census_tract": out = out.rename(columns={ct: "census_tract"})
        
        return out

    @classmethod
    def trips(cls, df: pd.DataFrame) -> pd.DataFrame:
        mapping = {
            cls.pick_col(df, ["started_at", "start_time", "starttime", "Start Date"], True, "start"): "started_at",
            cls.pick_col(df, ["ended_at", "end_time", "stoptime", "End Date"], True, "end"): "ended_at"
        }
        
        opt_map = {
            cls.pick_col(df, ["rideable_type", "bike_type", "vehicle_type"]): "rideable_type",
            cls.pick_col(df, ["start_census_block", "start_block"]): "start_census_block",
            cls.pick_col(df, ["end_census_block", "end_block"]): "end_census_block",
            cls.pick_col(df, ["start_lat", "startLatitude", "StartLat"]): "start_lat",
            cls.pick_col(df, ["start_lng", "startLongitude", "StartLng"]): "start_lng",
            cls.pick_col(df, ["end_lat", "endLatitude", "EndLat"]): "end_lat",
            cls.pick_col(df, ["end_lng", "endLongitude", "EndLng"]): "end_lng",
            cls.pick_col(df, ["start_station_id", "Start Station Id"]): "start_station_id",
            cls.pick_col(df, ["end_station_id", "End Station Id"]): "end_station_id"
        }
        mapping.update({k: v for k, v in opt_map.items() if k})
        
        df = df.rename(columns=mapping)
        if "start_station_id" in df.columns:
            df["start_station_id"] = cls.normalize_id(df["start_station_id"])
        if "end_station_id" in df.columns:
            df["end_station_id"] = cls.normalize_id(df["end_station_id"])
            
        return df


class _GeoUtils:
    @staticmethod
    def load_lines(path: Path, kind: str, target_crs, csv_candidates: tuple) -> gpd.GeoDataFrame:
        if kind == "shp":
            gdf = gpd.read_file(path)
            if gdf.crs is None: raise ValueError(f"No CRS in {path}")
            return gdf.to_crs(target_crs)
        
        if isinstance(csv_candidates, str): csv_candidates = (csv_candidates,)

        df = pd.read_csv(path, engine="c")
        wkt_col = _Standardizer.pick_col(df, csv_candidates, True, "WKT Column")
        
        def safe_load(x):
            try: return wkt.loads(str(x))
            except: return None
            
        df["geometry"] = df[wkt_col].apply(safe_load)
        gdf = gpd.GeoDataFrame(df[df["geometry"].notna()], geometry="geometry", crs="EPSG:4326")
        return gdf.to_crs(target_crs)

    @staticmethod
    def geocode_points(df: pd.DataFrame, lon_col: str, lat_col: str, blocks_gdf: gpd.GeoDataFrame, id_col: str) -> pd.DataFrame:
        points = df[[lon_col, lat_col]].dropna().drop_duplicates()
        if points.empty:
            return pd.DataFrame(columns=[lon_col, lat_col, "census_block"])

        gdf = gpd.GeoDataFrame(
            points, 
            geometry=[Point(xy) for xy in zip(points[lon_col], points[lat_col])],
            crs="EPSG:4326"
        )
        
        joined = gpd.sjoin(gdf, blocks_gdf[[id_col, "geometry"]], how="left", predicate="within")
        joined = joined.rename(columns={id_col: "census_block"})
        
        out = joined[[lon_col, lat_col, "census_block"]].copy()
        out = out.rename(columns={lon_col: "lon", lat_col: "lat"})
        return out

    @staticmethod
    def api_fill_blocks(df: pd.DataFrame, benchmark: str, vintage: str) -> pd.DataFrame:
        missing = df[df["census_block"].isna()][["lat", "lon"]].drop_duplicates()
        if missing.empty: return df
        
        if len(missing) > 5000:
            print(f"⚠️ Warning: Too many missing blocks ({len(missing)}). API fill capped at 5000.")
            missing = missing.head(5000)

        def fetch(r):
            try:
                url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={r.lon}&y={r.lat}&benchmark={benchmark}&vintage={vintage}&format=json"
                js = requests.get(url, timeout=5).json()
                return js["result"]["geographies"]["2020 Census Blocks"][0]["GEOID"]
            except: return None

        tqdm.pandas(desc="API Geocoding")
        missing["new_block"] = missing.progress_apply(fetch, axis=1)
        
        merged = df.merge(missing, on=["lat", "lon"], how="left")
        if "new_block" in merged.columns:
            merged["census_block"] = merged["census_block"].fillna(merged["new_block"])
            merged = merged.drop(columns=["new_block"])
        return merged


# ==================================================
# 3. METRIC CALCULATORS
# ==================================================
def _calc_availability(df: pd.DataFrame, level: str, gran: str, out_dir: Path, save: bool, ext_prefix: str = None) -> Dict:
    geo = "census_block" if level == "block" else "census_tract"
    agg = df.groupby([geo, "timestamp"], as_index=False)[
        ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]
    ].sum()
    
    agg["time_slot"] = agg["timestamp"].dt.floor(gran)
    raw = agg.groupby([geo, "time_slot"], as_index=False).mean(numeric_only=True).round(0)
    for c in raw.columns: 
        if "available" in c: raw[c] = raw[c].astype(int)
            
    if level == "tract":
        raw["census_tract"] = _Standardizer.normalize_tract(raw["census_tract"])
        if ext_prefix:
            raw = raw[~raw["census_tract"].str.startswith(ext_prefix)]

    norm = raw.copy()
    for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
        norm[f"{c}_norm"] = _minmax_norm(norm[c]).round(5)

    if save:
        raw.to_csv(out_dir / f"availability__raw__{level}.csv", index=False)
        norm.to_csv(out_dir / f"availability__norm__{level}.csv", index=False)
        
    return {"raw": raw, "norm": norm}


def _calc_usage(trips: pd.DataFrame, time_range: Tuple, blocks: List, gran_str: str, out_name: str, out_dir: Path, save: bool) -> pd.DataFrame:
    trips = trips.copy()
    trips["slot_start"] = trips["started_at"].dt.floor(gran_str)
    trips["slot_end"] = trips["ended_at"].dt.floor(gran_str)
    
    starts = trips.groupby(["start_census_block", "slot_start"]).size().rename("trips_starting")
    ends = trips.groupby(["end_census_block", "slot_end"]).size().rename("trips_ending")
    
    slots = pd.date_range(start=time_range[0].floor(gran_str), end=time_range[1].ceil(gran_str), freq=gran_str)
    grid = pd.MultiIndex.from_product([blocks, slots], names=["census_block", "time_slot"]).to_frame(index=False)
    
    out = grid.merge(starts, left_on=["census_block", "time_slot"], right_index=True, how="left") \
              .merge(ends, left_on=["census_block", "time_slot"], right_index=True, how="left")
              
    out = out.fillna(0).astype({"trips_starting": int, "trips_ending": int})
    
    if save: out.to_csv(out_dir / f"{out_name}.csv", index=False)
    return out

def _minmax_norm(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    mn, mx = s.min(skipna=True), s.max(skipna=True)
    if pd.isna(mn) or pd.isna(mx) or mx <= mn: return pd.Series(0.0, index=s.index)
    return (s - mn) / (mx - mn)

def _safe_ratio(numer: pd.Series, denom: pd.Series) -> pd.Series:
    return numer.div(denom.where(denom > 0, other=pd.NA)).fillna(0)

def _map_rideable_type_to_two_columns(trips: pd.DataFrame) -> pd.DataFrame:
    df = trips.copy()
    if "rideable_type" not in df.columns: return df
    t = df["rideable_type"].astype(str).str.lower()
    df["_is_ebike"] = t.str.contains("electric|ebike|e-bike|assist", na=False)
    return df


def _overlay_length_by_block(lines_gdf, blocks_gdf, blocks_id_col, out_col):
    """
    Computes length of lines per block.
    ASSUMES both GDFs are already projected to a METRIC CRS (meters).
    """
    inter = gpd.overlay(lines_gdf, blocks_gdf, how="intersection", keep_geom_type=False)
    if inter.empty:
        return pd.DataFrame({"census_block": pd.Series([], dtype=str), out_col: pd.Series([], dtype=float)})

    inter["seg_len"] = inter.geometry.length
    inter = inter.rename(columns={blocks_id_col: "census_block"})
    out = inter.groupby("census_block", as_index=False)["seg_len"].sum().rename(columns={"seg_len": out_col})
    
    # Rounding here is fine now because units are Meters, not Degrees
    out[out_col] = out[out_col].round(3)
    out["census_block"] = out["census_block"].astype(str)
    return out


def _protected_filter(lines_gdf, rule: Dict[str, Any]):
    col = _Standardizer.pick_col(lines_gdf, list(rule.get("col_candidates", [])), required=False, label="protected lane col")
    if col is None:
        return None, lines_gdf

    s = lines_gdf[col].astype(str)
    match_type = str(rule.get("match_type", "equals")).lower()
    match_value = str(rule.get("match_value", ""))

    if match_type == "equals":
        mask = s.str.upper() == match_value.upper()
    elif match_type == "contains":
        mask = s.str.upper().str.contains(match_value.upper(), na=False)
    elif match_type == "not_empty_or_contains":
        s2 = lines_gdf[col]
        non_empty = s2.notna() & (s.astype(str).str.strip() != "") & (~s.astype(str).str.lower().isin(["0", "false", "none", "nan"]))
        contains = s.astype(str).str.contains(match_value, case=False, na=False, regex=True)
        mask = non_empty | contains
    else:
        # fallback
        mask = s == match_value

    return col, lines_gdf[mask].copy()


# ==================================================
# 4. MAIN FUNCTION
# ==================================================
def run_docked_all_utilities_single_function(
    *,
    city: str,
    station_status_txt: Union[str, Path],
    station_information_csv: Union[str, Path],
    trip_csv: Union[str, Path],
    output_dir: Optional[Union[str, Path]] = None,
    save_outputs: bool = True,
    
    # Global Time Window
    time_start: Optional[Union[str, pd.Timestamp]] = None,
    time_end: Optional[Union[str, pd.Timestamp]] = None,
    
    # Toggles & Overrides
    compute_idle_time: bool = True,
    availability_time_start: Optional[Union[str, pd.Timestamp]] = None,
    availability_time_end: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_start: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_end: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_start: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_end: Optional[Union[str, pd.Timestamp]] = None,
    
    # Granularity & Settings
    availability_time_granularity: str = "1h",
    availability_group_level: str = "both",
    peak_time_slot: Optional[Union[str, pd.Timestamp]] = None,
    peak_metric: str = "total_vehicle_available",
    drop_staten_island: Optional[bool] = None,
    tracts_to_remove: Optional[Sequence[str]] = None,
    remove_tz_suffix: str = " EDT",
    fill_missing_with_census_api: bool = True,
    census_geocoder_benchmark: str = "Public_AR_Census2020",
    census_geocoder_vintage: str = "Census2020_Current",
) -> Dict[str, Any]:
    
    # ---------------- Setup ----------------
    city_key = city.strip().upper()
    aliases = {"PITTSBURGH": "PITT", "PIT": "PITT", "SANFRAN": "SF", "SAN FRANCISCO": "SF", "BAYWHEELS": "SF"}
    city_key = aliases.get(city_key, city_key)
    
    if city_key not in CITY_CONFIG:
        raise ValueError(f"City {city_key} not found in config.")
    
    cfg = CITY_CONFIG[city_key]
    out_dir = Path(output_dir or f"./{city_key}_ALL_IN_ONE_OUTPUTS")
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # Load Blocks ONCE
    print("Loading Census Blocks (Heavy Operation)...")
    blocks_id = str(cfg["geo"]["blocks_id"])
    blocks_gdf = gpd.read_file(cfg["assets"]["census_blocks"]).to_crs(epsg=4326)
    
    # Normalize Times
    ts_start = pd.to_datetime(time_start) if time_start else None
    ts_end = pd.to_datetime(time_end) if time_end else None
    
    t_avail_start = pd.to_datetime(availability_time_start) or ts_start
    t_avail_end = pd.to_datetime(availability_time_end) or ts_end
    t_usage_start = pd.to_datetime(usage_time_start) or ts_start
    t_usage_end = pd.to_datetime(usage_time_end) or ts_end
    t_idle_start = pd.to_datetime(idle_time_start) or ts_start
    t_idle_end = pd.to_datetime(idle_time_end) or ts_end

    results = {"city": city_key, "output_dir": str(out_dir)}

    # ---------------- 1. Parse Station Status ----------------
    print("Processing Station Status...")
    records = []
    with Path(station_status_txt).open("r", encoding="utf-8") as f:
        for line in f:
            if not line.strip(): continue
            try:
                js = json.loads(line)
                ts = list(js.keys())[0]
                for entry in js[ts]:
                    for vt in entry.get("vehicle_types_available", []):
                        entry[f"vehicle_type_{vt['vehicle_type_id']}_count"] = vt["count"]
                    entry.pop("vehicle_types_available", None)
                    entry["timestamp"] = ts
                    records.append(entry)
            except: continue
    
    ss_df = pd.DataFrame(records)
    if save_outputs: ss_df.to_csv(out_dir / f"{city_key.lower()}_station_status_flat.csv", index=False)
    
    ss_df["timestamp"] = ss_df["timestamp"].astype(str).str.replace(remove_tz_suffix, "", regex=False)
    ss_df["timestamp"] = pd.to_datetime(ss_df["timestamp"], errors="coerce")
    ss_df["station_id"] = _Standardizer.normalize_id(ss_df["station_id"])
    
    if save_outputs: ss_df.to_csv(out_dir / f"{city_key.lower()}_station_status_clean.csv", index=False)

    # ---------------- 2. Station Info & Geocoding ----------------
    print("Processing Station Information...")
    si = _Standardizer.stations(pd.read_csv(station_information_csv))
    
    if "census_block" not in si.columns or si["census_block"].isna().any():
        mapping = _GeoUtils.geocode_points(si, "lon", "lat", blocks_gdf, blocks_id)
        si = si.merge(mapping, on=["lon", "lat"], how="left", suffixes=("", "_new"))
        
        if "census_block_new" in si.columns:
            si["census_block"] = si["census_block"].fillna(si["census_block_new"])
            si = si.drop(columns=["census_block_new"])
        
        if fill_missing_with_census_api:
            si = _GeoUtils.api_fill_blocks(si, census_geocoder_benchmark, census_geocoder_vintage)

    si["census_block"] = si["census_block"].astype(str).str.split('.').str[0]
    si["census_tract"] = _Standardizer.tract_from_block(si["census_block"])
    si["census_tract"] = _Standardizer.normalize_tract(si["census_tract"])
    
    if save_outputs: si.to_csv(out_dir / f"{city_key.lower()}_station_information_done.csv", index=False)

    ss_done = ss_df.merge(si[["station_id", "census_block", "census_tract"]], on="station_id", how="left")
    
    for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available"]:
        if c in ss_done.columns:
            ss_done[c] = pd.to_numeric(ss_done[c], errors="coerce").fillna(0).astype(int)
        else:
            ss_done[c] = 0
            
    ss_done["total_vehicle_available"] = ss_done["num_bikes_available"] + ss_done["num_ebikes_available"]
    
    if save_outputs: ss_done.to_csv(out_dir / f"{city_key.lower()}_station_status_done.csv", index=False)

    # ---------------- 3. Availability Metrics ----------------
    print("Computing Availability...")
    ss_filt = ss_done.copy()
    if t_avail_start: ss_filt = ss_filt[ss_filt["timestamp"] >= t_avail_start]
    if t_avail_end: ss_filt = ss_filt[ss_filt["timestamp"] < t_avail_end]
    
    ext_prefix = cfg["geo"]["external_tract_prefix"]
    avail_res = {}
    
    if availability_group_level in {"block", "both"}:
        res = _calc_availability(ss_filt, "block", availability_time_granularity, out_dir, save_outputs)
        avail_res["block"] = res
        
    if availability_group_level in {"tract", "both"}:
        res = _calc_availability(ss_filt, "tract", availability_time_granularity, out_dir, save_outputs, ext_prefix)
        avail_res["tract"] = res

    # ---------------- 4. Capacity Metrics ----------------
    print("Computing Capacity...")
    tract_gdf = gpd.read_file(cfg["assets"]["tracts"]).to_crs(epsg=4326)
    tract_centroids = pd.DataFrame({
        "census_tract": tract_gdf[cfg["geo"]["tract_id"]].astype(str),
        "centroid_lon": tract_gdf.geometry.centroid.x,
        "centroid_lat": tract_gdf.geometry.centroid.y
    })
    
    tract_centroids["census_tract"] = _Standardizer.normalize_tract(tract_centroids["census_tract"])
    
    if cfg["geo"].get("drop_staten_island", False) and "COUNTYFP" in tract_gdf.columns:
        tract_centroids = tract_centroids[tract_gdf["COUNTYFP"].astype(str) != "085"]

    if save_outputs: tract_centroids.to_csv(out_dir / "centroid_tract_computed.csv", index=False)

    cap_block = si.groupby("census_block", as_index=False).agg(
        total_capacity=("capacity", "sum"), num_station=("station_id", "count")
    )
    if save_outputs: cap_block.to_csv(out_dir / "capacity_block.csv", index=False)

    cap_tract_agg = si.groupby("census_tract", as_index=False).agg(
        total_capacity=("capacity", "sum"), num_station=("station_id", "count")
    )
    cap_tract = tract_centroids[["census_tract"]].merge(cap_tract_agg, on="census_tract", how="left").fillna(0)
    
    if ext_prefix: cap_tract = cap_tract[~cap_tract["census_tract"].str.startswith(ext_prefix)]
        
    cap_tract["total_capacity_norm"] = _minmax_norm(cap_tract["total_capacity"]).round(5)
    cap_tract["num_station_norm"] = _minmax_norm(cap_tract["num_station"]).round(5)
    
    if save_outputs: cap_tract.to_csv(out_dir / "capacity_tract_norm.csv", index=False)

    # Peak logic
    tract_av_norm = avail_res["tract"]["norm"]
    peak_agg = tract_av_norm.groupby("time_slot")[peak_metric].sum()
    peak_ts = peak_time_slot if peak_time_slot else peak_agg.idxmax()
    
    peak_data = tract_av_norm[tract_av_norm["time_slot"] == peak_ts][
        ["census_tract", "total_vehicle_available", "num_docks_available"]
    ].rename(columns={"total_vehicle_available": "vehicle_capacity", "num_docks_available": "dock_capacity"})
    
    cap_df = cap_tract.merge(peak_data, on="census_tract", how="left").fillna(0)
    cap_df["vehicle_capacity_norm"] = _minmax_norm(cap_df["vehicle_capacity"]).round(5)
    cap_df["dock_capacity_norm"] = _minmax_norm(cap_df["dock_capacity"]).round(5)
    
    cap_df["occupancy_rate"] = _safe_ratio(cap_df["vehicle_capacity"], cap_df["total_capacity"])
    cap_df["return_pressure"] = 1.0 - _safe_ratio(cap_df["dock_capacity"], cap_df["total_capacity"])
    
    if save_outputs: cap_df.to_csv(out_dir / "capacity_tract_with_vehicle_and_docks_norm.csv", index=False)

    # ---------------- 5. Safety Metrics ----------------
    print("Computing Safety...")
    
    # CRITICAL FIX: Project to Metric CRS before calculating length
    metric_crs = cfg["geo"].get("metric_crs")
    if not metric_crs:
        raise ValueError(f"metric_crs is missing for {city_key}. Add it to CITY_CONFIG!")
    
    # 1. Project Blocks
    blocks_metric = blocks_gdf.to_crs(metric_crs)
    
    # 2. Project Lines
    streets = _GeoUtils.load_lines(
        Path(cfg["assets"]["centerline"]), 
        cfg["geo"]["safety_type"], 
        metric_crs, # Load directly into Metric CRS
        cfg["geo"].get("wkt_candidates", ())
    )
    
    bike_lanes = _GeoUtils.load_lines(
        Path(cfg["assets"]["bike_lanes"]), 
        cfg["geo"]["safety_type"], 
        metric_crs, # Load directly into Metric CRS
        cfg["geo"].get("wkt_candidates", ())
    )
    
    rule = cfg["safety_rule"]
    p_col, protected = _protected_filter(bike_lanes, rule)
    
    st_len = _overlay_length_by_block(streets, blocks_metric, blocks_id, "streets_leng")
    bl_len = _overlay_length_by_block(bike_lanes, blocks_metric, blocks_id, "total_bike_lane_length")
    
    if p_col:
        pr_len = _overlay_length_by_block(protected, blocks_metric, blocks_id, "protected_bike_lane_length")
    else:
        pr_len = pd.DataFrame({"census_block": st_len["census_block"].unique(), "protected_bike_lane_length": 0.0})

    safe_blk = st_len.merge(bl_len, on="census_block", how="left").merge(pr_len, on="census_block", how="left").fillna(0)
    
    # Normalize
    safe_blk["census_tract"] = _Standardizer.tract_from_block(safe_blk["census_block"])
    safe_blk["census_tract"] = _Standardizer.normalize_tract(safe_blk["census_tract"])
    
    safe_blk["bike_lane_ratio"] = _safe_ratio(safe_blk["total_bike_lane_length"], safe_blk["streets_leng"]).round(3)
    safe_blk["protected_bike_lane_ratio"] = _safe_ratio(safe_blk["protected_bike_lane_length"], safe_blk["streets_leng"]).round(3)
    
    if save_outputs: safe_blk.to_csv(out_dir / "safety_bike_lane_block.csv", index=False)

    cent_meta = pd.read_csv(out_dir / "centroid_tract_computed.csv", dtype={"census_tract": str})
    cent_meta["census_tract"] = _Standardizer.normalize_tract(cent_meta["census_tract"])
    
    safe_tract = safe_blk.groupby("census_tract", as_index=False)[
        ["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]
    ].sum()
    
    safe_tract = cent_meta.merge(safe_tract, on="census_tract", how="left").fillna(0)
    safe_tract["bike_lane_ratio"] = _safe_ratio(safe_tract["total_bike_lane_length"], safe_tract["streets_leng"])
    safe_tract["protected_bike_lane_ratio"] = _safe_ratio(safe_tract["protected_bike_lane_length"], safe_tract["streets_leng"])
    
    service_tracts = set(cap_tract["census_tract"].unique())
    safe_service = safe_tract[safe_tract["census_tract"].isin(service_tracts)].copy()
    
    safe_norm = safe_service.copy()
    safe_norm["bike_lane_ratio_norm"] = _minmax_norm(safe_norm["bike_lane_ratio"]).round(5)
    safe_norm["protected_bike_lane_ratio_norm"] = _minmax_norm(safe_norm["protected_bike_lane_ratio"]).round(5)
    
    if save_outputs:
        safe_tract.to_csv(out_dir / "safety_bike_lane_tract.csv", index=False)
        safe_service.to_csv(out_dir / "safety_bike_lane_service_area.csv", index=False)
        safe_norm.to_csv(out_dir / "safety_bike_lane_norm_tract.csv", index=False)

    # ---------------- 6. Usage Metrics ----------------
    print("Computing Usage...")
    trips = _Standardizer.trips(pd.read_csv(trip_csv, engine="c"))
    trips["started_at"] = pd.to_datetime(trips["started_at"], errors="coerce")
    trips["ended_at"] = pd.to_datetime(trips["ended_at"], errors="coerce")
    
    t_min, t_max = trips["started_at"].min(), trips["started_at"].max()
    t_win_start = t_usage_start if t_usage_start else t_min
    t_win_end = t_usage_end if t_usage_end else t_max
    
    if t_win_start > t_max or t_win_end < t_min:
        print(f"⚠️ WARNING: Request Window ({t_win_start} - {t_win_end}) does not overlap Trip Data ({t_min} - {t_max}).")

    station_block_map = si[["station_id", "census_block"]].copy()
    
    if "start_station_id" in trips.columns:
        if "start_census_block" in trips.columns: trips.drop(columns=["start_census_block"], inplace=True)
        trips = trips.merge(station_block_map, left_on="start_station_id", right_on="station_id", how="left")
        trips = trips.rename(columns={"census_block": "start_census_block"}).drop(columns=["station_id"])
        
    if "end_station_id" in trips.columns:
        if "end_census_block" in trips.columns: trips.drop(columns=["end_census_block"], inplace=True)
        trips = trips.merge(station_block_map, left_on="end_station_id", right_on="station_id", how="left")
        trips = trips.rename(columns={"census_block": "end_census_block"}).drop(columns=["station_id"])

    if "start_census_block" not in trips.columns or trips["start_census_block"].isna().all():
        print("Fallback: Using coordinate geocoding for trips...")
        if "start_census_block" in trips.columns: trips.drop(columns=["start_census_block"], inplace=True)
        s_map = _GeoUtils.geocode_points(trips, "start_lng", "start_lat", blocks_gdf, blocks_id)
        trips = trips.merge(s_map, left_on=["start_lng", "start_lat"], right_on=["lon", "lat"], how="left") \
                     .rename(columns={"census_block": "start_census_block"}).drop(columns=["lon", "lat"])
        
        if "end_census_block" in trips.columns: trips.drop(columns=["end_census_block"], inplace=True)
        e_map = _GeoUtils.geocode_points(trips, "end_lng", "end_lat", blocks_gdf, blocks_id)
        trips = trips.merge(e_map, left_on=["end_lng", "end_lat"], right_on=["lon", "lat"], how="left") \
                     .rename(columns={"census_block": "end_census_block"}).drop(columns=["lon", "lat"])

    trips["start_census_block"] = trips["start_census_block"].astype(str).str.split('.').str[0]
    trips["end_census_block"] = trips["end_census_block"].astype(str).str.split('.').str[0]
    
    usage_trips = trips[(trips["started_at"] >= t_win_start) & (trips["started_at"] <= t_win_end)].copy()
    usage_trips = usage_trips[(usage_trips["ended_at"] - usage_trips["started_at"]).dt.total_seconds() <= 240*60]
    
    all_blocks = pd.concat([usage_trips["start_census_block"], usage_trips["end_census_block"]]).unique()
    all_blocks = [x for x in all_blocks if x != "nan"]

    _calc_usage(usage_trips, (t_win_start, t_win_end), all_blocks, "5min", "usage_5min_block", out_dir, save_outputs)
    u_hr_blk = _calc_usage(usage_trips, (t_win_start, t_win_end), all_blocks, "1h", "usage_hourly_block", out_dir, save_outputs)
    
    u_hr_tract = u_hr_blk.copy()
    u_hr_tract["census_tract"] = _Standardizer.tract_from_block(u_hr_tract["census_block"])
    u_hr_tract["census_tract"] = _Standardizer.normalize_tract(u_hr_tract["census_tract"])
    if ext_prefix: u_hr_tract = u_hr_tract[~u_hr_tract["census_tract"].str.startswith(ext_prefix)]
    
    u_hr_tract = u_hr_tract.groupby(["census_tract", "time_slot"], as_index=False)[
        ["trips_starting", "trips_ending"]
    ].sum()
    
    if save_outputs: u_hr_tract.to_csv(out_dir / "usage_hourly_tract_raw.csv", index=False)
    
    if tracts_to_remove:
        u_hr_tract = u_hr_tract[~u_hr_tract["census_tract"].isin(tracts_to_remove)]
        
    u_hr_tract["trips_starting_norm"] = _minmax_norm(u_hr_tract["trips_starting"]).round(5)
    u_hr_tract["trips_ending_norm"] = _minmax_norm(u_hr_tract["trips_ending"]).round(5)
    
    if save_outputs: u_hr_tract.to_csv(out_dir / "usage_norm_hourly_tract.csv", index=False)

    # ---------------- 7. Idle Simulation ----------------
    if not compute_idle_time:
        print("Skipping Idle Time.")
        return results

    print("Computing Idle Time...")
    t_i_start = t_idle_start if t_idle_start else t_u_start
    t_i_end = t_idle_end if t_idle_end else t_u_end
    
    idle_trips = trips[(trips["started_at"] >= t_i_start) & (trips["started_at"] <= t_i_end)].copy()
    if "rideable_type" in idle_trips.columns:
        idle_trips = _map_rideable_type_to_two_columns(idle_trips)
        has_rideable = True
    else:
        has_rideable = False

    ss_idle = ss_done[(ss_done["timestamp"] >= t_i_start) & (ss_done["timestamp"] <= t_i_end)].copy()
    ss_idle["time_slot"] = ss_idle["timestamp"].dt.floor("5min") - pd.Timedelta(minutes=5)
    
    ss_agg = ss_idle.groupby(["census_block", "time_slot"], as_index=False)[
        ["total_vehicle_available"]
    ].sum()
    
    flux = _calc_usage(idle_trips, (t_i_start, t_i_end), all_blocks, "5min", "temp_flux", out_dir, False)
    flux["vehicles_moved"] = flux["trips_starting"] + flux["trips_ending"]
    
    merged_idle = ss_agg.merge(flux, on=["census_block", "time_slot"], how="left").fillna(0)
    merged_idle["idle_vehicles"] = (merged_idle["total_vehicle_available"] - merged_idle["vehicles_moved"]).clip(lower=0)
    merged_idle["hour"] = merged_idle["time_slot"].dt.floor("1h")
    
    if save_outputs: merged_idle.to_csv(out_dir / "idle_merged_5min.csv", index=False)

    idle_results = []
    for block, block_df in merged_idle.groupby("census_block"):
        for hour, hour_df in block_df.groupby("hour"):
            hour_df = hour_df.sort_values("time_slot")
            virtual_vehicles = deque()
            finalized_durations = []

            for _, row in hour_df.iterrows():
                current_time = row["time_slot"]
                current_idle = int(row["idle_vehicles"])

                while current_idle > len(virtual_vehicles):
                    virtual_vehicles.append(current_time)
                while current_idle < len(virtual_vehicles):
                    start_t = virtual_vehicles.popleft()
                    duration = (current_time - start_t).total_seconds() / 60.0
                    finalized_durations.append(duration)

            final_time = hour + pd.Timedelta(hours=1)
            while virtual_vehicles:
                start_t = virtual_vehicles.popleft()
                duration = (final_time - start_t).total_seconds() / 60.0
                finalized_durations.append(duration)

            avg_idle = round(sum(finalized_durations) / len(finalized_durations), 2) if finalized_durations else 0.0
            idle_results.append({"census_block": block, "hour": hour, "avg_idle_time": avg_idle})

    idle_df = pd.DataFrame(idle_results)
    if save_outputs: idle_df.to_csv(out_dir / "idle_time_block.csv", index=False)
    
    if not idle_df.empty:
        idle_df["census_tract"] = _Standardizer.tract_from_block(idle_df["census_block"].astype(str))
        idle_df["census_tract"] = _Standardizer.normalize_tract(idle_df["census_tract"])
        
        if ext_prefix:
            idle_df = idle_df[~idle_df["census_tract"].str.startswith(ext_prefix)]
            
        idle_tract = idle_df.groupby(["census_tract", "hour"], as_index=False)["avg_idle_time"].mean()
        idle_tract["avg_idle_time_norm"] = _minmax_norm(idle_tract["avg_idle_time"]).round(5)
        
        if save_outputs:
            idle_tract.to_csv(out_dir / "idle_time_norm_tract.csv", index=False)

    print("Done.")
    return results

In [6]:
"""
=============================================================================
DOCKED BIKE SHARE UTILITY ANALYTICS PIPELINE (PRODUCTION v2)
=============================================================================
Updates:
- ADDED: Multi-file aggregation. If 'trip_csv' is a directory, all CSVs inside 
  are merged automatically (handles NYC's 5-file split).
- ADDED: Zip file support for 'trip_csv'.
- IMPROVED: Comments and structure for readability.
"""

from __future__ import annotations

import json
import warnings
import zipfile
from collections import deque
from pathlib import Path
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from shapely import wkt
from shapely.geometry import Point
from tqdm import tqdm

# Suppress specific warnings for cleaner logs
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# =============================================================================
# 1. GLOBAL CONFIGURATION
#    Define cities, file paths, and coordinate systems here.
# =============================================================================
CITY_CONFIG: Dict[str, Dict[str, Any]] = {
    "SF": {
        "use_api_fallback": True, # SF often needs API for boundary stations
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Usage\San_Fran_Baywheels\tl_2024_06_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\San_Fran_Baywheels\tl_2024_06_tract.shp",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Streets___Active_and_Retired_20250626 (1).csv",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\San_Fran_Baywheels\Bikelane.csv",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "crs": "EPSG:4326",
            "metric_crs": "EPSG:26910",   # UTM Zone 10N (Meters)
            "safety_type": "csv_wkt",
            "wkt_candidates": ("geometry", "shape", "line", "the_geom", "wkt", "geometry_wkt", "WKT", "geom"),
            "external_tract_prefix": None,
            "drop_staten_island": False,
        },
        "safety_rule": {
            "col_candidates": ("BARRIER", "FACILITY_T", "BUFFERED", "RAISED", "SYMBOLOGY"),
            "match_type": "not_empty_or_contains",
            "match_value": r"PROTECT|SEPARAT|CYCLETRACK|BARRIER|RAISED|BUFFER",
        },
    },

    "NJ": {
        "use_api_fallback": False, # Disabled to ignore NYC stations in NJ files
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\tl_2024_34_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\tl_2024_34_tract.shp",
            "centroid_csv": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NJ\centroid_tract_nj.csv",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\Tran_road.shp",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NJ\bike-lanes-2020-division-of-transportation.shp",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "crs": "EPSG:4326",
            "metric_crs": "EPSG:32618",
            "safety_type": "shp",
            "wkt_candidates": (),
            "external_tract_prefix": None,
            "drop_staten_island": False,
        },
        "safety_rule": {
            "col_candidates": ("type", "facility", "facilitycl", "class", "lane_type"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },

    "NYC": {
        "use_api_fallback": True,
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NYC\tl_2024_36_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\NYC\tl_2024_36_tract.shp",
            "centroid_csv": r"D:\Research Fellowship\Capacity_NYC\centroid_tract_computed.csv",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\CSCL_PlowNYC_20250619.csv",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\NYC\New_York_City_Bike_Routes_20250619.csv",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "crs": "EPSG:4326",
            "metric_crs": "EPSG:32618",   # UTM Zone 18N (Meters)
            "safety_type": "csv_wkt",
            "wkt_candidates": ("geometry", "the_geom", "wkt", "geometry_wkt", "WKT"),
            "external_tract_prefix": "34", # Exclude NJ
            "drop_staten_island": True,
        },
        "safety_rule": {
            "col_candidates": ("facilitycl", "facility", "class", "ft", "type"),
            "match_type": "equals",
            "match_value": "I",
        },
    },
    
    "PITT": {
        "use_api_fallback": True,
        "assets": {
            "census_blocks": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\tl_2024_42_tabblock20.shp",
            "tracts": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\tl_2024_42_tract.shp",
            "centroid_csv": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Capacity\Pitt\centroid_tract_pa.csv",
            "centerline": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Pittsburgh_Street_Centerline.shp",
            "bike_lanes": r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\Safety\Pitt\Bike Map\Bike Lanes.shp",
        },
        "geo": {
            "blocks_id": "GEOID20",
            "tract_id": "GEOID",
            "crs": "EPSG:4326",
            "metric_crs": "EPSG:32617",
            "safety_type": "shp",
            "wkt_candidates": (),
            "external_tract_prefix": None,
            "drop_staten_island": False,
        },
        "safety_rule": {
            "col_candidates": ("facility", "type", "class", "status", "lane_type", "BIKE_FACIL", "CATEGORY"),
            "match_type": "contains",
            "match_value": "PROTECT",
        },
    },
}


# =============================================================================
# 2. DATA HELPER (Handling File IO and Aggregation)
# =============================================================================
class DataHelper:
    """Helper class for reading data, standardizing columns, and normalization."""

    @staticmethod
    def pick_col(df: pd.DataFrame, candidates: Sequence[str], required: bool = False, label: str = "") -> Optional[str]:
        """Smartly selects a column from a list of potential aliases."""
        cols = set(df.columns)
        for c in candidates:
            if c in cols: return c
        if required:
            raise ValueError(f"Missing required col for {label}. Tried: {candidates}")
        return None

    @staticmethod
    def normalize_id(s: pd.Series) -> pd.Series:
        """Standardizes IDs: Strings, uppercase, trimmed, no decimals."""
        return s.astype(str).str.strip().str.upper().str.replace(r"\.0$", "", regex=True)

    @staticmethod
    def normalize_tract(s: pd.Series) -> pd.Series:
        """Ensures 11-digit Tract IDs (pads with leading zeros)."""
        return s.astype(str).str.split('.').str[0].str.zfill(11)

    @staticmethod
    def tract_from_block(s: pd.Series) -> pd.Series:
        """Extracts Tract ID from a full Block ID."""
        s = s.astype(str).str.split('.').str[0]
        return s.str[:11]

    @classmethod
    def load_stations(cls, path: Path) -> pd.DataFrame:
        """Loads Station Information and standardizes names."""
        df = pd.read_csv(path)
        mapping = {
            cls.pick_col(df, ["station_id", "stationId", "id", "Station ID"], True, "station_id"): "station_id",
            cls.pick_col(df, ["lat", "latitude", "y"], True, "lat"): "lat",
            cls.pick_col(df, ["lon", "lng", "longitude", "x"], True, "lon"): "lon",
            cls.pick_col(df, ["capacity", "dock_count", "num_docks"], True, "capacity"): "capacity"
        }
        out = df.rename(columns=mapping)
        out["station_id"] = cls.normalize_id(out["station_id"])
        
        # Preserve existing geo columns if they exist
        cb = cls.pick_col(out, ["census_block", "block_geoid", "GEOID20"])
        if cb and cb != "census_block": out = out.rename(columns={cb: "census_block"})
        
        ct = cls.pick_col(out, ["census_tract", "tract_geoid", "GEOID"])
        if ct and ct != "census_tract": out = out.rename(columns={ct: "census_tract"})
        
        return out

    @classmethod
    def load_trips(cls, path: Union[str, Path]) -> pd.DataFrame:
        """
        Loads Trip Data.
        - If 'path' is a file: Loads that single CSV (or ZIP).
        - If 'path' is a directory: Loads and aggregates ALL *.csv files inside (e.g., NYC split files).
        """
        path = Path(path)
        dfs = []
        
        print(f"   -> Reading trip data from: {path}")

        # 1. Gather all dataframes
        if path.is_dir():
            # Directory Mode: Read all CSVs
            files = sorted(path.glob("*.csv"))
            if not files: raise ValueError(f"No .csv files found in directory: {path}")
            
            print(f"   -> Found {len(files)} trip files. Aggregating...")
            for f in files:
                # low_memory=False avoids DtypeWarnings on large files
                dfs.append(pd.read_csv(f, engine="c", low_memory=False))
        
        elif path.suffix.lower() == '.zip':
            # Zip Mode: Read all CSVs inside archive
            print("   -> Detected ZIP file. Reading internal CSVs...")
            with zipfile.ZipFile(path) as z:
                for name in z.namelist():
                    if name.lower().endswith('.csv') and not name.startswith('__MACOSX'):
                        with z.open(name) as f:
                            dfs.append(pd.read_csv(f, low_memory=False))
        else:
            # Single File Mode
            dfs.append(pd.read_csv(path, engine="c", low_memory=False))
            
        # 2. Concatenate
        if not dfs: raise ValueError("No trip data found.")
        df = pd.concat(dfs, ignore_index=True) if len(dfs) > 1 else dfs[0]
        
        # 3. Standardize Columns
        mapping = {
            cls.pick_col(df, ["started_at", "start_time", "starttime", "Start Date"], True, "start"): "started_at",
            cls.pick_col(df, ["ended_at", "end_time", "stoptime", "End Date"], True, "end"): "ended_at"
        }
        
        opt_map = {
            cls.pick_col(df, ["rideable_type", "bike_type", "vehicle_type"]): "rideable_type",
            cls.pick_col(df, ["start_lat", "startLatitude", "StartLat"]): "start_lat",
            cls.pick_col(df, ["start_lng", "startLongitude", "StartLng"]): "start_lng",
            cls.pick_col(df, ["end_lat", "endLatitude", "EndLat"]): "end_lat",
            cls.pick_col(df, ["end_lng", "endLongitude", "EndLng"]): "end_lng",
            cls.pick_col(df, ["start_station_id", "Start Station Id"]): "start_station_id",
            cls.pick_col(df, ["end_station_id", "End Station Id"]): "end_station_id"
        }
        mapping.update({k: v for k, v in opt_map.items() if k})
        
        df = df.rename(columns=mapping)
        
        # Normalize IDs
        if "start_station_id" in df.columns:
            df["start_station_id"] = cls.normalize_id(df["start_station_id"])
        if "end_station_id" in df.columns:
            df["end_station_id"] = cls.normalize_id(df["end_station_id"])
            
        return df


# =============================================================================
# 3. GEO HELPER (Spatial Operations)
# =============================================================================
class GeoHelper:
    """Handles loading shapes, reprojection, spatial joins, and API lookups."""

    @staticmethod
    def load_network_lines(path: Path, kind: str, target_crs, candidates: tuple) -> gpd.GeoDataFrame:
        """Loads geometric lines (streets/lanes) and projects to Metric CRS."""
        if kind == "shp":
            gdf = gpd.read_file(path)
            if gdf.crs is None: raise ValueError(f"No CRS found in {path}")
            return gdf.to_crs(target_crs)
        
        if isinstance(candidates, str): candidates = (candidates,)
        df = pd.read_csv(path, engine="c")
        wkt_col = DataHelper.pick_col(df, candidates, True, "WKT Column")
        
        def safe_load(x):
            try: return wkt.loads(str(x))
            except: return None
            
        df["geometry"] = df[wkt_col].apply(safe_load)
        gdf = gpd.GeoDataFrame(df[df["geometry"].notna()], geometry="geometry", crs="EPSG:4326")
        return gdf.to_crs(target_crs)

    @staticmethod
    def geocode_points(df: pd.DataFrame, lon_col: str, lat_col: str, blocks_gdf: gpd.GeoDataFrame, id_col: str) -> pd.DataFrame:
        """Spatially joins points to Census Blocks."""
        points = df[[lon_col, lat_col]].dropna().drop_duplicates()
        if points.empty:
            return pd.DataFrame(columns=[lon_col, lat_col, "census_block"])

        gdf = gpd.GeoDataFrame(
            points, 
            geometry=[Point(xy) for xy in zip(points[lon_col], points[lat_col])],
            crs="EPSG:4326"
        )
        
        joined = gpd.sjoin(gdf, blocks_gdf[[id_col, "geometry"]], how="left", predicate="within")
        joined = joined.rename(columns={id_col: "census_block"})
        
        out = joined[[lon_col, lat_col, "census_block"]].copy()
        out = out.rename(columns={lon_col: "lon", lat_col: "lat"})
        return out

    @staticmethod
    def api_fill_blocks(df: pd.DataFrame, benchmark: str, vintage: str) -> pd.DataFrame:
        """Queries the US Census API for missing block IDs."""
        missing = df[df["census_block"].isna()][["lat", "lon"]].drop_duplicates()
        if missing.empty: return df
        
        # Limit API calls to prevent timeouts
        if len(missing) > 5000:
            print(f"⚠️ Warning: {len(missing)} missing blocks. Capping API fallback at 5000 points.")
            missing = missing.head(5000)

        def fetch(r):
            try:
                url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={r.lon}&y={r.lat}&benchmark={benchmark}&vintage={vintage}&format=json"
                js = requests.get(url, timeout=5).json()
                return js["result"]["geographies"]["2020 Census Blocks"][0]["GEOID"]
            except: return None

        tqdm.pandas(desc="API Geocoding Fallback")
        missing["new_block"] = missing.progress_apply(fetch, axis=1)
        
        merged = df.merge(missing, on=["lat", "lon"], how="left")
        if "new_block" in merged.columns:
            merged["census_block"] = merged["census_block"].fillna(merged["new_block"])
            merged = merged.drop(columns=["new_block"])
        return merged


# =============================================================================
# 4. METRIC HELPER (Calculations)
# =============================================================================
class MetricHelper:
    """Contains logic for Availability, Usage, and Normalization."""

    @staticmethod
    def normalize(series: pd.Series) -> pd.Series:
        """Min-Max Normalization."""
        s = pd.to_numeric(series, errors="coerce")
        mn, mx = s.min(skipna=True), s.max(skipna=True)
        if pd.isna(mn) or pd.isna(mx) or mx <= mn: return pd.Series(0.0, index=s.index)
        return (s - mn) / (mx - mn)

    @staticmethod
    def safe_ratio(numer: pd.Series, denom: pd.Series) -> pd.Series:
        """Safe Division (returns 0 instead of NaN/Inf)."""
        return numer.div(denom.where(denom > 0, other=pd.NA)).fillna(0)

    @staticmethod
    def calc_availability(df: pd.DataFrame, level: str, gran: str, out_dir: Path, save: bool, ext_prefix: str = None) -> Dict:
        """Calculates aggregated availability metrics."""
        geo = "census_block" if level == "block" else "census_tract"
        agg = df.groupby([geo, "timestamp"], as_index=False)[
            ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]
        ].sum()
        
        agg["time_slot"] = agg["timestamp"].dt.floor(gran)
        raw = agg.groupby([geo, "time_slot"], as_index=False).mean(numeric_only=True).round(0)
        for c in raw.columns: 
            if "available" in c: raw[c] = raw[c].astype(int)
                
        if level == "tract":
            raw["census_tract"] = DataHelper.normalize_tract(raw["census_tract"])
            if ext_prefix: raw = raw[~raw["census_tract"].str.startswith(ext_prefix)]

        norm = raw.copy()
        for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available", "total_vehicle_available"]:
            norm[f"{c}_norm"] = MetricHelper.normalize(norm[c]).round(5)

        if save:
            raw.to_csv(out_dir / f"availability__raw__{level}.csv", index=False)
            norm.to_csv(out_dir / f"availability__norm__{level}.csv", index=False)
        return {"raw": raw, "norm": norm}

    @staticmethod
    def calc_usage(trips: pd.DataFrame, time_range: Tuple, blocks: List, gran_str: str, out_name: str, out_dir: Path, save: bool) -> pd.DataFrame:
        """Calculates usage (starts/ends) using a full grid to preserve 0s."""
        # Note: 'slot_start' must already exist in trips df
        
        starts = trips.groupby(["start_census_block", "slot_start"]).size().rename("trips_starting")
        ends = trips.groupby(["end_census_block", "slot_end"]).size().rename("trips_ending")
        
        # Grid expansion
        slots = pd.date_range(start=time_range[0].floor(gran_str), end=time_range[1].ceil(gran_str), freq=gran_str)
        grid = pd.MultiIndex.from_product([blocks, slots], names=["census_block", "time_slot"]).to_frame(index=False)
        
        out = grid.merge(starts, left_on=["census_block", "time_slot"], right_index=True, how="left") \
                  .merge(ends, left_on=["census_block", "time_slot"], right_index=True, how="left")
                  
        out = out.fillna(0).astype({"trips_starting": int, "trips_ending": int})
        if save: out.to_csv(out_dir / f"{out_name}.csv", index=False)
        return out

    @staticmethod
    def map_rideable_type(trips: pd.DataFrame) -> pd.DataFrame:
        df = trips.copy()
        if "rideable_type" not in df.columns: return df
        t = df["rideable_type"].astype(str).str.lower()
        df["_is_ebike"] = t.str.contains("electric|ebike|e-bike|assist", na=False)
        return df


# =============================================================================
# 5. MAIN PIPELINE
# =============================================================================
def run_docked_all_utilities_single_function(
    *,
    city: str,
    station_status_txt: Union[str, Path],
    station_information_csv: Union[str, Path],
    trip_csv: Union[str, Path],
    output_dir: Optional[Union[str, Path]] = None,
    save_outputs: bool = True,
    
    # Global Time Window
    time_start: Optional[Union[str, pd.Timestamp]] = None,
    time_end: Optional[Union[str, pd.Timestamp]] = None,
    
    # Feature Flags
    compute_idle_time: bool = True,
    fill_missing_with_census_api: bool = True,
    
    # Overrides (Optional)
    availability_time_start: Optional[Union[str, pd.Timestamp]] = None,
    availability_time_end: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_start: Optional[Union[str, pd.Timestamp]] = None,
    usage_time_end: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_start: Optional[Union[str, pd.Timestamp]] = None,
    idle_time_end: Optional[Union[str, pd.Timestamp]] = None,
    
    # Settings
    availability_time_granularity: str = "1h",
    availability_group_level: str = "both",
    peak_time_slot: Optional[Union[str, pd.Timestamp]] = None,
    peak_metric: str = "total_vehicle_available",
    drop_staten_island: Optional[bool] = None,
    tracts_to_remove: Optional[Sequence[str]] = None,
    remove_tz_suffix: str = " EDT",
    census_geocoder_benchmark: str = "Public_AR_Census2020",
    census_geocoder_vintage: str = "Census2020_Current",
) -> Dict[str, Any]:
    
    # --- 1. SETUP ---
    city_key = city.strip().upper()
    aliases = {"PITTSBURGH": "PITT", "PIT": "PITT", "SANFRAN": "SF", "SAN FRANCISCO": "SF", "BAYWHEELS": "SF"}
    city_key = aliases.get(city_key, city_key)
    
    if city_key not in CITY_CONFIG:
        raise ValueError(f"City '{city_key}' not found in configuration.")
    
    cfg = CITY_CONFIG[city_key]
    out_dir = Path(output_dir or f"./{city_key}_ALL_IN_ONE_OUTPUTS")
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # Override API Flag based on City Config
    should_api_fill = cfg.get("use_api_fallback", fill_missing_with_census_api)
    
    # Time Norm
    ts_start = pd.to_datetime(time_start) if time_start else None
    ts_end = pd.to_datetime(time_end) if time_end else None
    
    t_avail_s = pd.to_datetime(availability_time_start) or ts_start
    t_avail_e = pd.to_datetime(availability_time_end) or ts_end
    t_usage_s = pd.to_datetime(usage_time_start) or ts_start
    t_usage_e = pd.to_datetime(usage_time_end) or ts_end
    t_idle_s = pd.to_datetime(idle_time_start) or ts_start
    t_idle_e = pd.to_datetime(idle_time_end) or ts_end

    results = {"city": city_key, "output_dir": str(out_dir)}

    # --- 2. ASSETS & STATIONS ---
    print(f"[{city_key}] Loading Spatial Assets...")
    blocks_id = str(cfg["geo"]["blocks_id"])
    blocks_gdf = gpd.read_file(cfg["assets"]["census_blocks"]).to_crs(epsg=4326)
    
    print(f"[{city_key}] Processing Station Data...")
    records = []
    with Path(station_status_txt).open("r", encoding="utf-8") as f:
        for line in f:
            if not line.strip(): continue
            try:
                js = json.loads(line)
                ts = list(js.keys())[0]
                for entry in js[ts]:
                    for vt in entry.get("vehicle_types_available", []):
                        entry[f"vehicle_type_{vt['vehicle_type_id']}_count"] = vt["count"]
                    entry.pop("vehicle_types_available", None)
                    entry["timestamp"] = ts
                    records.append(entry)
            except: continue
    
    ss_df = pd.DataFrame(records)
    ss_df["timestamp"] = pd.to_datetime(ss_df["timestamp"].astype(str).str.replace(remove_tz_suffix, "", regex=False), errors="coerce")
    ss_df["station_id"] = DataHelper.normalize_id(ss_df["station_id"])
    if save_outputs: ss_df.to_csv(out_dir / f"{city_key.lower()}_station_status_clean.csv", index=False)

    si = DataHelper.load_stations(Path(station_information_csv))
    
    # Block Assignment (Geocode if needed)
    if "census_block" not in si.columns or si["census_block"].isna().any():
        mapping = GeoHelper.geocode_points(si, "lon", "lat", blocks_gdf, blocks_id)
        si = si.merge(mapping, on=["lon", "lat"], how="left", suffixes=("", "_new"))
        if "census_block_new" in si.columns:
            si["census_block"] = si["census_block"].fillna(si["census_block_new"])
            si = si.drop(columns=["census_block_new"])
        
        if should_api_fill:
            si = GeoHelper.api_fill_blocks(si, census_geocoder_benchmark, census_geocoder_vintage)

    si["census_block"] = si["census_block"].astype(str).str.split('.').str[0]
    si["census_tract"] = DataHelper.normalize_tract(DataHelper.tract_from_block(si["census_block"]))
    
    if save_outputs: si.to_csv(out_dir / f"{city_key.lower()}_station_information_done.csv", index=False)

    # Station Status Merge
    ss_done = ss_df.merge(si[["station_id", "census_block", "census_tract"]], on="station_id", how="left")
    for c in ["num_bikes_available", "num_ebikes_available", "num_docks_available"]:
        ss_done[c] = pd.to_numeric(ss_done.get(c, 0), errors="coerce").fillna(0).astype(int)
    ss_done["total_vehicle_available"] = ss_done["num_bikes_available"] + ss_done["num_ebikes_available"]
    
    if save_outputs: ss_done.to_csv(out_dir / f"{city_key.lower()}_station_status_done.csv", index=False)

    # --- 3. METRICS ---
    print(f"[{city_key}] Computing Availability...")
    ss_filt = ss_done.copy()
    if t_avail_s: ss_filt = ss_filt[ss_filt["timestamp"] >= t_avail_s]
    if t_avail_e: ss_filt = ss_filt[ss_filt["timestamp"] < t_avail_e]
    
    ext = cfg["geo"]["external_tract_prefix"]
    avail_res = {}
    if availability_group_level in {"block", "both"}:
        avail_res["block"] = MetricHelper.calc_availability(ss_filt, "block", availability_time_granularity, out_dir, save_outputs)
    if availability_group_level in {"tract", "both"}:
        avail_res["tract"] = MetricHelper.calc_availability(ss_filt, "tract", availability_time_granularity, out_dir, save_outputs, ext)

    print(f"[{city_key}] Computing Capacity...")
    tract_gdf = gpd.read_file(cfg["assets"]["tracts"]).to_crs(epsg=4326)
    centroids = pd.DataFrame({
        "census_tract": DataHelper.normalize_tract(tract_gdf[cfg["geo"]["tract_id"]]),
        "centroid_lon": tract_gdf.geometry.centroid.x,
        "centroid_lat": tract_gdf.geometry.centroid.y
    })
    
    if cfg["geo"].get("drop_staten_island", False) and "COUNTYFP" in tract_gdf.columns:
        centroids = centroids[tract_gdf["COUNTYFP"].astype(str) != "085"]
    if save_outputs: centroids.to_csv(out_dir / "centroid_tract_computed.csv", index=False)

    cap_tract_agg = si.groupby("census_tract", as_index=False).agg(total_capacity=("capacity", "sum"), num_station=("station_id", "count"))
    cap_tract = centroids[["census_tract"]].merge(cap_tract_agg, on="census_tract", how="left").fillna(0)
    if ext: cap_tract = cap_tract[~cap_tract["census_tract"].str.startswith(ext)]
    
    cap_tract["total_capacity_norm"] = MetricHelper.normalize(cap_tract["total_capacity"]).round(5)
    cap_tract["num_station_norm"] = MetricHelper.normalize(cap_tract["num_station"]).round(5)
    
    if save_outputs: cap_tract.to_csv(out_dir / "capacity_tract_norm.csv", index=False)

    # Peak Capacity
    tract_av = avail_res["tract"]["norm"]
    peak_ts = peak_time_slot if peak_time_slot else tract_av.groupby("time_slot")[peak_metric].sum().idxmax()
    peak_data = tract_av[tract_av["time_slot"] == peak_ts][["census_tract", "total_vehicle_available", "num_docks_available"]]
    peak_data = peak_data.rename(columns={"total_vehicle_available": "vehicle_capacity", "num_docks_available": "dock_capacity"})
    
    cap_df = cap_tract.merge(peak_data, on="census_tract", how="left").fillna(0)
    cap_df["vehicle_capacity_norm"] = MetricHelper.normalize(cap_df["vehicle_capacity"]).round(5)
    cap_df["dock_capacity_norm"] = MetricHelper.normalize(cap_df["dock_capacity"]).round(5)
    cap_df["occupancy_rate"] = MetricHelper.safe_ratio(cap_df["vehicle_capacity"], cap_df["total_capacity"])
    cap_df["return_pressure"] = 1.0 - MetricHelper.safe_ratio(cap_df["dock_capacity"], cap_df["total_capacity"])
    
    if save_outputs: cap_df.to_csv(out_dir / "capacity_tract_with_vehicle_and_docks_norm.csv", index=False)

    print(f"[{city_key}] Computing Safety...")
    metric_crs = cfg["geo"]["metric_crs"]
    streets = GeoHelper.load_network_lines(Path(cfg["assets"]["centerline"]), cfg["geo"]["safety_type"], metric_crs, cfg["geo"].get("wkt_candidates", ()))
    lanes = GeoHelper.load_network_lines(Path(cfg["assets"]["bike_lanes"]), cfg["geo"]["safety_type"], metric_crs, cfg["geo"].get("wkt_candidates", ()))
    
    rule = cfg["safety_rule"]
    p_col = DataHelper.pick_col(lanes, list(rule.get("col_candidates", [])), False)
    if p_col:
        match_val = rule["match_value"]
        if rule["match_type"] == "contains": mask = lanes[p_col].astype(str).str.upper().str.contains(str(match_val).upper(), na=False)
        elif rule["match_type"] == "equals": mask = lanes[p_col].astype(str).str.upper() == str(match_val).upper()
        else: mask = lanes[p_col].astype(str).str.contains(match_val, case=False, regex=True)
        protected = lanes[mask]
    else: protected = lanes.iloc[0:0]

    blocks_metric = blocks_gdf.to_crs(metric_crs)
    def _len(lines, name):
        if lines.empty: return pd.DataFrame({"census_block": [], name: []})
        ov = gpd.overlay(lines, blocks_metric, how="intersection", keep_geom_type=False)
        ov["len"] = ov.geometry.length
        return ov.groupby(blocks_id)["len"].sum().reset_index().rename(columns={blocks_id: "census_block", "len": name})

    st = _len(streets, "streets_leng")
    bl = _len(lanes, "total_bike_lane_length")
    pr = _len(protected, "protected_bike_lane_length")
    
    safe = st.merge(bl, on="census_block", how="left").merge(pr, on="census_block", how="left").fillna(0)
    safe["census_tract"] = DataHelper.normalize_tract(DataHelper.tract_from_block(safe["census_block"]))
    safe["bike_lane_ratio"] = MetricHelper.safe_ratio(safe["total_bike_lane_length"], safe["streets_leng"]).round(3)
    safe["protected_bike_lane_ratio"] = MetricHelper.safe_ratio(safe["protected_bike_lane_length"], safe["streets_leng"]).round(3)
    
    if save_outputs: safe.to_csv(out_dir / "safety_bike_lane_block.csv", index=False)

    safe_tract = safe.groupby("census_tract", as_index=False)[["streets_leng", "total_bike_lane_length", "protected_bike_lane_length"]].sum()
    meta = pd.read_csv(out_dir / "centroid_tract_computed.csv", dtype={"census_tract": str})
    meta["census_tract"] = DataHelper.normalize_tract(meta["census_tract"])
    
    safe_tract = meta.merge(safe_tract, on="census_tract", how="left").fillna(0)
    safe_tract["bike_lane_ratio"] = MetricHelper.safe_ratio(safe_tract["total_bike_lane_length"], safe_tract["streets_leng"])
    safe_tract["protected_bike_lane_ratio"] = MetricHelper.safe_ratio(safe_tract["protected_bike_lane_length"], safe_tract["streets_leng"])
    
    safe_service = safe_tract[safe_tract["census_tract"].isin(set(cap_tract["census_tract"].unique()))].copy()
    safe_norm = safe_service.copy()
    safe_norm["bike_lane_ratio_norm"] = MetricHelper.normalize(safe_norm["bike_lane_ratio"]).round(5)
    safe_norm["protected_bike_lane_ratio_norm"] = MetricHelper.normalize(safe_norm["protected_bike_lane_ratio"]).round(5)
    
    if save_outputs:
        safe_tract.to_csv(out_dir / "safety_bike_lane_tract.csv", index=False)
        safe_service.to_csv(out_dir / "safety_bike_lane_service_area.csv", index=False)
        safe_norm.to_csv(out_dir / "safety_bike_lane_norm_tract.csv", index=False)

    print(f"[{city_key}] Computing Usage...")
    trips = DataHelper.load_trips(trip_csv) # Supports dir/zip aggregation
    trips["started_at"] = pd.to_datetime(trips["started_at"], errors="coerce")
    trips["ended_at"] = pd.to_datetime(trips["ended_at"], errors="coerce")
    
    t_min, t_max = trips["started_at"].min(), trips["started_at"].max()
    t_win_s, t_win_e = (t_usage_s if t_usage_s else t_min), (t_usage_e if t_usage_e else t_max)
    if t_win_s > t_max or t_win_e < t_min:
        print(f"⚠️ WARNING: Request Window ({t_win_s}-{t_win_e}) does not overlap Trip Data ({t_min}-{t_max}).")
    
    trips = trips[(trips["started_at"] >= t_win_s) & (trips["started_at"] <= t_win_e)].copy()
    
    # Station Mapping (Primary)
    sm = si[["station_id", "census_block"]].copy()
    if "start_station_id" in trips.columns:
        if "start_census_block" in trips.columns: trips.drop(columns=["start_census_block"], inplace=True)
        trips = trips.merge(sm, left_on="start_station_id", right_on="station_id", how="left").rename(columns={"census_block": "start_census_block"}).drop(columns=["station_id"])
    if "end_station_id" in trips.columns:
        if "end_census_block" in trips.columns: trips.drop(columns=["end_census_block"], inplace=True)
        trips = trips.merge(sm, left_on="end_station_id", right_on="station_id", how="left").rename(columns={"census_block": "end_census_block"}).drop(columns=["station_id"])

    # Fallback Geocoding
    if "start_census_block" not in trips.columns or trips["start_census_block"].isna().mean() > 0.5:
        print("   -> Fallback: Geocoding Trips...")
        s_map = GeoHelper.geocode_points(trips, "start_lng", "start_lat", blocks_gdf, blocks_id)
        if "start_census_block" in trips.columns: trips.drop(columns=["start_census_block"], inplace=True)
        trips = trips.merge(s_map, left_on=["start_lng", "start_lat"], right_on=["lon", "lat"], how="left").rename(columns={"census_block": "start_census_block"}).drop(columns=["lon", "lat"])
        
        e_map = GeoHelper.geocode_points(trips, "end_lng", "end_lat", blocks_gdf, blocks_id)
        if "end_census_block" in trips.columns: trips.drop(columns=["end_census_block"], inplace=True)
        trips = trips.merge(e_map, left_on=["end_lng", "end_lat"], right_on=["lon", "lat"], how="left").rename(columns={"census_block": "end_census_block"}).drop(columns=["lon", "lat"])

    trips["start_census_block"] = trips["start_census_block"].astype(str).str.split('.').str[0]
    trips["end_census_block"] = trips["end_census_block"].astype(str).str.split('.').str[0]
    
    # Calculate Slots & Grid
    all_blocks = pd.concat([trips["start_census_block"], trips["end_census_block"]]).unique()
    all_blocks = [x for x in all_blocks if x != "nan"]
    
    trips["slot_start"] = trips["started_at"].dt.floor("5min")
    trips["slot_end"] = trips["ended_at"].dt.floor("5min")
    
    MetricHelper.calc_usage(trips, (t_win_s, t_win_e), all_blocks, "5min", "usage_5min_block", out_dir, save_outputs)
    u_hr = MetricHelper.calc_usage(trips, (t_win_s, t_win_e), all_blocks, "1h", "usage_hourly_block", out_dir, save_outputs)
    
    u_hr["census_tract"] = DataHelper.normalize_tract(DataHelper.tract_from_block(u_hr["census_block"]))
    if ext: u_hr = u_hr[~u_hr["census_tract"].str.startswith(ext)]
    
    u_tract = u_hr.groupby(["census_tract", "time_slot"], as_index=False)[["trips_starting", "trips_ending"]].sum()
    if tracts_to_remove: u_tract = u_tract[~u_tract["census_tract"].isin(tracts_to_remove)]
    
    u_tract["trips_starting_norm"] = MetricHelper.normalize(u_tract["trips_starting"]).round(5)
    u_tract["trips_ending_norm"] = MetricHelper.normalize(u_tract["trips_ending"]).round(5)
    
    if save_outputs:
        u_tract.to_csv(out_dir / "usage_hourly_tract_raw.csv", index=False)
        u_tract.to_csv(out_dir / "usage_norm_hourly_tract.csv", index=False)

    if compute_idle_time:
        print(f"[{city_key}] Computing Idle Time...")
        t_i_s, t_i_e = (t_idle_s if t_idle_s else t_win_s), (t_idle_e if t_idle_e else t_win_e)
        idle_trips = trips[(trips["started_at"] >= t_i_s) & (trips["started_at"] <= t_i_e)].copy()
        if "rideable_type" in idle_trips.columns:
            idle_trips = MetricHelper.map_rideable_type(idle_trips)
            has_rideable = True
        else: has_rideable = False

        ss_idle = ss_done[(ss_done["timestamp"] >= t_i_s) & (ss_done["timestamp"] <= t_i_e)].copy()
        ss_idle["time_slot"] = ss_idle["timestamp"].dt.floor("5min") - pd.Timedelta(minutes=5)
        
        inv = ss_idle.groupby(["census_block", "time_slot"], as_index=False)[["total_vehicle_available"]].sum()
        flux = MetricHelper.calc_usage(idle_trips, (t_i_s, t_i_e), all_blocks, "5min", "temp_flux", out_dir, False)
        flux["vehicles_moved"] = flux["trips_starting"] + flux["trips_ending"]
        
        if has_rideable:
            ts = idle_trips.groupby(["start_census_block", "slot_start", "_is_ebike"]).size().unstack(fill_value=0).reset_index()
            ts = ts.rename(columns={"start_census_block": "census_block", "slot_start": "time_slot", False: "trips_starting_bike", True: "trips_starting_ebike"})
            te = idle_trips.groupby(["end_census_block", "slot_end", "_is_ebike"]).size().unstack(fill_value=0).reset_index()
            te = te.rename(columns={"end_census_block": "census_block", "slot_end": "time_slot", False: "trips_ending_bike", True: "trips_ending_ebike"})
            flux = flux.merge(ts, on=["census_block", "time_slot"], how="left").merge(te, on=["census_block", "time_slot"], how="left").fillna(0)
            for c in ["trips_starting_bike", "trips_starting_ebike", "trips_ending_bike", "trips_ending_ebike"]:
                if c in flux.columns: flux[c] = flux[c].astype(int)
            flux["vehicles_moved_bike"] = flux.get("trips_starting_bike",0) + flux.get("trips_ending_bike",0)
            flux["vehicles_moved_ebike"] = flux.get("trips_starting_ebike",0) + flux.get("trips_ending_ebike",0)

        m = inv.merge(flux, on=["census_block", "time_slot"], how="left").fillna(0)
        m["idle_vehicles"] = (m["total_vehicle_available"] - m["vehicles_moved"]).clip(lower=0)
        m["hour"] = m["time_slot"].dt.floor("1h")
        if save_outputs: m.to_csv(out_dir / "idle_merged_5min.csv", index=False)
        
        idle_res = []
        for block, b_df in m.groupby("census_block"):
            for hour, h_df in b_df.groupby("hour"):
                h_df = h_df.sort_values("time_slot")
                pool, durations = deque(), []
                for _, row in h_df.iterrows():
                    curr_t, cnt = row["time_slot"], int(row["idle_vehicles"])
                    while cnt > len(pool): pool.append(curr_t)
                    while cnt < len(pool):
                        start_t = pool.popleft()
                        durations.append((curr_t - start_t).total_seconds() / 60.0)
                final_t = hour + pd.Timedelta(hours=1)
                while pool:
                    start_t = pool.popleft()
                    durations.append((final_t - start_t).total_seconds() / 60.0)
                avg = round(sum(durations) / len(durations), 2) if durations else 0.0
                idle_res.append({"census_block": block, "hour": hour, "avg_idle_time": avg})
        
        idle_df = pd.DataFrame(idle_res)
        if save_outputs: idle_df.to_csv(out_dir / "idle_time_block.csv", index=False)
        
        if not idle_df.empty:
            idle_df["census_tract"] = DataHelper.normalize_tract(DataHelper.tract_from_block(idle_df["census_block"].astype(str)))
            if ext: idle_df = idle_df[~idle_df["census_tract"].str.startswith(ext)]
            i_tract = idle_df.groupby(["census_tract", "hour"], as_index=False)["avg_idle_time"].mean()
            i_tract["avg_idle_time_norm"] = MetricHelper.normalize(i_tract["avg_idle_time"]).round(5)
            if save_outputs: i_tract.to_csv(out_dir / "idle_time_norm_tract.csv", index=False)

    print(f"--- Done. Outputs saved to {out_dir} ---")
    return results

In [ ]:
# Example Call for San Francisco Docked
out_docked = run_docked_all_utilities_single_function(
    city="NYC", # NYC / NJ / PITT / SF
    
    # Inputs
    station_status_txt=r"D:\Research Fellowship\Summer Research Stuff\Collected Data\Week 1\09-June\nyc_docked_station_status_6_9.txt",
    station_information_csv=r"D:\Research Fellowship\Summer Research Stuff\Clean_Utilities\GBFS_Census_Tract\NYC\NYC station information 06_09.csv",
    trip_csv=r"D:\Research Fellowship\Summer Research Stuff\202506-citibike-tripdata", # Use a Folder if there are multiple files or use a single file if there is one.
    
    # Output Location
    output_dir="NYC_Docked_Output_v2",
    
    # Global Time Window (Propagates to all utilities)
    time_start="2025-06-09 00:00:00",
    time_end="2025-06-09 23:00:00",
    
    # Optional: Disable idle simulation if it takes too long
    compute_idle_time=True
)

[NYC] Loading Spatial Assets...
[NYC] Processing Station Data...


API Geocoding Fallback: 100%|██████████| 84/84 [01:07<00:00,  1.24it/s]


[NYC] Computing Availability...
[NYC] Computing Capacity...
[NYC] Computing Safety...
[NYC] Computing Usage...
   -> Reading trip data from: D:\Research Fellowship\Summer Research Stuff\202506-citibike-tripdata
   -> Found 5 trip files. Aggregating...
   -> Fallback: Geocoding Trips...
[NYC] Computing Idle Time...
--- Done. Outputs saved to NYC_Docked_Output_v2 ---
